In [1]:
import create_population
import preprocessing
from feature_engineering import FeatureCreator
from feature_engineering_prediction import PredictionFeatureCreator

import prediction

import pandas as pd


import html5lib

from train_lightgbm_rank_niti import Trainer_lightgbm_rank_niti
from train_lightgbm_time import Trainer_lightgbm_time
from train_lightgbm_rank_kaiki import Trainer_lightgbm_rank_kaiki


from evaluation_lightgbm_rank_niti import Evaluator_lightgbm_rank_niti
from evaluation_lightgbm_time_kaiki import Evaluator_lightgbm_time_kaiki
from evaluation_lightgbm_rank_kaiki import Evaluator_lightgbm_rank_kaiki
from evaluation_pop import Evaluator_pop

%load_ext autoreload

In [2]:
%autoreload

In [3]:
#脚質の%をだすfuture
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 1000)
pd.reset_option('display.precision')
pd.reset_option('display.float_format')

In [4]:
population = create_population.create(from_="2023-01-01", to_="2023-12-31")


In [293]:
population

,race_id,date,horse_id
0,202306010101,2023-01-05,2020103575
1,202306010101,2023-01-05,2020107073
2,202306010101,2023-01-05,2020102562
3,202306010101,2023-01-05,2020106345
4,202306010101,2023-01-05,2020100039
...,...,...,...
46207,202309050912,2023-12-28,2017104873
46208,202309050912,2023-12-28,2019100653
46209,202309050912,2023-12-28,2017106240
46210,202309050912,2023-12-28,2018103205


In [ ]:
#まずpropreのhorseにて、コーナー順位から脚質を割り出す

In [36]:
import json
from pathlib import Path

import pandas as pd
import numpy as np
import ast
COMMON_DATA_DIR = Path("..", "..", "common", "data")
RAWDF_DIR = COMMON_DATA_DIR / "rawdf"
MAPPING_DIR = COMMON_DATA_DIR / "mapping"
POPULATION_DIR = Path("..", "data", "00_population")
OUTPUT_DIR = Path("..", "data", "01_preprocessed")
OUTPUT_DIR.mkdir(exist_ok=True, parents=True)

# カテゴリ変数を数値に変換するためのマッピング
with open(MAPPING_DIR / "sex.json", "r") as f:
    sex_mapping = json.load(f)
with open(MAPPING_DIR / "race_type.json", "r") as f:
    race_type_mapping = json.load(f)
with open(MAPPING_DIR / "around.json", "r") as f:
    around_mapping = json.load(f)
with open(MAPPING_DIR / "weather.json", "r") as f:
    weather_mapping = json.load(f)
with open(MAPPING_DIR / "ground_state.json", "r") as f:
    ground_state_mapping = json.load(f)
with open(MAPPING_DIR / "race_class.json", "r") as f:
    race_class_mapping = json.load(f)
with open(MAPPING_DIR / "place.json", "r") as f:
    place_mapping = json.load(f)

In [37]:
#horse_id直す
population_dir = POPULATION_DIR
populaton_filename = "population.csv"
input_dir = RAWDF_DIR
output_dir = OUTPUT_DIR
input_filename = "horse_results.csv"
output_filename = "horse_results.csv"
race_type_mapping = race_type_mapping
weather_mapping = weather_mapping
ground_state_mapping = ground_state_mapping
race_class_mapping = race_class_mapping
place_mapping = place_mapping

In [98]:
population = pd.read_csv(population_dir / populaton_filename, sep="\t")
# df = pd.read_csv(input_dir / input_filename, sep="\t").query(
#     "horse_id in @population['horse_id']"
# )
# クエリでリストを直接使用
population_horse_ids = population['horse_id'].tolist()

df = pd.read_csv(input_dir / input_filename, sep="\t").query(
    "horse_id in @population_horse_ids"
)
    

df["rank"] = pd.to_numeric(df["着順"], errors="coerce")
df.dropna(subset=["rank"], inplace=True)
df["date"] = pd.to_datetime(df["日付"])
df["weather"] = df["天気"].map(weather_mapping)
df["race_type"] = df["距離"].str[0].map(race_type_mapping)
df = df[df["race_type"] != 2]
df["course_len"] = df["距離"].str.extract(r"(\d+)").astype(int)
df["ground_state"] = df["馬場"].map(ground_state_mapping)
df["rank_diff"] = df["着差"].map(lambda x: 0 if x < 0 else x)
df["prize"] = df["賞金"].fillna(0)
df["umaban"] = df["馬番"].astype(int)

df["wakuban"] = df["枠番"].fillna(0).astype(int)


df["nobori"] = df["上り"].astype(float)

regex_race_class = "|".join(race_class_mapping)
df["race_class"] = (
    df["レース名"].str.extract(rf"({regex_race_class})")[0].map(race_class_mapping)
)
df["time"] = pd.to_datetime(df["タイム"], format="%M:%S.%f", errors="coerce")
df["time"] = (
    df["time"].dt.minute * 60
    + df["time"].dt.second
    + df["time"].dt.microsecond / 1000000
)
df["win"] = (df["rank"] == 1).astype(int)
df["rentai"] = (df["rank"] <= 2).astype(int)
df["show"] = (df["rank"] <= 3).astype(int)
df["place"] = df["開催"].str.extract(r"(\D+)")[0].map(place_mapping)
df.dropna(subset=["place"], inplace=True)    
df.rename(columns={"頭数": "n_horses"}, inplace=True)

pase_cols = df['ペース'].str.split('-', expand=True)
pase_cols.columns = [f'pase_{i+1}' for i in range(pase_cols.shape[1])]

# データ型変換と型確認
pase_cols = pase_cols.apply(pd.to_numeric, errors='coerce').astype('float64')

# 元のデータフレームと結合
df = pd.concat([df, pase_cols], axis=1)

# NoneをNaNに置き換え
# df = df.where(pd.notnull(df), np.nan)
df["place"] = df["place"].astype(int)
df["race_class"] = df["race_class"].astype(int)
df["ground_state"] = df["ground_state"].astype(int)
df["weather"] = df["weather"].astype(int)   
# コーナー通過順を分割して列を作成
corner_cols = df['通過'].str.split('-', expand=True)
corner_cols.columns = [f'corner_{i+1}' for i in range(corner_cols.shape[1])]
# オブジェクト型のデータを整数型に変換する
corner_cols = corner_cols.apply(pd.to_numeric, errors='coerce').astype('Int64')  
# nullable int型を指定
# 元のデータフレームと結合
df = pd.concat([df, corner_cols], axis=1)

# NoneをNaNに置き換え
df = df.where(pd.notnull(df), np.nan)

# rank / n_horses の特徴量を作成（欠損値を含む行はNaNに設定）
df["rank_per_horse"] = df["rank"].where(df["rank"].notna(), np.nan) / df["n_horses"].where(df["n_horses"].notna(), np.nan)

# corner_1 / n_horses の特徴量を作成（欠損値を含む行はNaNに設定）
df["corner_1_per_horse"] = df["corner_1"].where(df["corner_1"].notna(), np.nan) / df["n_horses"].where(df["n_horses"].notna(), np.nan)

df["corner_2_per_horse"] = df["corner_2"].where(df["corner_2"].notna(), np.nan) / df["n_horses"].where(df["n_horses"].notna(), np.nan)
df["corner_3_per_horse"] = df["corner_3"].where(df["corner_3"].notna(), np.nan) / df["n_horses"].where(df["n_horses"].notna(), np.nan)
df["corner_4_per_horse"] = df["corner_4"].where(df["corner_4"].notna(), np.nan) / df["n_horses"].where(df["n_horses"].notna(), np.nan)
df["time_courselen"] = df["time"] / df["course_len"]

In [99]:
#脚質
def determine_race_position(row):
    # 最終コーナーを決定（corner_1, 2, 3, 4 のいずれか）
    if pd.notna(row['corner_4']):
        final_corner = 'corner_4'
    elif pd.notna(row['corner_3']):
        final_corner = 'corner_3'
    elif pd.notna(row['corner_2']):
        final_corner = 'corner_2'
    elif pd.notna(row['corner_1']):
        final_corner = 'corner_1'
    else:
        return None  # すべて欠損値の場合

    # 逃げ判定：最終コーナー以外のコーナーを1位で通過した場合
    if (pd.notna(row['corner_1']) and row['corner_1'] == 1 or 
        pd.notna(row['corner_2']) and row['corner_2'] == 1 or 
        pd.notna(row['corner_3']) and row['corner_3'] == 1 or 
        pd.notna(row['corner_4']) and row['corner_4'] == 1) and pd.notna(row[final_corner]) and row[final_corner] != 1:
        return 1  # 逃げ

    # 先行判定：最終コーナーを4位以内で通過
    if pd.notna(row[final_corner]) and row[final_corner] <= 4:
        return 2  # 先行

    # 差し判定：出走頭数に応じて
    if row['n_horses'] >= 8 and pd.notna(row[final_corner]) and row[final_corner] <= (row['n_horses'] * 2) // 3:
        return 3  # 差し
    elif row['n_horses'] < 8:
        return 4  # 差しなし、追込

    # 追込判定：上記のいずれにも該当しない
    return 4  # 追込

# dfの各レースについて脚質を決定
df['race_position'] = df.apply(determine_race_position, axis=1)
df = df.where(pd.notnull(df), np.nan)
# 結果を表示
df[['corner_1', 'corner_2', 'corner_3', 'corner_4', 'n_horses', 'race_position']]


,corner_1,corner_2,corner_3,corner_4,n_horses,race_position
924,11,10,13,12,15.0,4
925,13,13,11,11,15.0,4
926,9,9,7,6,13.0,3
927,14,14,14,14,15.0,4
928,11,11,5,5,15.0,3
...,...,...,...,...,...,...
1077785,8,9,6,7,14.0,3
1077786,9,9,<NA>,<NA>,14.0,3
1077795,1,1,1,2,11.0,1
1077796,14,12,9,8,14.0,3


In [207]:
#脚質の%をだすfuture
from pathlib import Path

from tqdm.notebook import tqdm
import json
import re
from pathlib import Path

import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from tqdm.notebook import tqdm
from webdriver_manager.chrome import ChromeDriverManager
from io import StringIO 

DATA_DIR = Path("..", "data")
POPULATION_DIR = DATA_DIR / "00_population"
INPUT_DIR = DATA_DIR / "01_preprocessed"
OUTPUT_DIR = DATA_DIR / "02_features"
OUTPUT_DIR.mkdir(exist_ok=True, parents=True)

population_dir = POPULATION_DIR
poplation_filename = "population.csv"
input_dir = INPUT_DIR
results_filename = "results.csv"
race_info_filename = "race_info.csv"
horse_results_filename = "horse_results.csv"
jockey_leading_filename = "jockey_leading.csv"
trainer_leading_filename = "trainer_leading.csv"
peds_filename = "peds.csv"
sire_leading_filename = "sire_leading.csv"
output_dir = OUTPUT_DIR
output_filename = "features.csv"
population = pd.read_csv(population_dir / poplation_filename, sep="\t")
results = pd.read_csv(input_dir / results_filename, sep="\t")
race_info = pd.read_csv(input_dir / race_info_filename, sep="\t")
horse_results = pd.read_csv(input_dir / horse_results_filename, sep="\t")
jockey_leading = pd.read_csv(input_dir / jockey_leading_filename, sep="\t")
trainer_leading = pd.read_csv(
    input_dir / trainer_leading_filename, sep="\t"
)
peds = pd.read_csv(input_dir / peds_filename, sep="\t")
sire_leading = pd.read_csv(input_dir / sire_leading_filename, sep="\t")
output_dir = output_dir
output_filename = output_filename
agg_horse_per_group_cols_dfs = {}

In [41]:
#脚質の%をだすfuture
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 1000)
pd.reset_option('display.precision')
pd.reset_option('display.float_format')

In [100]:

def calculate_race_position_percentage(group, n_race):
    """
    過去nレースにおける脚質割合を計算する
    """
    # 過去nレースに絞る
    past_races = group.head(n_race)
    
    # 各脚質のカウント
    counts = past_races['race_position'].value_counts(normalize=True).to_dict()
    
    # 結果を辞書形式で返す（割合が存在しない場合は0.0を補完）
    return {
        "escape": counts.get(1, 0.0),
        "taking_lead": counts.get(2, 0.0),
        "in_front": counts.get(3, 0.0),
        "pursuit": counts.get(4, 0.0),
    }

# 過去nレースのリスト
n_race_list = [1, 3, 5, 10]
baselog = (
    population.merge(
        race_info[["race_id", "course_len", "race_type"]], on="race_id"
    )
    .merge(
        df,
        on=["horse_id", "course_len", "race_type"],
        suffixes=("", "_horse"),
    )
    .query("date_horse < date")
    .sort_values("date_horse", ascending=False)
)

# 集計用データフレームの初期化
merged_df = population.copy()
# grouped_dfを適用して計算
grouped_df = baselog.groupby(["race_id", "horse_id"])
# 各過去nレースの割合を計算して追加
for n_race in n_race_list:
    
    position_percentage = grouped_df.apply(
        lambda group: calculate_race_position_percentage(group, n_race=n_race),
        include_groups=False,  # グループ列を除外
    )
    
    # 結果をデータフレームとして展開
    position_percentage_df = position_percentage.apply(pd.Series).reset_index()
    position_percentage_df.rename(
        columns={
            "escape": f"escape_{n_race}races",
            "taking_lead": f"taking_lead_{n_race}races",
            "in_front": f"in_front_{n_race}races",
            "pursuit": f"pursuit_{n_race}races",
        },
        inplace=True,
    )
    
    # 結果をマージ
    merged_df = merged_df.merge(position_percentage_df, on=["race_id", "horse_id"], how="left")





In [43]:
# 結果を表示
merged_df.head(50)

,race_id,date,horse_id,escape_1races,taking_lead_1races,in_front_1races,pursuit_1races,escape_3races,taking_lead_3races,in_front_3races,pursuit_3races,escape_5races,taking_lead_5races,in_front_5races,pursuit_5races,escape_10races,taking_lead_10races,in_front_10races,pursuit_10races
0,202306010101,2023-01-05,2020103575,0.0,1.0,0.0,0.0,0.0,1.000000,0.000000,0.000000,0.0,1.000000,0.000000,0.000000,0.0,1.000000,0.000000,0.000000
1,202306010101,2023-01-05,2020107073,0.0,1.0,0.0,0.0,0.0,0.500000,0.500000,0.000000,0.0,0.500000,0.500000,0.000000,0.0,0.500000,0.500000,0.000000
2,202306010101,2023-01-05,2020102562,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,202306010101,2023-01-05,2020106345,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,202306010101,2023-01-05,2020100039,0.0,0.0,1.0,0.0,0.0,0.333333,0.666667,0.000000,0.0,0.333333,0.666667,0.000000,0.0,0.333333,0.666667,0.000000
5,202306010101,2023-01-05,2020104817,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,202306010101,2023-01-05,2020104295,0.0,0.0,1.0,0.0,0.0,0.000000,1.000000,0.000000,0.0,0.000000,1.000000,0.000000,0.0,0.000000,1.000000,0.000000
7,202306010101,2023-01-05,2020105491,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,202306010101,2023-01-05,2020104358,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,202306010101,2023-01-05,2020105324,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [101]:
# dominant_position_category を計算
position_columns_5 = ["escape_5races", "taking_lead_5races", "in_front_5races", "pursuit_5races"]
position_columns_3 = ["escape_3races", "taking_lead_3races", "in_front_3races", "pursuit_3races"]
position_columns_1 = ["escape_1races", "taking_lead_1races", "in_front_1races", "pursuit_1races"]


def determine_dominant_position(row):
    # 5racesの中から計算
    if not row[position_columns_5].isnull().all():
        max_column = row[position_columns_5].idxmax()
        return {
            "escape_5races": 1,
            "taking_lead_5races": 2,
            "in_front_5races": 3,
            "pursuit_5races": 4,
        }[max_column]
    # 3racesの中から計算
    elif not row[position_columns_3].isnull().all():
        max_column = row[position_columns_3].idxmax()
        return {
            "escape_3races": 1,
            "taking_lead_3races": 2,
            "in_front_3races": 3,
            "pursuit_3races": 4,
        }[max_column]
    # 1racesの中から計算
    elif not row[position_columns_1].isnull().all():
        max_column = row[position_columns_1].idxmax()
        return {
            "escape_1races": 1,
            "taking_lead_1races": 2,
            "in_front_1races": 3,
            "pursuit_1races": 4,
        }[max_column]
    # すべて欠損の場合は1を返す
    else:
        return 2

# 各行に対して dominant_position_category を適用
merged_df["dominant_position_category"] = merged_df.apply(determine_dominant_position, axis=1)


In [56]:
merged_df.head(50)

,race_id,date,horse_id,escape_1races,taking_lead_1races,in_front_1races,pursuit_1races,escape_3races,taking_lead_3races,in_front_3races,pursuit_3races,escape_5races,taking_lead_5races,in_front_5races,pursuit_5races,escape_10races,taking_lead_10races,in_front_10races,pursuit_10races,dominant_position_category
0,202306010101,2023-01-05,2020103575,0.0,1.0,0.0,0.0,0.0,1.000000,0.000000,0.000000,0.0,1.000000,0.000000,0.000000,0.0,1.000000,0.000000,0.000000,2
1,202306010101,2023-01-05,2020107073,0.0,1.0,0.0,0.0,0.0,0.500000,0.500000,0.000000,0.0,0.500000,0.500000,0.000000,0.0,0.500000,0.500000,0.000000,2
2,202306010101,2023-01-05,2020102562,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
3,202306010101,2023-01-05,2020106345,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
4,202306010101,2023-01-05,2020100039,0.0,0.0,1.0,0.0,0.0,0.333333,0.666667,0.000000,0.0,0.333333,0.666667,0.000000,0.0,0.333333,0.666667,0.000000,3
5,202306010101,2023-01-05,2020104817,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
6,202306010101,2023-01-05,2020104295,0.0,0.0,1.0,0.0,0.0,0.000000,1.000000,0.000000,0.0,0.000000,1.000000,0.000000,0.0,0.000000,1.000000,0.000000,3
7,202306010101,2023-01-05,2020105491,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
8,202306010101,2023-01-05,2020104358,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
9,202306010101,2023-01-05,2020105324,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2


In [47]:
merged_df

,race_id,date,horse_id,escape_1races,taking_lead_1races,in_front_1races,pursuit_1races,escape_3races,taking_lead_3races,in_front_3races,pursuit_3races,escape_5races,taking_lead_5races,in_front_5races,pursuit_5races,escape_10races,taking_lead_10races,in_front_10races,pursuit_10races,dominant_position_category
0,202306010101,2023-01-05,2020103575,0.0,1.0,0.0,0.0,0.0,1.000000,0.000000,0.000000,0.0,1.000000,0.000000,0.0,0.0,1.000000,0.000000,0.000000,2
1,202306010101,2023-01-05,2020107073,0.0,1.0,0.0,0.0,0.0,0.500000,0.500000,0.000000,0.0,0.500000,0.500000,0.0,0.0,0.500000,0.500000,0.000000,2
2,202306010101,2023-01-05,2020102562,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
3,202306010101,2023-01-05,2020106345,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
4,202306010101,2023-01-05,2020100039,0.0,0.0,1.0,0.0,0.0,0.333333,0.666667,0.000000,0.0,0.333333,0.666667,0.0,0.0,0.333333,0.666667,0.000000,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47667,202309050912,2023-12-28,2017104873,0.0,0.0,0.0,1.0,0.0,0.000000,0.333333,0.666667,0.0,0.000000,0.200000,0.8,0.0,0.000000,0.200000,0.800000,4
47668,202309050912,2023-12-28,2019100653,0.0,0.0,1.0,0.0,0.0,0.000000,1.000000,0.000000,0.0,0.000000,1.000000,0.0,0.0,0.000000,1.000000,0.000000,3
47669,202309050912,2023-12-28,2017106240,0.0,1.0,0.0,0.0,0.0,0.333333,0.000000,0.666667,0.0,0.200000,0.200000,0.6,0.0,0.300000,0.400000,0.300000,4
47670,202309050912,2023-12-28,2018103205,0.0,0.0,0.0,1.0,0.0,0.333333,0.000000,0.666667,0.0,0.200000,0.400000,0.4,0.0,0.200000,0.500000,0.300000,3


In [103]:



#ペース作成
"""
merged_df にはrace_id（レースのid）と、そのレースに出走するhorse_id(馬のid)の列があり、
さらに dominant_position_categoryというその馬の脚質を表す列がある
脚質は1-4の四種類の数字カテゴリが入っており
レースごとに、出走する馬全体における、その脚質がいる割合と絶対数を求め、
それぞれ四つの列を新たに作成
"""
# dominant_position_category の脚質マッピング
category_mapping_per = {1: "escape_per", 2: "taking_lead_per", 3: "in_front_per", 4: "pursuit_per"}
category_mapping_count = {1: "escape_count", 2: "taking_lead_count", 3: "in_front_count", 4: "pursuit_count"}

# 各レースで dominant_position_category の割合を計算
position_ratios = (
    merged_df.groupby("race_id")["dominant_position_category"]
    .value_counts(normalize=True)  # 割合を計算
    .unstack(fill_value=0)         # 行: race_id, 列: dominant_position_category の形に
    .rename(columns=category_mapping_per)  # 列名を割合用に置き換え
)

# 各レースで dominant_position_category の絶対数を計算
position_counts = (
    merged_df.groupby("race_id")["dominant_position_category"]
    .value_counts()                # 絶対数を計算
    .unstack(fill_value=0)         # 行: race_id, 列: dominant_position_category の形に
    .rename(columns=category_mapping_count)  # 列名を絶対数用に置き換え
)

# 割合と絶対数を結合
position_data = position_ratios.join(position_counts)
columns_to_remove = list(position_data.columns)
merged_df = merged_df.drop(columns=columns_to_remove, errors="ignore")

# 元の DataFrame にマージ
merged_df = merged_df.merge(position_data, how="left", on="race_id")

In [150]:
if 'pace_category' in merged_df.columns:
    merged_df = merged_df.drop(columns=['pace_category'])

# ペースカテゴリを計算する関数
def calculate_pace_category(row):
    escape_count = row.get("escape_count", 0)
    taking_lead_per = row.get("taking_lead_per", 0)

    if escape_count >= 3:
        return 1  # ハイペース
    elif escape_count == 2:
        if taking_lead_per > 0.7:
            return 1  # ハイペース
        else:
            return 2  
    elif escape_count == 1:
        if taking_lead_per > 0.75:
            return 1  # ハイペース
        elif 0.65 < taking_lead_per <= 0.75:
            return 2  # ハイミドルペース
        elif 0.55 < taking_lead_per <= 0.65:
            return 3  # スローミドルペース
        else:
            return 4  # スローペース
    elif escape_count == 0:
        # 逃げ馬が0の場合、先行馬の割合に基づく
        if taking_lead_per > 0.88:
            return 1  # ハイペース
        elif 0.75 < taking_lead_per <= 0.88:
            return 2  # ハイミドルペース
        elif 0.65 < taking_lead_per <= 0.75:
            return 3  # スローミドルペース
        else:
            return 4  # スローペース
    return None  # 予期しない場合は None
merged_df["pace_category"] = merged_df.apply(calculate_pace_category, axis=1)




In [116]:
# 結果の確認
merged_df.head(100)

,race_id,date,horse_id,escape_1races,taking_lead_1races,in_front_1races,pursuit_1races,escape_3races,taking_lead_3races,in_front_3races,pursuit_3races,escape_5races,taking_lead_5races,in_front_5races,pursuit_5races,escape_10races,taking_lead_10races,in_front_10races,pursuit_10races,dominant_position_category,escape_per,taking_lead_per,in_front_per,pursuit_per,escape_count,taking_lead_count,in_front_count,pursuit_count,pace_category
0,202306010101,2023-01-05,2020103575,0.0,1.0,0.0,0.0,0.0,1.000000,0.000000,0.000000,0.0,1.000000,0.000000,0.000000,0.000,1.000000,0.000000,0.000000,2,0.0,0.8125,0.1875,0.0000,0,13,3,0,2
1,202306010101,2023-01-05,2020107073,0.0,1.0,0.0,0.0,0.0,0.500000,0.500000,0.000000,0.0,0.500000,0.500000,0.000000,0.000,0.500000,0.500000,0.000000,2,0.0,0.8125,0.1875,0.0000,0,13,3,0,2
2,202306010101,2023-01-05,2020102562,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,0.0,0.8125,0.1875,0.0000,0,13,3,0,2
3,202306010101,2023-01-05,2020106345,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,0.0,0.8125,0.1875,0.0000,0,13,3,0,2
4,202306010101,2023-01-05,2020100039,0.0,0.0,1.0,0.0,0.0,0.333333,0.666667,0.000000,0.0,0.333333,0.666667,0.000000,0.000,0.333333,0.666667,0.000000,3,0.0,0.8125,0.1875,0.0000,0,13,3,0,2
5,202306010101,2023-01-05,2020104817,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,0.0,0.8125,0.1875,0.0000,0,13,3,0,2
6,202306010101,2023-01-05,2020104295,0.0,0.0,1.0,0.0,0.0,0.000000,1.000000,0.000000,0.0,0.000000,1.000000,0.000000,0.000,0.000000,1.000000,0.000000,3,0.0,0.8125,0.1875,0.0000,0,13,3,0,2
7,202306010101,2023-01-05,2020105491,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,0.0,0.8125,0.1875,0.0000,0,13,3,0,2
8,202306010101,2023-01-05,2020104358,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,0.0,0.8125,0.1875,0.0000,0,13,3,0,2
9,202306010101,2023-01-05,2020105324,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,0.0,0.8125,0.1875,0.0000,0,13,3,0,2


In [151]:
category_percentage_kyakusitu = merged_df["pace_category"].value_counts(normalize=True) * 100
category_percentage_kyakusitu

pace_category
4    43.365078
3    21.096241
1    19.904766
2    15.633915
Name: proportion, dtype: float64

In [291]:
%autoreload

In [320]:
#futre_修正、馬場状態
# レース結果テーブルの前処理
results_preprocessed = preprocessing.process_results()
# 馬の過去成績テーブルの加工
horse_results_preprocessed = preprocessing.process_horse_results()
# レース情報テーブルの前処理
race_info_preprocessed = preprocessing.process_race_info()

In [321]:
import re
from pathlib import Path

import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from tqdm.notebook import tqdm
from webdriver_manager.chrome import ChromeDriverManager
from io import StringIO 

DATA_DIR = Path("..", "data")
POPULATION_DIR = DATA_DIR / "00_population"
INPUT_DIR = DATA_DIR / "01_preprocessed"
OUTPUT_DIR = DATA_DIR / "02_features"
OUTPUT_DIR.mkdir(exist_ok=True, parents=True)

population_dir = POPULATION_DIR
poplation_filename = "population.csv"
input_dir = INPUT_DIR
results_filename= "results.csv"
race_info_filename = "race_info.csv"
horse_results_filename = "horse_results.csv"
jockey_leading_filename = "jockey_leading.csv"
trainer_leading_filename = "trainer_leading.csv"
peds_filename = "peds.csv"
sire_leading_filename = "sire_leading.csv"
output_dir = OUTPUT_DIR
output_filename = "features.csv"

population = pd.read_csv(population_dir / poplation_filename, sep="\t")
results = pd.read_csv(input_dir / results_filename, sep="\t")
race_info = pd.read_csv(input_dir / race_info_filename, sep="\t")
horse_results = pd.read_csv(input_dir / horse_results_filename, sep="\t")
jockey_leading = pd.read_csv(input_dir / jockey_leading_filename, sep="\t")
trainer_leading = pd.read_csv(
    input_dir / trainer_leading_filename, sep="\t"
)
peds = pd.read_csv(input_dir / peds_filename, sep="\t")
sire_leading = pd.read_csv(input_dir / sire_leading_filename, sep="\t")
output_dir = output_dir
agg_horse_per_group_cols_dfs = {}

In [120]:
#直近レースから、馬場状態調査、race_resultsで行う
"""
1/競馬場xタイプxランクx距離ごとの平均タイムと直近のレースとの差異の平均を特徴量に、そのまま数値として入れれる
2/芝以外を除去
3/競馬場x芝タイプで集計(グループバイ)
4/"race_date_day_count"の当該未満かつ、800以内が一週間以内のデータになるはず
5/その平均を取る
6/+なら軽く、-なら重く、それぞれ5段階のカテゴリに入れる
"""

merged_df = population.copy()        
# df1 = (
#     merged_df
#     .merge(race_info[["race_id", "place","weather","ground_state","course_len","race_type","race_date_day_count"]], on="race_id",how="left")
# )

df = (
    merged_df
    .merge(race_info[["race_id", "place","weather","ground_state","course_len","race_type","race_date_day_count"]], on="race_id")
)
df["place"] = df["place"].astype(int)


old_merged_df = population.copy()      


df_old2 = (
    old_merged_df
    .merge(results[["race_id", "horse_id","time","rank_per_horse"]], on=["race_id", "horse_id"])
    .merge(race_info[["race_id", "place","weather","ground_state","race_class","course_len","race_type","race_date_day_count"]], on="race_id")
)
df_old2["place"] = df_old2["place"].astype(int)
df_old2["race_class"] = df_old2["race_class"].astype(int)
# 距離/競馬場/タイプ/レースランク
df_old2["distance_place_type_race_class"] = (df_old2["course_len"].astype(str) + df_old2["place"].astype(str) + df_old2["race_type"].astype(str) + df_old2["race_class"].astype(str)).astype(int)


baselog_old = (
    population.merge(
        race_info[["race_id", "course_len", "race_type", "around"]], on="race_id"
    )
    .merge(
        horse_results,
        on=["horse_id", "course_len", "race_type"],
        suffixes=("", "_horse"),
    )
    .query("date_horse < date")
    .sort_values("date_horse", ascending=False)
)
df_old = (
    baselog_old
    .merge(results[["race_id", "horse_id", "sex"]], on=["race_id", "horse_id"])
)
df_old["place"] = df_old["place"].astype(int)
df_old["race_class"] = df_old["race_class"].astype(int)
df_old["ground_state"] = df_old["ground_state"].astype(int)
df_old["around"] = df_old["around"].fillna(3).astype(int)
df_old["weather"] = df_old["weather"].astype(int)  
             

df_old["distance_place_type_race_class"] = (df_old["course_len"].astype(str) + df_old["place"].astype(str) + df_old["race_type"].astype(str) + df_old["race_class"].astype(str)).astype(int)
target_mean_1 = df_old.groupby("distance_place_type_race_class")["time"].mean()
# 平均値をカテゴリ変数にマッピング
df_old["distance_place_type_race_class_encoded"] = df_old["distance_place_type_race_class"].map(target_mean_1)


df_old = df_old[["distance_place_type_race_class",'distance_place_type_race_class_encoded']]


columns_to_merge = [
    ("distance_place_type_race_class",'distance_place_type_race_class_encoded'),
]

# 各ペアを順番に処理
for original_col, encoded_col in columns_to_merge:
    df2_subset = df_old[[original_col, encoded_col]].drop_duplicates()  # 重複を削除
    df2_subset = df2_subset.reset_index(drop=True)  # インデックスをリセット
    df_old2 = df_old2.merge(df2_subset, on=original_col, how='left')  # dfにマージ
df_old2["distance_place_type_race_class_encoded_time_diff"] = df_old2['distance_place_type_race_class_encoded'] - df_old2["time"]


# # df1にあってdfにないものを取得
# df3 = df1.merge(df, on=["race_id", "date", "horse_id"], how="left", indicator=True)

# # '_merge'列が 'left_only' になっている行だけを抽出
# df_diff = df3[df3['_merge'] == 'left_only']
# df_diff
# race_info[["race_id"] == 202306010103]


In [121]:
df_old2= df_old2[df_old2["race_type"] != 2]
df_old2_1 = df_old2[df_old2["race_type"] != 0]
df_old2

,race_id,date,horse_id,time,rank_per_horse,place,weather,ground_state,race_class,course_len,race_type,race_date_day_count,distance_place_type_race_class,distance_place_type_race_class_encoded,distance_place_type_race_class_encoded_time_diff
0,202306010101,2023-01-05,2020103575,72.6,0.0625,6,1,0,1,1200,0,73803501,1200601,73.111571,0.511571
1,202306010101,2023-01-05,2020107073,72.9,0.1250,6,1,0,1,1200,0,73803501,1200601,73.111571,0.211571
2,202306010101,2023-01-05,2020102562,72.9,0.1875,6,1,0,1,1200,0,73803501,1200601,73.111571,0.211571
3,202306010101,2023-01-05,2020106345,73.4,0.2500,6,1,0,1,1200,0,73803501,1200601,73.111571,-0.288429
4,202306010101,2023-01-05,2020100039,73.9,0.3125,6,1,0,1,1200,0,73803501,1200601,73.111571,-0.788429
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45897,202309050912,2023-12-28,2017104873,69.5,0.7500,9,1,0,4,1200,1,73839212,1200914,68.841143,-0.658857
45898,202309050912,2023-12-28,2019100653,69.5,0.8125,9,1,0,4,1200,1,73839212,1200914,68.841143,-0.658857
45899,202309050912,2023-12-28,2017106240,69.6,0.8750,9,1,0,4,1200,1,73839212,1200914,68.841143,-0.758857
45900,202309050912,2023-12-28,2018103205,69.8,0.9375,9,1,0,4,1200,1,73839212,1200914,68.841143,-0.958857


In [ ]:
#こっからは汚れないdfに対しての処理

In [122]:

# 1. df と df_old2_1 の準備
# df = <your df DataFrame>
# df_old2_1 = <your df_old2_1 DataFrame>

# 2. df の各行について処理
def compute_mean_for_row(row, df_old2_1):
    # race_type == 0 の場合は NaN を返す
    if row["race_type"] == 0:
        return np.nan
        
    target_day_count = row["race_date_day_count"]  # df の各行の race_date_day_count

    # 3. df_old2_1 から条件に一致する行をフィルタリング
    filtered_df_old2_1 = df_old2_1[
        (df_old2_1["race_date_day_count"] >= (target_day_count - 1200)) &  # race_date_day_count が target_day_count-1200 以上
        (df_old2_1["race_date_day_count"] <= (target_day_count - 1)) &  # race_date_day_count が target_day_count-1 以下
        (df_old2_1["place"] == row["place"]) &  # place が一致
        (df_old2_1["weather"].isin([0, 1, 2])) &  # weather が 0, 1, 2 のいずれか
        (df_old2_1["ground_state"] == 0)   # ground_state が 0
        # &
        # (df_old2_1["rank_per_horse"] < 0.87)  # rank_per_horse が 0.87 未満
    ]

    # 4. フィルタリングした行の "distance_place_type_race_class_encoded_time_diff" の平均を計算
    mean_time_diff = filtered_df_old2_1["distance_place_type_race_class_encoded_time_diff"].mean()

    # 5. 計算結果を返す（NaNの場合も考慮）
    return mean_time_diff if not np.isnan(mean_time_diff) else np.nan

# 6. df の各行に対して、計算した平均値を新しい列に追加
df["mean_ground_state_time_diff"] = df.apply(compute_mean_for_row, axis=1, df_old2_1=df_old2_1)



In [16]:
# # 結果の確認
# df.head(200)
df_race_type_1 = df[df["race_type"] == 1]
df_race_type_1

,race_id,date,horse_id,place,weather,ground_state,course_len,race_type,race_date_day_count,mean_ground_state_time_diff
48,202306010105,2023-01-05,2020101772,6,1,0,1600,1,73803505,NaN
49,202306010105,2023-01-05,2020100301,6,1,0,1600,1,73803505,NaN
50,202306010105,2023-01-05,2020102740,6,1,0,1600,1,73803505,NaN
51,202306010105,2023-01-05,2020103246,6,1,0,1600,1,73803505,NaN
52,202306010105,2023-01-05,2020105053,6,1,0,1600,1,73803505,NaN
...,...,...,...,...,...,...,...,...,...,...
42276,202309050912,2023-12-28,2017104873,9,1,0,1200,1,73839212,0.076383
42277,202309050912,2023-12-28,2019100653,9,1,0,1200,1,73839212,0.076383
42278,202309050912,2023-12-28,2017106240,9,1,0,1200,1,73839212,0.076383
42279,202309050912,2023-12-28,2018103205,9,1,0,1200,1,73839212,0.076383


In [19]:
# race_type == 1 のかつ、dfmean_ground_state_time_diff が 0 以下の行をフィルタリング
df_race_type_1_filtered = df_race_type_1[df_race_type_1["mean_ground_state_time_diff"] <= -1.5]

# その結果を先頭200行分表示
len(df_race_type_1_filtered)
df_race_type_1_filtered

,race_id,date,horse_id,place,weather,ground_state,course_len,race_type,race_date_day_count,mean_ground_state_time_diff
7380,202310020606,2023-02-26,2019110042,10,1,0,1200,1,73808706,-3.237818
7381,202310020606,2023-02-26,2019110064,10,1,0,1200,1,73808706,-3.237818
7382,202310020606,2023-02-26,2019106194,10,1,0,1200,1,73808706,-3.237818
7383,202310020606,2023-02-26,2018104559,10,1,0,1200,1,73808706,-3.237818
7384,202310020606,2023-02-26,2017101448,10,1,0,1200,1,73808706,-3.237818
...,...,...,...,...,...,...,...,...,...,...
39074,202307040106,2023-12-02,2020102834,7,1,0,1600,1,73836606,-1.577451
39075,202307040106,2023-12-02,2019106590,7,1,0,1600,1,73836606,-1.577451
39076,202307040106,2023-12-02,2020105809,7,1,0,1600,1,73836606,-1.577451
39077,202307040106,2023-12-02,2019106863,7,1,0,1600,1,73836606,-1.577451


In [123]:
# 新たな列を作成
def assign_value(row):
    # weather が 3, 4, 5 または ground_state が 0 以外の場合は 5 を設定
    if row["weather"] in [3, 4, 5] or row["ground_state"] != 0:
        return 7
    # mean_ground_state_time_diff に基づいて分類
    if 2.0 <= row["mean_ground_state_time_diff"]:
        return 1  #　超高速馬場1
    elif 1 <= row["mean_ground_state_time_diff"] < 2.0:
        return 2  # 高速馬場2
    
    elif 0.4 <= row["mean_ground_state_time_diff"] < 1:
        return 3  # 軽い馬場3
        
    elif -0.4 <= row["mean_ground_state_time_diff"] < 0.4:
        return 4  # 標準的な馬場4

    
    elif -1 <= row["mean_ground_state_time_diff"] < -0.4:
        return 5  # やや重い馬場5
    
    elif -2 <= row["mean_ground_state_time_diff"] < -1:
        return 6  # 重い馬場5
    
    elif row["mean_ground_state_time_diff"] < -2:
        return 7  # 道悪7
    # 該当しない場合は NaN を設定
    return float("nan")

# 新しい列を追加
df["ground_state_level"] = df.apply(assign_value, axis=1)


In [125]:
df_race_type_1 = df[df["race_type"] == 1]
df_race_type_1.head(100)

,race_id,date,horse_id,place,weather,ground_state,course_len,race_type,race_date_day_count,mean_ground_state_time_diff,ground_state_level
64,202306010105,2023-01-05,2020101772,6,1,0,1600,1,73803505,NaN,NaN
65,202306010105,2023-01-05,2020100301,6,1,0,1600,1,73803505,NaN,NaN
66,202306010105,2023-01-05,2020102740,6,1,0,1600,1,73803505,NaN,NaN
67,202306010105,2023-01-05,2020103246,6,1,0,1600,1,73803505,NaN,NaN
68,202306010105,2023-01-05,2020105053,6,1,0,1600,1,73803505,NaN,NaN
69,202306010105,2023-01-05,2020102958,6,1,0,1600,1,73803505,NaN,NaN
70,202306010105,2023-01-05,2020101261,6,1,0,1600,1,73803505,NaN,NaN
71,202306010105,2023-01-05,2020105940,6,1,0,1600,1,73803505,NaN,NaN
72,202306010105,2023-01-05,2020100406,6,1,0,1600,1,73803505,NaN,NaN
73,202306010105,2023-01-05,2020102600,6,1,0,1600,1,73803505,NaN,NaN


In [126]:
# ground_state_level の各値の割合を計算
level_counts = df_race_type_1["ground_state_level"].value_counts(normalize=True) * 100

# 結果を表示
print(level_counts)

ground_state_level
4.0    42.495479
7.0    28.765823
3.0    16.048825
5.0     8.951175
2.0     2.653707
6.0     1.084991
Name: proportion, dtype: float64


In [34]:
weather_count = df_race_type_1["weather"].value_counts(normalize=True) * 100
print(weather_count)

weather
1    59.801464
2    30.823223
3     4.888198
0     4.386844
4     0.100271
Name: proportion, dtype: float64


In [35]:
weather_count = df_race_type_1["ground_state"].value_counts(normalize=True) * 100
print(weather_count )

ground_state
0    71.272436
2    16.935726
1     9.425449
3     2.366389
Name: proportion, dtype: float64


In [216]:
df.tail(100)

,race_id,date,horse_id,place,weather,ground_state,course_len,race_type,race_date_day_count,mean_ground_state_time_diff,ground_state_level
42181,202309050903,2023-12-28,2021103277,9,1,0,1800,1,73839203,-0.058734,3.0
42182,202309050903,2023-12-28,2021104215,9,1,0,1800,1,73839203,-0.058734,3.0
42183,202309050903,2023-12-28,2021104590,9,1,0,1800,1,73839203,-0.058734,3.0
42184,202309050903,2023-12-28,2021105018,9,1,0,1800,1,73839203,-0.058734,3.0
42185,202309050903,2023-12-28,2021102811,9,1,0,1800,1,73839203,-0.058734,3.0
42186,202309050906,2023-12-28,2021105276,9,1,0,1800,0,73839206,NaN,NaN
42187,202309050906,2023-12-28,2021100838,9,1,0,1800,0,73839206,NaN,NaN
42188,202309050906,2023-12-28,2021104581,9,1,0,1800,0,73839206,NaN,NaN
42189,202309050906,2023-12-28,2021104873,9,1,0,1800,0,73839206,NaN,NaN
42190,202309050906,2023-12-28,2021105094,9,1,0,1800,0,73839206,NaN,NaN


In [335]:

merged_df = merged_df.merge(df[["race_id","date","horse_id","mean_ground_state_time_diff","ground_state_level"]],on=["race_id","date","horse_id"])	

,race_id,date,horse_id,place,weather,ground_state,course_len,race_type,race_date_day_count,mean_ground_state_time_diff,ground_state_level
0,202306010101,2023-01-05,2020103575,6,1,0,1200,0,73803501,NaN,NaN
1,202306010101,2023-01-05,2020107073,6,1,0,1200,0,73803501,NaN,NaN
2,202306010101,2023-01-05,2020102562,6,1,0,1200,0,73803501,NaN,NaN
3,202306010101,2023-01-05,2020106345,6,1,0,1200,0,73803501,NaN,NaN
4,202306010101,2023-01-05,2020100039,6,1,0,1200,0,73803501,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
46207,202309050912,2023-12-28,2017104873,9,1,0,1200,1,73839212,0.029045,3.0
46208,202309050912,2023-12-28,2019100653,9,1,0,1200,1,73839212,0.029045,3.0
46209,202309050912,2023-12-28,2017106240,9,1,0,1200,1,73839212,0.029045,3.0
46210,202309050912,2023-12-28,2018103205,9,1,0,1200,1,73839212,0.029045,3.0


In [336]:
merged_df

,race_id,date,horse_id,mean_ground_state_time_diff,ground_state_level
0,202306010101,2023-01-05,2020103575,NaN,NaN
1,202306010101,2023-01-05,2020107073,NaN,NaN
2,202306010101,2023-01-05,2020102562,NaN,NaN
3,202306010101,2023-01-05,2020106345,NaN,NaN
4,202306010101,2023-01-05,2020100039,NaN,NaN
...,...,...,...,...,...
46207,202309050912,2023-12-28,2017104873,0.029045,3.0
46208,202309050912,2023-12-28,2019100653,0.029045,3.0
46209,202309050912,2023-12-28,2017106240,0.029045,3.0
46210,202309050912,2023-12-28,2018103205,0.029045,3.0


In [82]:
#予測の時はこれ
baselog2 = (
    population.merge(
        race_info[["race_id", "course_len", "race_type"]], on="race_id"
    )
    .merge(
        results,
        on=["horse_id", "course_len", "race_type"],
        suffixes=("", "_horse"),
    )
    .query("date_horse < date")
    .sort_values("date_horse", ascending=False)
)


,race_id,date,horse_id
0,202306010101,2023-01-05,2020103575
1,202306010101,2023-01-05,2020107073
2,202306010101,2023-01-05,2020102562
3,202306010101,2023-01-05,2020106345
4,202306010101,2023-01-05,2020100039
...,...,...,...
42917,202309050912,2023-12-28,2017104873
42918,202309050912,2023-12-28,2019100653
42919,202309050912,2023-12-28,2017106240
42920,202309050912,2023-12-28,2018103205


In [ ]:
df = (
    results[["race_id", "horse_id","time"]]
    .merge(race_info[["race_id", "race_type","place","race_date_day_count"]], on="race_id")
    .merge(merged_df[["race_id", "horse_id", "date"]], on=["race_id", "horse_id"], how="left")
)
df["kaisai_count"] = df['race_id'].astype(str).str[-6:-4]

In [188]:
#prepro.paseを図る.horse_results
import json
from pathlib import Path

import pandas as pd
import numpy as np
import ast
COMMON_DATA_DIR = Path("..", "..", "common", "data")
RAWDF_DIR = COMMON_DATA_DIR / "rawdf"
MAPPING_DIR = COMMON_DATA_DIR / "mapping"
POPULATION_DIR = Path("..", "data", "00_population")
OUTPUT_DIR = Path("..", "data", "01_preprocessed")
OUTPUT_DIR.mkdir(exist_ok=True, parents=True)

# カテゴリ変数を数値に変換するためのマッピング
with open(MAPPING_DIR / "sex.json", "r") as f:
    sex_mapping = json.load(f)
with open(MAPPING_DIR / "race_type.json", "r") as f:
    race_type_mapping = json.load(f)
with open(MAPPING_DIR / "around.json", "r") as f:
    around_mapping = json.load(f)
with open(MAPPING_DIR / "weather.json", "r") as f:
    weather_mapping = json.load(f)
with open(MAPPING_DIR / "ground_state.json", "r") as f:
    ground_state_mapping = json.load(f)
with open(MAPPING_DIR / "race_class.json", "r") as f:
    race_class_mapping = json.load(f)
with open(MAPPING_DIR / "place.json", "r") as f:
    place_mapping = json.load(f)

In [189]:
population_dir = POPULATION_DIR
populaton_filename = "population.csv"
input_dir = RAWDF_DIR
output_dir = OUTPUT_DIR
input_filename = "horse_results.csv"
output_filename = "horse_results.csv"
race_type_mapping = race_type_mapping
weather_mapping = weather_mapping
ground_state_mapping = ground_state_mapping
race_class_mapping = race_class_mapping
place_mapping = place_mapping

In [190]:
population = pd.read_csv(population_dir / populaton_filename, sep="\t")
# df = pd.read_csv(input_dir / input_filename, sep="\t").query(
#     "horse_id in @population['horse_id']"
# )
# クエリでリストを直接使用
population_horse_ids = population['horse_id'].tolist()

df = pd.read_csv(input_dir / input_filename, sep="\t").query(
    "horse_id in @population_horse_ids"
)
    

df["rank"] = pd.to_numeric(df["着順"], errors="coerce")
df.dropna(subset=["rank"], inplace=True)
df["date"] = pd.to_datetime(df["日付"])
df["weather"] = df["天気"].map(weather_mapping)
df["race_type"] = df["距離"].str[0].map(race_type_mapping)
df = df[df["race_type"] != 2]
df["course_len"] = df["距離"].str.extract(r"(\d+)").astype(int)
df["ground_state"] = df["馬場"].map(ground_state_mapping)
df["rank_diff"] = df["着差"].map(lambda x: 0 if x < 0 else x)
df["prize"] = df["賞金"].fillna(0)
df["umaban"] = df["馬番"].astype(int)

df["wakuban"] = df["枠番"].fillna(0).astype(int)


df["nobori"] = df["上り"].astype(float)

regex_race_class = "|".join(race_class_mapping)
df["race_class"] = (
    df["レース名"].str.extract(rf"({regex_race_class})")[0].map(race_class_mapping)
)
df["time"] = pd.to_datetime(df["タイム"], format="%M:%S.%f", errors="coerce")
df["time"] = (
    df["time"].dt.minute * 60
    + df["time"].dt.second
    + df["time"].dt.microsecond / 1000000
)
df["win"] = (df["rank"] == 1).astype(int)
df["rentai"] = (df["rank"] <= 2).astype(int)
df["show"] = (df["rank"] <= 3).astype(int)
df["place"] = df["開催"].str.extract(r"(\D+)")[0].map(place_mapping)
df.dropna(subset=["place"], inplace=True)    
df.rename(columns={"頭数": "n_horses"}, inplace=True)

pase_cols = df['ペース'].str.split('-', expand=True)
pase_cols.columns = [f'pase_{i+1}' for i in range(pase_cols.shape[1])]

# データ型変換と型確認
pase_cols = pase_cols.apply(pd.to_numeric, errors='coerce').astype('float64')

# 元のデータフレームと結合
df = pd.concat([df, pase_cols], axis=1)


# pase_2 - pase_1 の計算
df['pase_diff'] = df['pase_2'] - df['pase_1']
# course_len の上2桁を取得して奇数かどうかを判定
df['course_len_prefix'] = df['course_len'].astype(str).str[:2].astype(float)  # 上2桁を取得

# 奇数の場合に pase_diff を NaN に設定
df.loc[df['course_len_prefix'] % 2 != 0, 'pase_diff'] = None
# pase_diff をカテゴリに分ける関数
def categorize_pase_diff(value):
    if value < -1.0:
        return 1  # ハイペース
    elif -1.0 <= value <= 0.0:
        return 2  # ハイミドルペース
    elif 0.0 < value <= 1.0:
        return 3  # スローミドルペース
    elif value > 1.0:
        return 4  # スローペース
    return None  # 値が NaN の場合など
    

# 新しい列にカテゴリを適用
df["pase_category"] = df["pase_diff"].apply(categorize_pase_diff)


# NoneをNaNに置き換え
df = df.where(pd.notnull(df), np.nan)



In [157]:
df.head(100)

,horse_id,日付,開催,天気,R,レース名,映像,n_horses,枠番,馬番,オッズ,人気,着順,騎手,斤量,距離,馬場,馬場指数,タイム,着差,ﾀｲﾑ指数,通過,ペース,上り,馬体重,厩舎ｺﾒﾝﾄ,備考,勝ち馬(2着馬),賞金,受賞歴,rank,date,weather,race_type,course_len,ground_state,rank_diff,prize,umaban,wakuban,nobori,race_class,time,win,rentai,show,place,pase_1,pase_2,pase_diff,course_len_prefix,pase_category
924,2013105125,2016/09/03,2新潟11,晴,4.0,3歳未勝利,NaN,15.0,4.0,7.0,158.7,14.0,6,嘉藤貴行,56.0,ダ1800,良,**,1:57.2,2.4,**,11-10-13-12,36.9-38.4,39.4,468(-2),NaN,NaN,キングドラゴン,NaN,NaN,6.0,2016-09-03,1.0,0,1800,0.0,2.4,0.0,7,4,39.4,1.0,117.2,0,0,0,4.0,36.9,38.4,1.5,18.0,4.0
925,2013105125,2016/08/21,2新潟8,晴,3.0,3歳未勝利,NaN,15.0,4.0,6.0,158.1,13.0,5,嘉藤貴行,56.0,ダ1800,良,**,1:56.1,1.3,**,13-13-11-11,36.9-39.7,39.7,470(+2),NaN,NaN,サムライダマシー,48.0,NaN,5.0,2016-08-21,1.0,0,1800,0.0,1.3,48.0,6,4,39.7,1.0,116.1,0,0,0,4.0,36.9,39.7,2.8,18.0,4.0
926,2013105125,2016/07/30,2新潟1,晴,6.0,3歳未勝利,NaN,13.0,7.0,10.0,132.4,13.0,8,伊藤工真,56.0,ダ1800,良,**,1:56.4,1.7,**,9-9-7-6,38.0-38.1,38.8,468(-2),NaN,NaN,ネコエルフ,NaN,NaN,8.0,2016-07-30,1.0,0,1800,0.0,1.7,0.0,10,7,38.8,1.0,116.4,0,0,0,4.0,38.0,38.1,0.1,18.0,3.0
927,2013105125,2016/07/17,2福島6,曇,6.0,3歳未勝利,NaN,15.0,6.0,11.0,116.5,14.0,12,江田照男,56.0,ダ1700,良,**,1:49.3,2.5,**,14-14-14-14,30.6-37.1,38.3,470(-4),NaN,NaN,ボーシエル,NaN,NaN,12.0,2016-07-17,2.0,0,1700,0.0,2.5,0.0,11,6,38.3,1.0,109.3,0,0,0,3.0,30.6,37.1,NaN,17.0,NaN
928,2013105125,2016/06/26,3東京8,晴,2.0,3歳未勝利,NaN,15.0,7.0,12.0,71.8,11.0,9,江田照男,56.0,ダ2100,稍,**,2:15.9,1.9,**,11-11-5-5,31.4-38.0,39.3,474(-4),NaN,NaN,カブキモノ,NaN,NaN,9.0,2016-06-26,1.0,0,2100,2.0,1.9,0.0,12,7,39.3,1.0,135.9,0,0,0,5.0,31.4,38.0,NaN,21.0,NaN
929,2013105125,2016/04/09,3中山5,晴,2.0,3歳未勝利,NaN,14.0,1.0,1.0,17.4,5.0,9,柴田大知,56.0,ダ1800,重,**,1:57.7,2.9,**,14-14-12-12,36.7-38.4,40.0,478(+2),NaN,NaN,ポルトドレーヴ,NaN,NaN,9.0,2016-04-09,1.0,0,1800,1.0,2.9,0.0,1,1,40.0,1.0,117.7,0,0,0,6.0,36.7,38.4,1.7,18.0,4.0
930,2013105125,2016/03/27,2中京6,晴,2.0,3歳未勝利,NaN,14.0,6.0,9.0,16.2,5.0,5,太宰啓介,56.0,ダ1800,良,**,1:56.5,1.3,**,12-12-11-9,38.4-38.3,38.5,476(+4),NaN,NaN,ヤマニンシルフ,50.0,NaN,5.0,2016-03-27,1.0,0,1800,0.0,1.3,50.0,9,6,38.5,1.0,116.5,0,0,0,7.0,38.4,38.3,-0.1,18.0,2.0
931,2013105125,2016/03/13,2中京2,曇,6.0,3歳未勝利,NaN,12.0,8.0,11.0,17.7,5.0,4,太宰啓介,56.0,ダ1900,良,**,2:03.6,1.3,**,11-9-8-8,31.1-38.4,38.9,472(0),NaN,NaN,ケイツートール,75.0,NaN,4.0,2016-03-13,2.0,0,1900,0.0,1.3,75.0,11,8,38.9,1.0,123.6,0,0,0,7.0,31.1,38.4,NaN,19.0,NaN
932,2013105125,2016/02/07,1東京4,晴,2.0,3歳未勝利,NaN,16.0,5.0,9.0,39.3,11.0,10,丸山元気,56.0,ダ2100,良,**,2:17.8,2.9,**,12-14-12-13,30.9-37.6,39.5,472(+2),NaN,NaN,マイネルトゥラン,NaN,NaN,10.0,2016-02-07,1.0,0,2100,0.0,2.9,0.0,9,5,39.5,1.0,137.8,0,0,0,5.0,30.9,37.6,NaN,21.0,NaN
933,2013105125,2016/01/24,1中京4,晴,2.0,3歳未勝利,NaN,14.0,4.0,6.0,132.6,11.0,4,松田大作,56.0,ダ1900,良,**,2:05.5,0.9,**,12-12-11-10,31.3-40.3,38.8,470(-2),NaN,NaN,ディアクーシー,75.0,NaN,4.0,2016-01-24,1.0,0,1900,0.0,0.9,75.0,6,4,38.8,1.0,125.5,0,0,0,7.0,31.3,40.3,NaN,19.0,NaN


In [158]:
category_percentage = df["pase_category"].value_counts(normalize=True) * 100
category_percentage

pase_category
4.0    45.210489
3.0    20.090360
1.0    18.373517
2.0    16.325634
Name: proportion, dtype: float64

In [90]:
#良馬場0に絞る
filtered_df = df[df["ground_state"] == 0]

# フィルタリング後に 'pase_category' の割合を計算
category_percentage = filtered_df["pase_category"].value_counts(normalize=True) * 100
category_percentage 

pase_category
4.0    41.630053
1.0    20.907305
3.0    20.320295
2.0    17.142347
Name: proportion, dtype: float64

In [85]:
#ホースidごとの経験してきたレース展開
raceid_pase_percentage = df.groupby("horse_id")["pase_category"].value_counts(normalize=True).unstack(fill_value=0) * 100
raceid_pase_percentage 

pase_category,1.0,2.0,3.0,4.0
horse_id,,,,
2011106610,50.909091,25.454545,16.363636,7.272727
2012100683,12.500000,16.666667,20.833333,50.000000
2012103532,34.285714,31.428571,14.285714,20.000000
2012104463,44.736842,23.684211,18.421053,13.157895
2012104504,17.241379,10.344828,24.137931,48.275862
...,...,...,...,...
2021110155,20.000000,0.000000,40.000000,40.000000
2021110156,0.000000,0.000000,0.000000,100.000000
2021110157,50.000000,50.000000,0.000000,0.000000


In [77]:
# 結果確認
df.head(100)

,horse_id,日付,開催,天気,R,レース名,映像,n_horses,枠番,馬番,オッズ,人気,着順,騎手,斤量,距離,馬場,馬場指数,タイム,着差,ﾀｲﾑ指数,通過,ペース,上り,馬体重,厩舎ｺﾒﾝﾄ,備考,勝ち馬(2着馬),賞金,受賞歴,rank,date,weather,race_type,course_len,ground_state,rank_diff,prize,umaban,wakuban,nobori,race_class,time,win,rentai,show,place,pase_1,pase_2,pase_diff,course_len_prefix
924,2013105125,2016/09/03,2新潟11,晴,4.0,3歳未勝利,NaN,15.0,4.0,7.0,158.7,14.0,6,嘉藤貴行,56.0,ダ1800,良,**,1:57.2,2.4,**,11-10-13-12,36.9-38.4,39.4,468(-2),NaN,NaN,キングドラゴン,NaN,NaN,6.0,2016-09-03,1.0,0,1800,0.0,2.4,0.0,7,4,39.4,1.0,117.2,0,0,0,4.0,36.9,38.4,1.5,18.0
925,2013105125,2016/08/21,2新潟8,晴,3.0,3歳未勝利,NaN,15.0,4.0,6.0,158.1,13.0,5,嘉藤貴行,56.0,ダ1800,良,**,1:56.1,1.3,**,13-13-11-11,36.9-39.7,39.7,470(+2),NaN,NaN,サムライダマシー,48.0,NaN,5.0,2016-08-21,1.0,0,1800,0.0,1.3,48.0,6,4,39.7,1.0,116.1,0,0,0,4.0,36.9,39.7,2.8,18.0
926,2013105125,2016/07/30,2新潟1,晴,6.0,3歳未勝利,NaN,13.0,7.0,10.0,132.4,13.0,8,伊藤工真,56.0,ダ1800,良,**,1:56.4,1.7,**,9-9-7-6,38.0-38.1,38.8,468(-2),NaN,NaN,ネコエルフ,NaN,NaN,8.0,2016-07-30,1.0,0,1800,0.0,1.7,0.0,10,7,38.8,1.0,116.4,0,0,0,4.0,38.0,38.1,0.1,18.0
927,2013105125,2016/07/17,2福島6,曇,6.0,3歳未勝利,NaN,15.0,6.0,11.0,116.5,14.0,12,江田照男,56.0,ダ1700,良,**,1:49.3,2.5,**,14-14-14-14,30.6-37.1,38.3,470(-4),NaN,NaN,ボーシエル,NaN,NaN,12.0,2016-07-17,2.0,0,1700,0.0,2.5,0.0,11,6,38.3,1.0,109.3,0,0,0,3.0,30.6,37.1,NaN,17.0
928,2013105125,2016/06/26,3東京8,晴,2.0,3歳未勝利,NaN,15.0,7.0,12.0,71.8,11.0,9,江田照男,56.0,ダ2100,稍,**,2:15.9,1.9,**,11-11-5-5,31.4-38.0,39.3,474(-4),NaN,NaN,カブキモノ,NaN,NaN,9.0,2016-06-26,1.0,0,2100,2.0,1.9,0.0,12,7,39.3,1.0,135.9,0,0,0,5.0,31.4,38.0,NaN,21.0
929,2013105125,2016/04/09,3中山5,晴,2.0,3歳未勝利,NaN,14.0,1.0,1.0,17.4,5.0,9,柴田大知,56.0,ダ1800,重,**,1:57.7,2.9,**,14-14-12-12,36.7-38.4,40.0,478(+2),NaN,NaN,ポルトドレーヴ,NaN,NaN,9.0,2016-04-09,1.0,0,1800,1.0,2.9,0.0,1,1,40.0,1.0,117.7,0,0,0,6.0,36.7,38.4,1.7,18.0
930,2013105125,2016/03/27,2中京6,晴,2.0,3歳未勝利,NaN,14.0,6.0,9.0,16.2,5.0,5,太宰啓介,56.0,ダ1800,良,**,1:56.5,1.3,**,12-12-11-9,38.4-38.3,38.5,476(+4),NaN,NaN,ヤマニンシルフ,50.0,NaN,5.0,2016-03-27,1.0,0,1800,0.0,1.3,50.0,9,6,38.5,1.0,116.5,0,0,0,7.0,38.4,38.3,-0.1,18.0
931,2013105125,2016/03/13,2中京2,曇,6.0,3歳未勝利,NaN,12.0,8.0,11.0,17.7,5.0,4,太宰啓介,56.0,ダ1900,良,**,2:03.6,1.3,**,11-9-8-8,31.1-38.4,38.9,472(0),NaN,NaN,ケイツートール,75.0,NaN,4.0,2016-03-13,2.0,0,1900,0.0,1.3,75.0,11,8,38.9,1.0,123.6,0,0,0,7.0,31.1,38.4,NaN,19.0
932,2013105125,2016/02/07,1東京4,晴,2.0,3歳未勝利,NaN,16.0,5.0,9.0,39.3,11.0,10,丸山元気,56.0,ダ2100,良,**,2:17.8,2.9,**,12-14-12-13,30.9-37.6,39.5,472(+2),NaN,NaN,マイネルトゥラン,NaN,NaN,10.0,2016-02-07,1.0,0,2100,0.0,2.9,0.0,9,5,39.5,1.0,137.8,0,0,0,5.0,30.9,37.6,NaN,21.0
933,2013105125,2016/01/24,1中京4,晴,2.0,3歳未勝利,NaN,14.0,4.0,6.0,132.6,11.0,4,松田大作,56.0,ダ1900,良,**,2:05.5,0.9,**,12-12-11-10,31.3-40.3,38.8,470(-2),NaN,NaN,ディアクーシー,75.0,NaN,4.0,2016-01-24,1.0,0,1900,0.0,0.9,75.0,6,4,38.8,1.0,125.5,0,0,0,7.0,31.3,40.3,NaN,19.0


In [78]:
# 結果確認
df[['pase_1', 'pase_2', 'pase_diff']].head(100)

,pase_1,pase_2,pase_diff
924,36.9,38.4,1.5
925,36.9,39.7,2.8
926,38.0,38.1,0.1
927,30.6,37.1,NaN
928,31.4,38.0,NaN
929,36.7,38.4,1.7
930,38.4,38.3,-0.1
931,31.1,38.4,NaN
932,30.9,37.6,NaN
933,31.3,40.3,NaN


In [5]:
%autoreload

In [6]:
#futre_修正、pase_cross
# レース結果テーブルの前処理
results_preprocessed = preprocessing.process_results()
# 馬の過去成績テーブルの加工
horse_results_preprocessed = preprocessing.process_horse_results()
# レース情報テーブルの前処理
race_info_preprocessed = preprocessing.process_race_info()

In [10]:
import re
from pathlib import Path

import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from tqdm.notebook import tqdm
from webdriver_manager.chrome import ChromeDriverManager
from io import StringIO 

DATA_DIR = Path("..", "data")
POPULATION_DIR = DATA_DIR / "00_population"
INPUT_DIR = DATA_DIR / "01_preprocessed"
OUTPUT_DIR = DATA_DIR / "02_features"
OUTPUT_DIR.mkdir(exist_ok=True, parents=True)

population_dir = POPULATION_DIR
poplation_filename = "population.csv"
input_dir = INPUT_DIR
results_filename= "results.csv"
race_info_filename = "race_info.csv"
horse_results_filename = "horse_results.csv"
jockey_leading_filename = "jockey_leading.csv"
trainer_leading_filename = "trainer_leading.csv"
peds_filename = "peds.csv"
sire_leading_filename = "sire_leading.csv"
output_dir = OUTPUT_DIR
output_filename = "features.csv"

population = pd.read_csv(population_dir / poplation_filename, sep="\t")
results = pd.read_csv(input_dir / results_filename, sep="\t")
race_info = pd.read_csv(input_dir / race_info_filename, sep="\t")
horse_results = pd.read_csv(input_dir / horse_results_filename, sep="\t")
jockey_leading = pd.read_csv(input_dir / jockey_leading_filename, sep="\t")
trainer_leading = pd.read_csv(
    input_dir / trainer_leading_filename, sep="\t"
)
peds = pd.read_csv(input_dir / peds_filename, sep="\t")
sire_leading = pd.read_csv(input_dir / sire_leading_filename, sep="\t")
output_dir = output_dir
agg_horse_per_group_cols_dfs = {}

In [382]:
baselog = (
    population.merge(
        race_info[["race_id", "course_len", "race_type"]], on="race_id"
    )
    .merge(
        horse_results,
        on=["horse_id", "course_len", "race_type"],
        suffixes=("", "_horse"),
    )
    .query("date_horse < date")
    .sort_values("date_horse", ascending=False)
)
grouped_df = baselog.groupby(["race_id", "horse_id"])
merged_df = population.copy()

In [345]:
baselog

,race_id,date,horse_id,course_len,race_type,rank_per_horse,date_horse,rank,prize,rank_diff,umaban,wakuban,weather,ground_state,race_class,n_horses,time,time_courselen,nobori,win,rentai,show,place,corner_1_per_horse,corner_2_per_horse,corner_3_per_horse,corner_4_per_horse,race_position,pase_diff,pase_category,corner_1,corner_2,corner_3,corner_4,pase_1,pase_2
243237,202309050903,2023-12-28,2021104215,1800,1,1.000000,2023-12-23,14.0,0.0,2.0,9,6,1,0,1,14.0,110.3,0.061278,34.7,0,0,0,9,1.000000,1.000000,NaN,NaN,4,-1.0,2.0,14,14.0,NaN,NaN,35.4,34.4
243222,202309050903,2023-12-28,2021105731,1800,1,0.857143,2023-12-23,12.0,0.0,1.6,12,7,1,0,1,14.0,109.9,0.061056,35.0,0,0,0,9,0.428571,0.571429,NaN,NaN,3,-1.0,2.0,6,8.0,NaN,NaN,35.4,34.4
238904,202306050704,2023-12-23,2021103822,1800,0,0.562500,2023-12-17,9.0,0.0,4.2,14,7,1,0,0,16.0,120.9,0.067167,42.1,0,0,0,6,0.875000,0.875000,0.750000,0.812500,4,2.3,4.0,14,14.0,12.0,13.0,38.3,40.6
241527,202309050807,2023-12-24,2020102273,1800,0,0.916667,2023-12-17,11.0,0.0,2.7,9,7,1,2,2,12.0,117.1,0.065056,40.0,0,0,0,7,0.833333,0.833333,0.833333,0.916667,4,2.2,4.0,10,10.0,10.0,11.0,37.5,39.7
242833,202306050910,2023-12-28,2018106429,1200,0,0.454545,2023-12-17,5.0,155.0,0.7,1,1,1,2,5,11.0,73.4,0.061167,36.6,0,0,0,9,0.909091,0.909091,NaN,NaN,4,3.1,4.0,10,10.0,NaN,NaN,34.8,37.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101518,202305020808,2023-05-14,2011106610,1600,1,0.375000,2014-01-19,6.0,0.0,0.7,14,7,1,0,0,16.0,97.7,0.061062,37.4,0,0,0,6,0.125000,0.062500,0.125000,NaN,1,0.7,3.0,2,1.0,2.0,NaN,36.0,36.7
180371,202309040510,2023-09-18,2011106610,1600,1,0.375000,2014-01-19,6.0,0.0,0.7,14,7,1,0,0,16.0,97.7,0.061062,37.4,0,0,0,6,0.125000,0.062500,0.125000,NaN,1,0.7,3.0,2,1.0,2.0,NaN,36.0,36.7
212467,202305050312,2023-11-11,2011106610,1600,1,0.375000,2014-01-19,6.0,0.0,0.7,14,7,1,0,0,16.0,97.7,0.061062,37.4,0,0,0,6,0.125000,0.062500,0.125000,NaN,1,0.7,3.0,2,1.0,2.0,NaN,36.0,36.7
188998,202308020109,2023-10-07,2011106610,1600,1,0.375000,2014-01-19,6.0,0.0,0.7,14,7,1,0,0,16.0,97.7,0.061062,37.4,0,0,0,6,0.125000,0.062500,0.125000,NaN,1,0.7,3.0,2,1.0,2.0,NaN,36.0,36.7


In [383]:
merged_df = population.copy()

"""
"pase_category"にある1234という数字カテゴリごとにwin,rentai,showを
新たな列が必要

baselogの"pase_category"==が1,2,3,4ごとにそれぞれ分ける
baselog1,2,3,4ごとに、win,rentai,showのn_racesごとの平均を算出
最終的にbaselog1,2,3,4をconcatする

"""
# pase_category ごとにデータをフィルタリング
baselog1 = baselog[baselog["pase_category"] == 1]
baselog2 = baselog[baselog["pase_category"] == 2]
baselog3 = baselog[baselog["pase_category"] == 3]
baselog4 = baselog[baselog["pase_category"] == 4]
n_races: list[int] = [1, 3, 5, 10]
# 結果を格納するためのリスト
result_dfs = []

# 各pase_categoryに対してn_racesごとの平均を計算
for pase_category, df1 in zip([1, 2, 3, 4], [baselog1, baselog2, baselog3, baselog4]):
    grouped_df = df1.groupby(["race_id", "horse_id"])
    
    for n_race in tqdm(n_races, desc=f"pase_category_win_{pase_category}"):
        # n_raceに基づいて集計
        agg_df = (
            grouped_df.head(n_race)
            .groupby(["race_id", "horse_id"])[["win", "rentai", "show"]]
            .agg("mean")
        )
        
        # 列名を修正
        agg_df.columns = [
            f"{col}_{n_race}races_per_pase_category{pase_category}" for col in agg_df.columns
        ]
        
        # # 平均と標準偏差を基に相対値を計算
        # tmp_df = agg_df.groupby(["race_id"])
        # relative_df = ((agg_df - tmp_df.mean()) / tmp_df.std()).add_suffix("_relative")
        
        # # 生成したデータフレームをresult_dfsに追加
        # result_dfs.append(relative_df)
        result_dfs.append(agg_df)


# 結果をconcatして1つのデータフレームにまとめる
final_agg_df = pd.concat(result_dfs, axis=1)

# merge_dfとマージ
merged_df = merged_df.merge(final_agg_df, on=["race_id", "horse_id"], how="left")



pase_category_win_1:   0%|          | 0/4 [00:00<?, ?it/s]

pase_category_win_2:   0%|          | 0/4 [00:00<?, ?it/s]

pase_category_win_3:   0%|          | 0/4 [00:00<?, ?it/s]

pase_category_win_4:   0%|          | 0/4 [00:00<?, ?it/s]

In [348]:
# 結果の確認
merged_df

,race_id,date,horse_id,win_1races_per_pase_category1,rentai_1races_per_pase_category1,show_1races_per_pase_category1,win_3races_per_pase_category1,rentai_3races_per_pase_category1,show_3races_per_pase_category1,win_5races_per_pase_category1,rentai_5races_per_pase_category1,show_5races_per_pase_category1,win_10races_per_pase_category1,rentai_10races_per_pase_category1,show_10races_per_pase_category1,win_1races_per_pase_category2,rentai_1races_per_pase_category2,show_1races_per_pase_category2,win_3races_per_pase_category2,rentai_3races_per_pase_category2,show_3races_per_pase_category2,win_5races_per_pase_category2,rentai_5races_per_pase_category2,show_5races_per_pase_category2,win_10races_per_pase_category2,rentai_10races_per_pase_category2,show_10races_per_pase_category2,win_1races_per_pase_category3,rentai_1races_per_pase_category3,show_1races_per_pase_category3,win_3races_per_pase_category3,rentai_3races_per_pase_category3,show_3races_per_pase_category3,win_5races_per_pase_category3,rentai_5races_per_pase_category3,show_5races_per_pase_category3,win_10races_per_pase_category3,rentai_10races_per_pase_category3,show_10races_per_pase_category3,win_1races_per_pase_category4,rentai_1races_per_pase_category4,show_1races_per_pase_category4,win_3races_per_pase_category4,rentai_3races_per_pase_category4,show_3races_per_pase_category4,win_5races_per_pase_category4,rentai_5races_per_pase_category4,show_5races_per_pase_category4,win_10races_per_pase_category4,rentai_10races_per_pase_category4,show_10races_per_pase_category4
0,202306010101,2023-01-05,2020103575,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0,1.0,0.0,1.000000,1.000000,0.0,1.000000,1.000000,0.000000,1.000000,1.000000
1,202306010101,2023-01-05,2020107073,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0,1.0,0.0,0.500000,0.500000,0.0,0.500000,0.500000,0.000000,0.500000,0.500000
2,202306010101,2023-01-05,2020102562,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,202306010101,2023-01-05,2020106345,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,202306010101,2023-01-05,2020100039,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,1.0,0.0,0.333333,0.666667,0.0,0.333333,0.666667,0.000000,0.333333,0.666667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46207,202309050912,2023-12-28,2017104873,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.500000,0.500000,0.0,0.50,0.50,0.0,0.50,0.50,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.2,0.0,0.000000,0.200000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.142857,0.142857,0.142857
46208,202309050912,2023-12-28,2019100653,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
46209,202309050912,2023-12-28,2017106240,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.333333,0.333333,0.0,0.25,0.25,0.0,0.25,0.25,0.0,0.0,1.0,0.0,0.0,0.666667,0.0,0.2,0.6,0.0,0.222222,0.555556,0.0,0.0,0.0,0.0,0.000000,0.000000,0.2,0.200000,0.400000,0.100000,0.200000,0.400000
46210,202309050912,2023-12-28,2018103205,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,0.5,0.500000,0.500000,0.5

In [427]:

"""
過去nレースにおける脚質割合を計算し、当該レースのペースを脚質の割合から予想する
"""
def calculate_race_position_percentage(group, n_race: int):
    """
    過去nレースにおける脚質割合を計算する
    """
    # 過去nレースに絞る
    past_races = group.head(n_race)
    
    # 各脚質のカウント
    counts = past_races['race_position'].value_counts(normalize=True).to_dict()
    
    # 結果を辞書形式で返す（割合が存在しない場合は0.0を補完）
    return {
        "escape": counts.get(1, 0.0),
        "taking_lead": counts.get(2, 0.0),
        "in_front": counts.get(3, 0.0),
        "pursuit": counts.get(4, 0.0),
    }

# 過去nレースのリスト
n_race_list = [1, 3, 5, 10]
baselog2 = (
    population.merge(
        race_info[["race_id", "course_len", "race_type"]], on="race_id"
    )
    .merge(
        horse_results,
        on=["horse_id", "course_len", "race_type"],
        suffixes=("", "_horse"),
    )
    .query("date_horse < date")
    .sort_values("date_horse", ascending=False)
)

# 集計用データフレームの初期化
merged_df2 = population.copy()
# grouped_dfを適用して計算
grouped_df2 = baselog2.groupby(["race_id", "horse_id"])
# 各過去nレースの割合を計算して追加
for n_race in n_race_list:
    # `n_race`を整数として関数に渡す
    position_percentage = grouped_df2.apply(
        lambda group: calculate_race_position_percentage(group, n_race=n_race),
        include_groups=False,  # グループ列を除外
    )
    
    # 結果をデータフレームとして展開
    position_percentage_df = position_percentage.apply(pd.Series).reset_index()
    position_percentage_df.rename(
        columns={
            "escape": f"escape_{n_race}races",
            "taking_lead": f"taking_lead_{n_race}races",
            "in_front": f"in_front_{n_race}races",
            "pursuit": f"pursuit_{n_race}races",
        },
        inplace=True,
    )
    
    # 結果をマージ
    merged_df2 = merged_df2.merge(position_percentage_df, on=["race_id", "horse_id"], how="left")



# dominant_position_category を計算
position_columns_5 = ["escape_5races", "taking_lead_5races", "in_front_5races", "pursuit_5races"]
position_columns_3 = ["escape_3races", "taking_lead_3races", "in_front_3races", "pursuit_3races"]
position_columns_1 = ["escape_1races", "taking_lead_1races", "in_front_1races", "pursuit_1races"]


def determine_dominant_position(row):
    
    # 5racesの中から計算
    if not row[position_columns_5].isnull().all():
        max_column = row[position_columns_5].idxmax()
        return {
            "escape_5races": 1,
            "taking_lead_5races": 2,
            "in_front_5races": 3,
            "pursuit_5races": 4,
        }[max_column]
    # 3racesの中から計算
    elif not row[position_columns_3].isnull().all():
        max_column = row[position_columns_3].idxmax()
        return {
            "escape_3races": 1,
            "taking_lead_3races": 2,
            "in_front_3races": 3,
            "pursuit_3races": 4,
        }[max_column]
    # 1racesの中から計算
    elif not row[position_columns_1].isnull().all():
        max_column = row[position_columns_1].idxmax()
        return {
            "escape_1races": 1,
            "taking_lead_1races": 2,
            "in_front_1races": 3,
            "pursuit_1races": 4,
        }[max_column]
    # すべて欠損の場合は1を返す
    else:
        return 2

# 各行に対して dominant_position_category を適用
merged_df2["dominant_position_category"] = merged_df2.apply(determine_dominant_position, axis=1)




    #ペース作成
"""
merged_df にはrace_id（レースのid）と、そのレースに出走するhorse_id(馬のid)の列があり、
さらに dominant_position_categoryというその馬の脚質を表す列がある
脚質は1-4の四種類の数字カテゴリが入っており
レースごとに、出走する馬全体における、その脚質がいる割合と絶対数を求め、
それぞれ四つの列を新たに作成
"""
# dominant_position_category の脚質マッピング
category_mapping_per = {1: "escape_per", 2: "taking_lead_per", 3: "in_front_per", 4: "pursuit_per"}
category_mapping_count = {1: "escape_count", 2: "taking_lead_count", 3: "in_front_count", 4: "pursuit_count"}

# 各レースで dominant_position_category の割合を計算
position_ratios = (
    merged_df2.groupby("race_id")["dominant_position_category"]
    .value_counts(normalize=True)  # 割合を計算
    .unstack(fill_value=0)         # 行: race_id, 列: dominant_position_category の形に
    .rename(columns=category_mapping_per)  # 列名を割合用に置き換え
)

# 各レースで dominant_position_category の絶対数を計算
position_counts = (
    merged_df2.groupby("race_id")["dominant_position_category"]
    .value_counts()                # 絶対数を計算
    .unstack(fill_value=0)         # 行: race_id, 列: dominant_position_category の形に
    .rename(columns=category_mapping_count)  # 列名を絶対数用に置き換え
)

# 割合と絶対数を結合
position_data = position_ratios.join(position_counts)
columns_to_remove = list(position_data.columns)
merged_df2 = merged_df2.drop(columns=columns_to_remove, errors="ignore")

# 元の DataFrame にマージ
merged_df2 = merged_df2.merge(position_data, how="left", on="race_id")


if 'pace_category' in merged_df2.columns:
    merged_df2 = merged_df2.drop(columns=['pace_category'])

# ペースカテゴリを計算する関数
def calculate_pace_category(row):
    escape_count = row.get("escape_count", 0)
    taking_lead_per = row.get("taking_lead_per", 0)

    if escape_count >= 3:
        return 1  # ハイペース
    elif escape_count == 2:
        if taking_lead_per > 0.7:
            return 1  # ハイペース
        else:
            return 2  
    elif escape_count == 1:
        if taking_lead_per > 0.75:
            return 1  # ハイペース
        elif 0.65 < taking_lead_per <= 0.75:
            return 2  # ハイミドルペース
        elif 0.55 < taking_lead_per <= 0.65:
            return 3  # スローミドルペース
        else:
            return 4  # スローペース
    elif escape_count == 0:
        # 逃げ馬が0の場合、先行馬の割合に基づく
        if taking_lead_per > 0.88:
            return 1  # ハイペース
        elif 0.75 < taking_lead_per <= 0.88:
            return 2  # ハイミドルペース
        elif 0.65 < taking_lead_per <= 0.75:
            return 3  # スローミドルペース
        else:
            return 4  # スローペース
    return None  # 予期しない場合は None
merged_df2["pace_category2"] = merged_df2.apply(calculate_pace_category, axis=1)





In [428]:
merged_df2

,race_id,date,horse_id,escape_1races,taking_lead_1races,in_front_1races,pursuit_1races,escape_3races,taking_lead_3races,in_front_3races,pursuit_3races,escape_5races,taking_lead_5races,in_front_5races,pursuit_5races,escape_10races,taking_lead_10races,in_front_10races,pursuit_10races,dominant_position_category,escape_per,taking_lead_per,in_front_per,pursuit_per,escape_count,taking_lead_count,in_front_count,pursuit_count,pace_category2
0,202306010101,2023-01-05,2020103575,0.0,1.0,0.0,0.0,0.0,1.000000,0.000000,0.000000,0.0,1.000000,0.000000,0.0,0.0,1.000000,0.000000,0.000000,2,0.0,0.8125,0.1875,0.0000,0,13,3,0,2
1,202306010101,2023-01-05,2020107073,0.0,1.0,0.0,0.0,0.0,0.500000,0.500000,0.000000,0.0,0.500000,0.500000,0.0,0.0,0.500000,0.500000,0.000000,2,0.0,0.8125,0.1875,0.0000,0,13,3,0,2
2,202306010101,2023-01-05,2020102562,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,0.0,0.8125,0.1875,0.0000,0,13,3,0,2
3,202306010101,2023-01-05,2020106345,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,0.0,0.8125,0.1875,0.0000,0,13,3,0,2
4,202306010101,2023-01-05,2020100039,0.0,0.0,1.0,0.0,0.0,0.333333,0.666667,0.000000,0.0,0.333333,0.666667,0.0,0.0,0.333333,0.666667,0.000000,3,0.0,0.8125,0.1875,0.0000,0,13,3,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46207,202309050912,2023-12-28,2017104873,0.0,0.0,0.0,1.0,0.0,0.000000,0.333333,0.666667,0.0,0.000000,0.200000,0.8,0.0,0.000000,0.200000,0.800000,4,0.0,0.6250,0.1875,0.1875,0,10,3,3,4
46208,202309050912,2023-12-28,2019100653,0.0,0.0,1.0,0.0,0.0,0.000000,1.000000,0.000000,0.0,0.000000,1.000000,0.0,0.0,0.000000,1.000000,0.000000,3,0.0,0.6250,0.1875,0.1875,0,10,3,3,4
46209,202309050912,2023-12-28,2017106240,0.0,1.0,0.0,0.0,0.0,0.333333,0.000000,0.666667,0.0,0.200000,0.200000,0.6,0.0,0.300000,0.400000,0.300000,4,0.0,0.6250,0.1875,0.1875,0,10,3,3,4
46210,202309050912,2023-12-28,2018103205,0.0,0.0,0.0,1.0,0.0,0.333333,0.000000,0.666667,0.0,0.200000,0.400000,0.4,0.0,0.200000,0.500000,0.300000,3,0.0,0.6250,0.1875,0.1875,0,10,3,3,4


In [358]:
merged_df

,race_id,date,horse_id,win_1races_per_pase_category1,rentai_1races_per_pase_category1,show_1races_per_pase_category1,win_3races_per_pase_category1,rentai_3races_per_pase_category1,show_3races_per_pase_category1,win_5races_per_pase_category1,rentai_5races_per_pase_category1,show_5races_per_pase_category1,win_10races_per_pase_category1,rentai_10races_per_pase_category1,show_10races_per_pase_category1,win_1races_per_pase_category2,rentai_1races_per_pase_category2,show_1races_per_pase_category2,win_3races_per_pase_category2,rentai_3races_per_pase_category2,show_3races_per_pase_category2,win_5races_per_pase_category2,rentai_5races_per_pase_category2,show_5races_per_pase_category2,win_10races_per_pase_category2,rentai_10races_per_pase_category2,show_10races_per_pase_category2,win_1races_per_pase_category3,rentai_1races_per_pase_category3,show_1races_per_pase_category3,win_3races_per_pase_category3,rentai_3races_per_pase_category3,show_3races_per_pase_category3,win_5races_per_pase_category3,rentai_5races_per_pase_category3,show_5races_per_pase_category3,win_10races_per_pase_category3,rentai_10races_per_pase_category3,show_10races_per_pase_category3,win_1races_per_pase_category4,rentai_1races_per_pase_category4,show_1races_per_pase_category4,win_3races_per_pase_category4,rentai_3races_per_pase_category4,show_3races_per_pase_category4,win_5races_per_pase_category4,rentai_5races_per_pase_category4,show_5races_per_pase_category4,win_10races_per_pase_category4,rentai_10races_per_pase_category4,show_10races_per_pase_category4
0,202306010101,2023-01-05,2020103575,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0,1.0,0.0,1.000000,1.000000,0.0,1.000000,1.000000,0.000000,1.000000,1.000000
1,202306010101,2023-01-05,2020107073,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0,1.0,0.0,0.500000,0.500000,0.0,0.500000,0.500000,0.000000,0.500000,0.500000
2,202306010101,2023-01-05,2020102562,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,202306010101,2023-01-05,2020106345,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,202306010101,2023-01-05,2020100039,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,1.0,0.0,0.333333,0.666667,0.0,0.333333,0.666667,0.000000,0.333333,0.666667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46207,202309050912,2023-12-28,2017104873,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.500000,0.500000,0.0,0.50,0.50,0.0,0.50,0.50,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.2,0.0,0.000000,0.200000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.142857,0.142857,0.142857
46208,202309050912,2023-12-28,2019100653,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
46209,202309050912,2023-12-28,2017106240,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.333333,0.333333,0.0,0.25,0.25,0.0,0.25,0.25,0.0,0.0,1.0,0.0,0.0,0.666667,0.0,0.2,0.6,0.0,0.222222,0.555556,0.0,0.0,0.0,0.0,0.000000,0.000000,0.2,0.200000,0.400000,0.100000,0.200000,0.400000
46210,202309050912,2023-12-28,2018103205,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,0.5,0.500000,0.500000,0.5

In [431]:
merged_df = merged_df.merge(merged_df2[["race_id","date","horse_id","pace_category2"]],
                            on=["race_id","date","horse_id"],)

In [373]:
merged_df[["pace_category","win_3races_per_pase_category1","win_3races_per_pase_category2","win_3races_per_pase_category3","win_3races_per_pase_category4"]].tail(100)

,pace_category,win_3races_per_pase_category1,win_3races_per_pase_category2,win_3races_per_pase_category3,win_3races_per_pase_category4
46112,1,NaN,NaN,NaN,NaN
46113,1,NaN,NaN,NaN,NaN
46114,1,NaN,NaN,NaN,NaN
46115,1,NaN,NaN,NaN,NaN
46116,1,NaN,NaN,NaN,NaN
46117,1,NaN,NaN,0.000000,1.000000
46118,1,NaN,NaN,1.000000,NaN
46119,1,NaN,NaN,NaN,1.000000
46120,1,NaN,1.000000,NaN,NaN
46121,1,NaN,NaN,1.000000,NaN


In [432]:

# merged_df のコピーを作成
new_columns = []

# merged_df に対して、1, 3, 5, 10 の各nに対して処理
for n in [1, 3, 5, 10]:  # nの値を1, 3, 5, 10とする
    for result_type in ["win", "rentai", "show"]:  # win, rentai, showの各列を処理
        # 新しい列名を設定
        new_col_name = f"{result_type}_for_pase_category_n{n}"
        new_columns.append(new_col_name)

        # nとresult_typeに基づいて、該当する列を選択
        for category in [1, 2, 3, 4]:  # pase_categoryに基づいて1から4のカテゴリを処理
            # 該当するpase_categoryとnを持つ元の列を動的に選択
            col_name = f"{result_type}_{n}races_per_pase_category{category}"
            
            # merged_dfに新しい列を代入（pase_categoryに関わらず同じ列に結果を格納）
            merged_df.loc[merged_df["pace_category"] == category, new_col_name] = merged_df[col_name]





KeyError: 'win_1races_per_pase_category1'

In [ ]:
# 結果の確認
merged_df[new_columns].tail(100)



In [386]:
merged_df.columns.tolist()


['race_id',
 'date',
 'horse_id',
 'win_1races_per_pase_category1',
 'rentai_1races_per_pase_category1',
 'show_1races_per_pase_category1',
 'win_3races_per_pase_category1',
 'rentai_3races_per_pase_category1',
 'show_3races_per_pase_category1',
 'win_5races_per_pase_category1',
 'rentai_5races_per_pase_category1',
 'show_5races_per_pase_category1',
 'win_10races_per_pase_category1',
 'rentai_10races_per_pase_category1',
 'show_10races_per_pase_category1',
 'win_1races_per_pase_category2',
 'rentai_1races_per_pase_category2',
 'show_1races_per_pase_category2',
 'win_3races_per_pase_category2',
 'rentai_3races_per_pase_category2',
 'show_3races_per_pase_category2',
 'win_5races_per_pase_category2',
 'rentai_5races_per_pase_category2',
 'show_5races_per_pase_category2',
 'win_10races_per_pase_category2',
 'rentai_10races_per_pase_category2',
 'show_10races_per_pase_category2',
 'win_1races_per_pase_category3',
 'rentai_1races_per_pase_category3',
 'show_1races_per_pase_category3',
 'win_

In [389]:
merged_df111 = merged_df[["race_id","date","horse_id",
    'win_for_pase_category_n1',
 'rentai_for_pase_category_n1',
 'show_for_pase_category_n1',
 'win_for_pase_category_n3',
 'rentai_for_pase_category_n3',
 'show_for_pase_category_n3',
 'win_for_pase_category_n5',
 'rentai_for_pase_category_n5',
 'show_for_pase_category_n5',
 'win_for_pase_category_n10',
 'rentai_for_pase_category_n10',
 'show_for_pase_category_n10']]

In [400]:
merged_df = merged_df111[["race_id","horse_id",
    'win_for_pase_category_n1',
 'rentai_for_pase_category_n1',
 'show_for_pase_category_n1',
 'win_for_pase_category_n3',
 'rentai_for_pase_category_n3',
 'show_for_pase_category_n3',
 'win_for_pase_category_n5',
 'rentai_for_pase_category_n5',
 'show_for_pase_category_n5',
 'win_for_pase_category_n10',
 'rentai_for_pase_category_n10',
 'show_for_pase_category_n10']]

In [390]:
merged_df = merged_df[["race_id","horse_id",
    'win_for_pase_category_n1',
 'rentai_for_pase_category_n1',
 'show_for_pase_category_n1',
 'win_for_pase_category_n3',
 'rentai_for_pase_category_n3',
 'show_for_pase_category_n3',
 'win_for_pase_category_n5',
 'rentai_for_pase_category_n5',
 'show_for_pase_category_n5',
 'win_for_pase_category_n10',
 'rentai_for_pase_category_n10',
 'show_for_pase_category_n10']]

In [392]:
merged_df4 = population.copy()
merged_df4

,race_id,date,horse_id
0,202306010101,2023-01-05,2020103575
1,202306010101,2023-01-05,2020107073
2,202306010101,2023-01-05,2020102562
3,202306010101,2023-01-05,2020106345
4,202306010101,2023-01-05,2020100039
...,...,...,...
46207,202309050912,2023-12-28,2017104873
46208,202309050912,2023-12-28,2019100653
46209,202309050912,2023-12-28,2017106240
46210,202309050912,2023-12-28,2018103205


In [396]:
merged_df

,race_id,horse_id,win_for_pase_category_n1,rentai_for_pase_category_n1,show_for_pase_category_n1,win_for_pase_category_n3,rentai_for_pase_category_n3,show_for_pase_category_n3,win_for_pase_category_n5,rentai_for_pase_category_n5,show_for_pase_category_n5,win_for_pase_category_n10,rentai_for_pase_category_n10,show_for_pase_category_n10
0,202306010101,2020103575,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,202306010101,2020107073,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,202306010101,2020102562,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,202306010101,2020106345,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,202306010101,2020100039,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46207,202309050912,2017104873,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.142857,0.142857,0.142857
46208,202309050912,2019100653,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000
46209,202309050912,2017106240,0.0,0.0,0.0,0.0,0.0,0.000000,0.2,0.2,0.4,0.100000,0.200000,0.400000
46210,202309050912,2018103205,0.0,0.0,0.0,0.0,0.0,0.333333,0.0,0.0,0.4,0.000000,0.000000,0.300000


In [401]:
# race_id ごとに標準化（相対値）を計算
tmp_df = merged_df.groupby("race_id")

# 各レースごとに、平均と標準偏差を計算
# .transform を使って、同じサイズのデータフレームに変換
relative_df = (merged_df - tmp_df.transform("mean")) / tmp_df.transform("std")

# カラム名をわかりやすくするために "_relative" を追加
relative_df = relative_df.add_suffix("_relative")

# 元のデータフレームに標準化されたデータを結合
merged_df1000 = merged_df.join(relative_df, how="left")

# 結果を確認
merged_df1000.head(100)


,race_id,horse_id,win_for_pase_category_n1,rentai_for_pase_category_n1,show_for_pase_category_n1,win_for_pase_category_n3,rentai_for_pase_category_n3,show_for_pase_category_n3,win_for_pase_category_n5,rentai_for_pase_category_n5,show_for_pase_category_n5,win_for_pase_category_n10,rentai_for_pase_category_n10,show_for_pase_category_n10,horse_id_relative,race_id_relative,rentai_for_pase_category_n1_relative,rentai_for_pase_category_n10_relative,rentai_for_pase_category_n3_relative,rentai_for_pase_category_n5_relative,show_for_pase_category_n1_relative,show_for_pase_category_n10_relative,show_for_pase_category_n3_relative,show_for_pase_category_n5_relative,win_for_pase_category_n1_relative,win_for_pase_category_n10_relative,win_for_pase_category_n3_relative,win_for_pase_category_n5_relative
0,202306010101,2020103575,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.099102,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,202306010101,2020107073,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.509995,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,202306010101,2020102562,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.565087,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,202306010101,2020106345,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.175111,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,202306010101,2020100039,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1.725680,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,202306010101,2020104817,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.472224,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,202306010101,2020104295,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.232101,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,202306010101,2020105491,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.782267,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,202306010101,2020104358,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.261082,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,202306010101,2020105324,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.705446,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [404]:
merged_df1000.columns
merged_df1000 = merged_df1000[['race_id', 'horse_id', 'win_for_pase_category_n1',
       'rentai_for_pase_category_n1', 'show_for_pase_category_n1',
       'win_for_pase_category_n3', 'rentai_for_pase_category_n3',
       'show_for_pase_category_n3', 'win_for_pase_category_n5',
       'rentai_for_pase_category_n5', 'show_for_pase_category_n5',
       'win_for_pase_category_n10', 'rentai_for_pase_category_n10',
       'show_for_pase_category_n10', 
       'rentai_for_pase_category_n1_relative',
       'rentai_for_pase_category_n10_relative',
       'rentai_for_pase_category_n3_relative',
       'rentai_for_pase_category_n5_relative',
       'show_for_pase_category_n1_relative',
       'show_for_pase_category_n10_relative',
       'show_for_pase_category_n3_relative',
       'show_for_pase_category_n5_relative',
       'win_for_pase_category_n1_relative',
       'win_for_pase_category_n10_relative',
       'win_for_pase_category_n3_relative',
       'win_for_pase_category_n5_relative']]
merged_df4 = population.copy()
merged_df1000 = merged_df4.merge(merged_df1000, on=["race_id", "horse_id"], how="left")

In [405]:
merged_df1000

,race_id,date,horse_id,win_for_pase_category_n1,rentai_for_pase_category_n1,show_for_pase_category_n1,win_for_pase_category_n3,rentai_for_pase_category_n3,show_for_pase_category_n3,win_for_pase_category_n5,rentai_for_pase_category_n5,show_for_pase_category_n5,win_for_pase_category_n10,rentai_for_pase_category_n10,show_for_pase_category_n10,rentai_for_pase_category_n1_relative,rentai_for_pase_category_n10_relative,rentai_for_pase_category_n3_relative,rentai_for_pase_category_n5_relative,show_for_pase_category_n1_relative,show_for_pase_category_n10_relative,show_for_pase_category_n3_relative,show_for_pase_category_n5_relative,win_for_pase_category_n1_relative,win_for_pase_category_n10_relative,win_for_pase_category_n3_relative,win_for_pase_category_n5_relative
0,202306010101,2023-01-05,2020103575,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,202306010101,2023-01-05,2020107073,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,202306010101,2023-01-05,2020102562,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,202306010101,2023-01-05,2020106345,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,202306010101,2023-01-05,2020100039,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46207,202309050912,2023-12-28,2017104873,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.142857,0.142857,0.142857,-0.267261,-0.135886,-0.523328,-0.561683,-0.393398,-0.615416,-0.769765,-0.952419,-0.267261,-0.036565,-0.44594,-0.526546
46208,202309050912,2023-12-28,2019100653,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,-0.267261,-0.636519,-0.523328,-0.561683,-0.393398,-1.052214,-0.769765,-0.952419,-0.267261,-0.548475,-0.44594,-0.526546
46209,202309050912,2023-12-28,2017106240,0.0,0.0,0.0,0.0,0.0,0.000000,0.2,0.2,0.4,0.100000,0.200000,0.400000,-0.267261,0.064367,-0.523328,0.122105,-0.393398,0.170819,-0.769765,0.219789,-0.267261,-0.190138,-0.44594,0.175515
46210,202309050912,2023-12-28,2018103205,0.0,0.0,0.0,0.0,0.0,0.333333,0.0,0.0,0.4,0.000000,0.000000,0.300000,-0.267261,-0.636519,-0.523328,-0.561683,-0.393398,-0.134939,0.167340,0.219789,-0.267261,-0.548475,-0.44594,-0.526546


In [15]:
#まとめて実行
# レース結果テーブルの前処理
results_preprocessed = preprocessing.process_results()
# 馬の過去成績テーブルの加工
horse_results_preprocessed = preprocessing.process_horse_results()
# レース情報テーブルの前処理
race_info_preprocessed = preprocessing.process_race_info()

In [77]:
curve_counts = horse_results["pase_category"].value_counts()
print(curve_counts)

pase_category
1.0    52244
2.0    23167
4.0    21077
3.0    18773
Name: count, dtype: int64


In [42]:
"""
特筆見る

過去レースで
◎スローペースで勝った差し・追い込み馬は瞬発力のある証拠
◎ハイペースで勝った逃げ・先行馬は高く評価しましょう
をつくる、それの方が数字が多くなるように


脚質がある
ペースの速さがある
以下のときで、showに入っているとき
+レースクラス+1、でいいかも
入ってなかったら0
過去3.5.10.100raceの最大だけ見る
それを標準化としないver

＿＿ハイペース1→4
逃げ先行がバテるため、差し追い込みが有利

ミドルハイペース2→3

ミドルローペース3→2

＿＿ローペース4→1
後続を寄せ付けないため逃げ先行が有利


逃げ1
234

"""
n_races: list[int] = [1, 3, 5, 10]

baselog = (
    population
    .merge(
        horse_results,
        on="horse_id",
        suffixes=("", "_horse"),
    )
    .query("date_horse < date")
    .sort_values("date_horse", ascending=False)
)
def calculate_scores(baselog):
    # スコア初期化
    baselog["score_pase_potision"] = 0.0

    # スローペースで差し・追い込み馬がshowに入った場合
    condition1 = (baselog["pase_category"] == 1) & (baselog["race_position"].isin([3, 4])) & (baselog["show"] == 1)
    baselog.loc[condition1, "score_pase_potision"] += baselog["race_class"] + 1

    # ハイペースで逃げ・先行馬がshowに入った場合
    condition2 = (baselog["pase_category"] == 4) & (baselog["race_position"].isin([1, 2])) & (baselog["show"] == 1)
    baselog.loc[condition2, "score_pase_potision"] += baselog["race_class"] + 1

    # ミドルスローペースで差し・追い込み馬がshowに入った場合
    condition3 = (baselog["pase_category"] == 2) & (baselog["race_position"].isin([3, 4])) & (baselog["show"] == 1)
    baselog.loc[condition3, "score_pase_potision"] += (baselog["race_class"] + 1) / 1.8

    # ミドルハイペースで逃げ・先行馬がshowに入った場合
    condition4 = (baselog["pase_category"] == 3) & (baselog["race_position"].isin([1, 2])) & (baselog["show"] == 1)
    baselog.loc[condition4, "score_pase_potision"] += (baselog["race_class"] + 1) / 1.8

    # 逃げ・先行馬がshowに入った場合
    condition5 =baselog["pase_category"].isin([1, 2]) & (baselog["race_position"].isin([1, 2])) & (baselog["show"] == 1)
    baselog.loc[condition5, "score_pase_potision"] += (baselog["race_class"] + 1) / 3.6
    # 差し・追い込み馬showに入った場合
    condition6 =baselog["pase_category"].isin([3, 4]) & (baselog["race_position"].isin([3, 4])) & (baselog["show"] == 1)
    baselog.loc[condition6, "score_pase_potision"] += (baselog["race_class"] + 1) / 3.6    

     # showに入りらなかった場合 
    condition7 = (baselog["show"] != 1)
    baselog.loc[condition7, "score_pase_potision"] += 0 
   
    # # その他のペースでshowに入った場合
    # condition5 = ~(
    #     (baselog["pase_category"].isin([1, 2, 3, 4])) & 
    #     (baselog["race_position"].isin([1, 2, 3, 4])) & 
    #     (baselog["show"] == 1)
    # )
    # baselog.loc[condition5 & (baselog["show"] == 1), "score_pase_potision"] += (baselog["race_class"] + 1) / 3.6

    return baselog
result_df = calculate_scores(baselog)

grouped_df = result_df.groupby(["race_id", "horse_id"])
merged_df = population.copy()

for n_race in tqdm(n_races, desc=f"agg_pase_per_score"):
    # スコアの集計（標準化しない版）
    raw_df = (
        grouped_df.head(n_race)
        .groupby(["race_id", "horse_id"])[["score_pase_potision"]]
        .agg(["mean", "max", "min"])
    )
    raw_df.columns = [
        "_".join(col) + f"_{n_race}races_per_score_raw" for col in raw_df.columns
    ]
    
    # スコアの集計（標準化する版）
    std_df = raw_df.copy()  # 同じデータを使用して標準化
    tmp_df = raw_df.groupby(["race_id"])
    std_df = ((raw_df - tmp_df.mean()) / tmp_df.std()).add_suffix("_relative")
    
    # merged_df に追加
    merged_df = merged_df.merge(raw_df, on=["race_id", "horse_id"], how="left")
    merged_df = merged_df.merge(std_df, on=["race_id", "horse_id"], how="left")


agg_pase_per_score:   0%|          | 0/4 [00:00<?, ?it/s]

In [43]:

sorted_df = merged_df.sort_values(by="score_pase_potision_mean_1races_per_score_raw", ascending=False)

# 表示 (上位10行のみ)
sorted_df.head(10)

,race_id,date,horse_id,score_pase_potision_mean_1races_per_score_raw,score_pase_potision_max_1races_per_score_raw,score_pase_potision_min_1races_per_score_raw,score_pase_potision_mean_1races_per_score_raw_relative,score_pase_potision_max_1races_per_score_raw_relative,score_pase_potision_min_1races_per_score_raw_relative,score_pase_potision_mean_3races_per_score_raw,score_pase_potision_max_3races_per_score_raw,score_pase_potision_min_3races_per_score_raw,score_pase_potision_mean_3races_per_score_raw_relative,score_pase_potision_max_3races_per_score_raw_relative,score_pase_potision_min_3races_per_score_raw_relative,score_pase_potision_mean_5races_per_score_raw,score_pase_potision_max_5races_per_score_raw,score_pase_potision_min_5races_per_score_raw,score_pase_potision_mean_5races_per_score_raw_relative,score_pase_potision_max_5races_per_score_raw_relative,score_pase_potision_min_5races_per_score_raw_relative,score_pase_potision_mean_10races_per_score_raw,score_pase_potision_max_10races_per_score_raw,score_pase_potision_min_10races_per_score_raw,score_pase_potision_mean_10races_per_score_raw_relative,score_pase_potision_max_10races_per_score_raw_relative,score_pase_potision_min_10races_per_score_raw_relative
7811,202306020211,2023-02-26,2018104963,9.0,9.0,9.0,2.463463,2.463463,2.463463,6.074074,9.0,2.222222,2.681374,1.761061,2.539756,3.644444,9.0,0.000000,1.397054,1.518705,NaN,3.194444,9.0,0.000000,1.672371,1.436141,NaN
12758,202306030511,2023-04-08,2020101533,9.0,9.0,9.0,3.308536,3.308536,3.308536,4.296296,9.0,0.000000,3.067352,2.880635,-0.258199,4.296296,9.0,0.000000,3.198656,2.499447,-0.258199,4.296296,9.0,0.000000,3.252676,2.499426,-0.258199
19814,202305021211,2023-05-28,2020103532,9.0,9.0,9.0,2.478524,2.478524,2.478524,3.740741,9.0,0.000000,1.893504,1.946930,-0.360182,2.875000,9.0,0.000000,1.606613,1.992725,-0.294692,2.875000,9.0,0.000000,1.716800,1.992725,-0.294692
30092,202301020411,2023-08-20,2017101431,9.0,9.0,9.0,2.003144,2.003144,2.003144,5.666667,9.0,0.000000,1.832434,1.618484,NaN,3.400000,9.0,0.000000,0.963119,1.130855,NaN,1.922222,9.0,0.000000,0.106996,1.240372,NaN
45489,202309050711,2023-12-23,2019105394,9.0,9.0,9.0,2.640755,2.640755,2.640755,4.203704,9.0,1.666667,2.865669,2.555020,3.880570,2.522222,9.0,0.000000,1.549563,1.850065,NaN,2.372222,9.0,0.000000,1.784803,1.632191,NaN
42068,202305050812,2023-11-26,2017106711,9.0,9.0,9.0,3.162289,3.162289,3.162289,5.666667,9.0,0.000000,2.379638,2.244779,-0.354246,5.000000,9.0,0.000000,1.869880,1.514214,-0.354246,4.133333,9.0,0.000000,1.786639,1.170703,-0.352599
11316,202309020111,2023-03-25,2020103466,9.0,9.0,9.0,2.414950,2.414950,2.414950,3.740741,9.0,0.555556,2.213538,2.443286,1.352494,2.944444,9.0,0.555556,2.238367,2.467377,1.352494,2.944444,9.0,0.555556,2.228052,2.467377,1.352494
30094,202301020411,2023-08-20,2018105165,9.0,9.0,9.0,2.003144,2.003144,2.003144,6.000000,9.0,0.000000,1.994014,1.618484,NaN,5.400000,9.0,0.000000,2.337549,1.130855,NaN,4.527778,9.0,0.000000,2.509924,1.240372,NaN
20467,202305030211,2023-06-04,2019104462,9.0,9.0,9.0,2.217034,2.217034,2.217034,4.481481,9.0,0.000000,1.797878,1.534236,-0.235702,3.188889,9.0,0.000000,1.288446,1.187654,NaN,3.180556,9.0,0.000000,1.606492,1.298919,NaN
10994,202306030111,2023-03-25,2019104706,9.0,9.0,9.0,3.054793,3.054793,3.054793,5.407407,9.0,2.222222,2.247931,1.553724,3.175426,3.688889,9.0,0.000000,2.387373,1.568311,NaN,2.419753,9.0,0.000000,1.622784,1.515159,NaN


In [32]:
merged_df

,race_id,date,horse_id,score_pase_potision_mean_1races_per_score_raw,score_pase_potision_max_1races_per_score_raw,score_pase_potision_min_1races_per_score_raw,score_pase_potision_mean_1races_per_score_raw_relative,score_pase_potision_max_1races_per_score_raw_relative,score_pase_potision_min_1races_per_score_raw_relative,score_pase_potision_mean_3races_per_score_raw,score_pase_potision_max_3races_per_score_raw,score_pase_potision_min_3races_per_score_raw,score_pase_potision_mean_3races_per_score_raw_relative,score_pase_potision_max_3races_per_score_raw_relative,score_pase_potision_min_3races_per_score_raw_relative,score_pase_potision_mean_5races_per_score_raw,score_pase_potision_max_5races_per_score_raw,score_pase_potision_min_5races_per_score_raw,score_pase_potision_mean_5races_per_score_raw_relative,score_pase_potision_max_5races_per_score_raw_relative,score_pase_potision_min_5races_per_score_raw_relative,score_pase_potision_mean_10races_per_score_raw,score_pase_potision_max_10races_per_score_raw,score_pase_potision_min_10races_per_score_raw,score_pase_potision_mean_10races_per_score_raw_relative,score_pase_potision_max_10races_per_score_raw_relative,score_pase_potision_min_10races_per_score_raw_relative
0,202306010101,2023-01-05,2020103575,2.000000,2.000000,2.000000,2.561738,2.561738,2.561738,0.666667,2.000000,0.0,1.789587,2.015564,NaN,0.500000,2.000000,0.0,1.733082,2.015564,NaN,0.500000,2.000000,0.0,1.897193,2.015564,NaN
1,202306010101,2023-01-05,2020107073,2.000000,2.000000,2.000000,2.561738,2.561738,2.561738,0.666667,2.000000,0.0,1.789587,2.015564,NaN,0.666667,2.000000,0.0,2.464083,2.015564,NaN,0.666667,2.000000,0.0,2.679348,2.015564,NaN
2,202306010101,2023-01-05,2020102562,0.000000,0.000000,0.000000,-0.365963,-0.365963,-0.365963,0.000000,0.000000,0.0,-0.461073,-0.465130,NaN,0.000000,0.000000,0.0,-0.459921,-0.465130,NaN,0.000000,0.000000,0.0,-0.449274,-0.465130,NaN
3,202306010101,2023-01-05,2020106345,0.000000,0.000000,0.000000,-0.365963,-0.365963,-0.365963,0.000000,0.000000,0.0,-0.461073,-0.465130,NaN,0.000000,0.000000,0.0,-0.459921,-0.465130,NaN,0.000000,0.000000,0.0,-0.449274,-0.465130,NaN
4,202306010101,2023-01-05,2020100039,0.000000,0.000000,0.000000,-0.365963,-0.365963,-0.365963,0.851852,2.000000,0.0,2.414770,2.015564,NaN,0.511111,2.000000,0.0,1.781815,2.015564,NaN,0.365079,2.000000,0.0,1.264019,2.015564,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46207,202309050912,2023-12-28,2017104873,0.000000,0.000000,0.000000,-0.365963,-0.365963,-0.365963,0.000000,0.000000,0.0,-0.596671,-0.639148,NaN,0.000000,0.000000,0.0,-0.636031,-0.675097,NaN,0.000000,0.000000,0.0,-1.028721,-1.293952,NaN
46208,202309050912,2023-12-28,2019100653,0.000000,0.000000,0.000000,-0.365963,-0.365963,-0.365963,0.000000,0.000000,0.0,-0.596671,-0.639148,NaN,0.000000,0.000000,0.0,-0.636031,-0.675097,NaN,0.000000,0.000000,0.0,-1.028721,-1.293952,NaN
46209,202309050912,2023-12-28,2017106240,2.777778,2.777778,2.777778,2.561738,2.561738,2.561738,0.925926,2.777778,0.0,1.015953,1.358190,NaN,0.555556,2.777778,0.0,0.299309,0.695658,NaN,0.877778,6.000000,0.0,0.560460,1.173983,NaN
46210,202309050912,2023-12-28,2018103205,0.000000,0.000000,0.000000,-0.365963,-0.365963,-0.365963,0.000000,0.000000,0.0,-0.596671,-0.639148,NaN,0.000000,0.000000,0.0,-0.636031,-0.675097,NaN,0.400000,4.000000,0.0,-0.304537,0.351338,NaN


In [421]:

baselog[["race_class","show","pase_category",'race_position']]
baselog

,race_id,date,horse_id,rank_per_horse,date_horse,rank,prize,rank_diff,umaban,wakuban,weather,race_type,course_len,ground_state,race_class,n_horses,time,time_courselen,nobori,win,rentai,show,place,corner_1_per_horse,corner_2_per_horse,corner_3_per_horse,corner_4_per_horse,race_position,pase_diff,pase_category,corner_1,corner_2,corner_3,corner_4,pase_1,pase_2
665397,202309050903,2023-12-28,2021105731,0.857143,2023-12-23,12.0,0.0,1.6,12,7,1,1,1800,0,1,14.0,109.9,0.061056,35.0,0,0,0,9,0.428571,0.571429,NaN,NaN,3,-1.0,2.0,6,8.0,NaN,NaN,35.4,34.4
665438,202309050903,2023-12-28,2021104215,1.000000,2023-12-23,14.0,0.0,2.0,9,6,1,1,1800,0,1,14.0,110.3,0.061278,34.7,0,0,0,9,1.000000,1.000000,NaN,NaN,4,-1.0,2.0,14,14.0,NaN,NaN,35.4,34.4
665455,202309050903,2023-12-28,2021105018,0.933333,2023-12-23,14.0,0.0,3.3,8,5,1,0,1200,0,1,15.0,76.9,0.064083,40.2,0,0,0,9,0.666667,0.800000,NaN,NaN,4,2.4,4.0,10,12.0,NaN,NaN,35.6,38.0
665461,202309050903,2023-12-28,2021102811,1.000000,2023-12-23,17.0,0.0,3.3,10,5,1,1,1400,0,0,17.0,84.3,0.060214,35.9,0,0,0,9,0.764706,0.882353,NaN,NaN,4,0.0,2.0,13,15.0,NaN,NaN,34.8,34.8
657017,202309050707,2023-12-23,2020101582,0.875000,2023-12-17,14.0,0.0,1.2,9,5,1,1,1600,0,2,16.0,95.7,0.059812,36.3,0,0,0,7,0.625000,0.437500,0.4375,NaN,3,0.3,3.0,10,7.0,7.0,NaN,35.4,35.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495603,202309040510,2023-09-18,2011106610,0.375000,2014-01-19,6.0,0.0,0.7,14,7,1,1,1600,0,0,16.0,97.7,0.061062,37.4,0,0,0,6,0.125000,0.062500,0.1250,NaN,1,0.7,3.0,2,1.0,2.0,NaN,36.0,36.7
379650,202307030412,2023-07-09,2011106610,0.375000,2014-01-19,6.0,0.0,0.7,14,7,1,1,1600,0,0,16.0,97.7,0.061062,37.4,0,0,0,6,0.125000,0.062500,0.1250,NaN,1,0.7,3.0,2,1.0,2.0,NaN,36.0,36.7
354277,202309030809,2023-06-25,2011106610,0.375000,2014-01-19,6.0,0.0,0.7,14,7,1,1,1600,0,0,16.0,97.7,0.061062,37.4,0,0,0,6,0.125000,0.062500,0.1250,NaN,1,0.7,3.0,2,1.0,2.0,NaN,36.0,36.7
564911,202308020909,2023-10-29,2011106610,0.375000,2014-01-19,6.0,0.0,0.7,14,7,1,1,1600,0,0,16.0,97.7,0.061062,37.4,0,0,0,6,0.125000,0.062500,0.1250,NaN,1,0.7,3.0,2,1.0,2.0,NaN,36.0,36.7


In [20]:
import re
from pathlib import Path

import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from tqdm.notebook import tqdm
from webdriver_manager.chrome import ChromeDriverManager
from io import StringIO 

DATA_DIR = Path("..", "data")
POPULATION_DIR = DATA_DIR / "00_population"
INPUT_DIR = DATA_DIR / "01_preprocessed"
OUTPUT_DIR = DATA_DIR / "02_features"
OUTPUT_DIR.mkdir(exist_ok=True, parents=True)

population_dir = POPULATION_DIR
poplation_filename = "population.csv"
input_dir = INPUT_DIR
results_filename= "results.csv"
race_info_filename = "race_info.csv"
horse_results_filename = "horse_results.csv"
jockey_leading_filename = "jockey_leading.csv"
trainer_leading_filename = "trainer_leading.csv"
peds_filename = "peds.csv"
sire_leading_filename = "sire_leading.csv"
output_dir = OUTPUT_DIR
output_filename = "features.csv"

population = pd.read_csv(population_dir / poplation_filename, sep="\t")
results = pd.read_csv(input_dir / results_filename, sep="\t")
race_info = pd.read_csv(input_dir / race_info_filename, sep="\t")
horse_results = pd.read_csv(input_dir / horse_results_filename, sep="\t")
jockey_leading = pd.read_csv(input_dir / jockey_leading_filename, sep="\t")
trainer_leading = pd.read_csv(
    input_dir / trainer_leading_filename, sep="\t"
)
peds = pd.read_csv(input_dir / peds_filename, sep="\t")
sire_leading = pd.read_csv(input_dir / sire_leading_filename, sep="\t")
output_dir = output_dir
agg_horse_per_group_cols_dfs = {}

In [82]:
race_info

,race_id,date,race_type,around,course_len,weather,ground_state,race_class,place,season,place_season,place_season_type,kaisai_race,kaisai_race_type,day,day_type,kaisai_count,kaisai_count_type,place_season_day_type,day_condition,place_season_condition_type,place_season_condition_type_categori,place_course_category,place_course_tough,goal_range,curve,goal_slope,place_season_type_course_len,lap_type,race_day_count,race_date_day_count,goal_range_100
0,202306010101,2023-01-05,0,0,1200,1,0,1,6,4,64,640,1,10,1,10,1,10,64010,1,6410,-3,-10000,-10000,-10000,-10000,-10000,6401200,-1,1,73803501,NaN
1,202306010102,2023-01-05,0,0,1800,1,0,1,6,4,64,640,2,20,1,10,1,10,64010,1,6410,-3,-10000,-10000,-10000,-10000,-10000,6401800,-1,2,73803502,NaN
2,202306010103,2023-01-05,0,0,1800,1,0,0,6,4,64,640,3,30,1,10,1,10,64010,1,6410,-3,-10000,-10000,-10000,-10000,-10000,6401800,-1,3,73803503,NaN
3,202306010104,2023-01-05,0,0,1800,1,0,2,6,4,64,640,4,40,1,10,1,10,64010,1,6410,-3,-10000,-10000,-10000,-10000,-10000,6401800,-1,4,73803504,NaN
4,202306010105,2023-01-05,1,0,1600,1,0,1,6,4,64,641,5,51,1,11,1,11,64011,1,6411,3,602,1,310,5,3,6411600,4,5,73803505,3.100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3324,202309050908,2023-12-28,0,0,1400,1,0,2,9,4,94,940,8,80,9,90,5,50,94090,2,9420,-2,-10000,-10000,-10000,-10000,-10000,9401400,-1,8,73839208,NaN
3325,202309050909,2023-12-28,0,0,1800,1,0,3,9,4,94,940,9,90,9,90,5,50,94090,2,9420,-2,-10000,-10000,-10000,-10000,-10000,9401800,-1,9,73839209,NaN
3326,202309050910,2023-12-28,1,0,2000,1,0,3,9,4,94,941,10,101,9,91,5,51,94091,2,9421,2,901,1,358,-10000,-10000,9412000,-1,10,73839210,3.587
3327,202309050911,2023-12-28,0,0,1800,1,0,5,9,4,94,940,11,110,9,90,5,50,94090,2,9420,-2,-10000,-10000,-10000,-10000,-10000,9401800,-1,11,73839211,NaN


In [84]:
curve_counts = race_info["goal_range_100"].value_counts()
print(curve_counts)

goal_range_100
3.100     213
2.930     149
2.920     123
4.736     120
3.587     119
3.450     103
2.661      99
2.621      85
10.000     77
6.587      59
4.037      57
5.259      22
4.125      18
Name: count, dtype: int64


In [47]:
"""
特筆見る

直線が短い差しは不利
長いと先行不利、というわけではない


"""
n_races: list[int] = [1, 3, 5, 10]

baselog = (
    population
    .merge(
        horse_results,
        on="horse_id",
        suffixes=("", "_horse"),
    )
    .merge(
        race_info[["race_id", "goal_range_100"]], 
        on="race_id",
    )
    .query("date_horse < date")
    .sort_values("date_horse", ascending=False)
)
def calculate_scores(baselog):
    # スコア初期化
    baselog["score_goal_range"] = 0.0

    # 直線が短いで差し・追い込み馬がshowに入った場合
    condition1 = (baselog["goal_range_100"] < 3) & (baselog["race_position"].isin([3, 4])) & (baselog["show"] == 1)
    baselog.loc[condition1, "score_goal_range"] += (baselog["race_class"] + 1) / 2

    # 直線が短いで差し・追い込み馬がshowに入った場合
    condition2 = (baselog["goal_range_100"] < 4) & (baselog["goal_range_100"] >= 3) & (baselog["race_position"].isin([3, 4])) & (baselog["show"] == 1)
    baselog.loc[condition2, "score_goal_range"] +=(baselog["race_class"] + 1) / 2.7

    # 直線が短いで差し・追い込み馬がshowに入った場合
    condition3 = (baselog["goal_range_100"] < 4.2) & (baselog["goal_range_100"] >= 4) & (baselog["race_position"].isin([3, 4])) & (baselog["show"] == 1)
    baselog.loc[condition3, "score_goal_range"] +=(baselog["race_class"] + 1) / 3.5

    # 先行馬がshowに入った場合
    condition4 = (baselog["goal_range_100"] >= 0) & (baselog["race_position"].isin([1, 2])) & (baselog["show"] == 1)
    baselog.loc[condition4, "score_goal_range"] += (baselog["race_class"] + 1) / 4.5

    # 直線が短いで差し・追い込み馬がshowに入った場合
    condition5 = (baselog["goal_range_100"] >= 4.2) & (baselog["race_position"].isin([3, 4])) & (baselog["show"] == 1)
    baselog.loc[condition5, "score_goal_range"] +=(baselog["race_class"] + 1) / 4.5

     # showに入りらなかった場合 
    condition7 = (baselog["show"] != 1)
    baselog.loc[condition7, "score_goal_range"] += 0 

    
    return baselog
result_df = calculate_scores(baselog)

grouped_df = result_df.groupby(["race_id", "horse_id"])
merged_df = population.copy()

for n_race in tqdm(n_races, desc=f"agg_goal_per_score"):
    # スコアの集計（標準化しない版）
    raw_df = (
        grouped_df.head(n_race)
        .groupby(["race_id", "horse_id"])[["score_goal_range"]]
        .agg(["mean", "max", "min"])
    )
    raw_df.columns = [
        "_".join(col) + f"_{n_race}races_per_goal_raw" for col in raw_df.columns
    ]
    
    # スコアの集計（標準化する版）
    std_df = raw_df.copy()  # 同じデータを使用して標準化
    tmp_df = raw_df.groupby(["race_id"])
    std_df = ((raw_df - tmp_df.mean()) / tmp_df.std()).add_suffix("_relative")
    
    # merged_df に追加
    merged_df = merged_df.merge(raw_df, on=["race_id", "horse_id"], how="left")
    merged_df = merged_df.merge(std_df, on=["race_id", "horse_id"], how="left")


agg_goal_per_score:   0%|          | 0/4 [00:00<?, ?it/s]

In [50]:

sorted_df = merged_df.sort_values(by="score_goal_range_mean_5races_per_goal_raw", ascending=False)

# 表示 (上位10行のみ)
sorted_df.head(10)


,race_id,date,horse_id,score_goal_range_mean_1races_per_goal_raw,score_goal_range_max_1races_per_goal_raw,score_goal_range_min_1races_per_goal_raw,score_goal_range_mean_1races_per_goal_raw_relative,score_goal_range_max_1races_per_goal_raw_relative,score_goal_range_min_1races_per_goal_raw_relative,score_goal_range_mean_3races_per_goal_raw,score_goal_range_max_3races_per_goal_raw,score_goal_range_min_3races_per_goal_raw,score_goal_range_mean_3races_per_goal_raw_relative,score_goal_range_max_3races_per_goal_raw_relative,score_goal_range_min_3races_per_goal_raw_relative,score_goal_range_mean_5races_per_goal_raw,score_goal_range_max_5races_per_goal_raw,score_goal_range_min_5races_per_goal_raw,score_goal_range_mean_5races_per_goal_raw_relative,score_goal_range_max_5races_per_goal_raw_relative,score_goal_range_min_5races_per_goal_raw_relative,score_goal_range_mean_10races_per_goal_raw,score_goal_range_max_10races_per_goal_raw,score_goal_range_min_10races_per_goal_raw,score_goal_range_mean_10races_per_goal_raw_relative,score_goal_range_max_10races_per_goal_raw_relative,score_goal_range_min_10races_per_goal_raw_relative
45678,202306050811,2023-12-24,2020102899,3.333333,3.333333,3.333333,1.242090,1.242090,1.242090,3.209877,3.333333,2.962963,1.595417,0.818754,1.899661,3.111111,3.333333,2.592593,1.922662,0.772400,2.911932,2.629630,3.333333,0.222222,2.096927,0.632480,0.968246
12431,202309020411,2023-04-02,2018105081,3.333333,3.333333,3.333333,1.421036,1.421036,1.421036,3.209877,3.333333,2.962963,1.839978,1.105299,2.170479,2.962963,3.333333,2.592593,2.026554,0.934262,3.181981,1.888889,3.333333,0.000000,1.558423,0.914412,-0.250000
5961,202309010211,2023-02-12,2019105283,3.333333,3.333333,3.333333,1.785602,1.785602,1.785602,3.209877,3.333333,2.962963,2.813628,1.363200,3.328201,2.859259,3.333333,1.333333,2.835734,1.100568,3.328201,2.419753,3.333333,0.222222,2.762413,1.434264,3.328201
45672,202306050811,2023-12-24,2019104740,2.000000,2.000000,2.000000,0.293225,0.293225,0.293225,2.444444,3.333333,2.000000,0.770259,0.818754,1.013720,2.800000,3.333333,2.000000,1.530901,0.772400,2.163484,2.185185,3.333333,0.666667,1.317950,0.632480,3.550235
23524,202309030811,2023-06-25,2019105219,2.000000,2.000000,2.000000,1.120869,1.120869,1.120869,2.888889,3.333333,2.000000,2.140945,0.955529,2.770335,2.725926,3.333333,2.000000,2.004823,0.925330,3.880570,2.308642,3.333333,0.222222,2.403477,0.806421,3.880570
30094,202301020411,2023-08-20,2018105165,4.500000,4.500000,4.500000,2.125175,2.125175,2.125175,3.000000,4.500000,0.000000,1.709365,1.139192,NaN,2.700000,4.500000,0.000000,1.814400,1.029282,NaN,2.347222,4.500000,0.000000,1.855338,1.098487,NaN
10812,202309011211,2023-03-19,2019104909,3.333333,3.333333,3.333333,3.007238,3.007238,3.007238,2.765432,3.333333,2.000000,2.878144,1.878456,3.474396,2.696296,3.333333,2.000000,2.784481,1.488991,3.474396,1.644444,3.333333,0.000000,2.348277,1.243571,NaN
12427,202309020411,2023-04-02,2019104740,3.333333,3.333333,3.333333,1.421036,1.421036,1.421036,3.333333,3.333333,3.333333,1.953732,1.105299,2.498099,2.622222,3.333333,1.555556,1.638333,0.934262,1.767767,1.861111,3.333333,0.370370,1.503541,0.914412,3.750000
36844,202308020511,2023-10-15,2020103656,3.333333,3.333333,3.333333,1.592919,1.592919,1.592919,3.333333,3.333333,3.333333,2.636385,1.182198,3.733812,2.592593,3.333333,0.370370,3.003172,1.070045,4.006938,2.592593,3.333333,0.370370,3.112806,1.068098,4.006938
30084,202301020411,2023-08-20,2018104541,4.000000,4.000000,4.000000,1.827254,1.827254,1.827254,2.333333,4.000000,0.000000,1.083139,0.865055,NaN,2.500000,4.000000,0.000000,1.576590,0.712760,NaN,2.312500,4.000000,0.000000,1.800133,0.639367,NaN


In [115]:
%autoreload

In [116]:
#まとめて実行
# レース結果テーブルの前処理
results_preprocessed = preprocessing.process_results()
# 馬の過去成績テーブルの加工
horse_results_preprocessed = preprocessing.process_horse_results()
# レース情報テーブルの前処理
race_info_preprocessed = preprocessing.process_race_info()

In [127]:
import re
from pathlib import Path

import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from tqdm.notebook import tqdm
from webdriver_manager.chrome import ChromeDriverManager
from io import StringIO 

DATA_DIR = Path("..", "data")
POPULATION_DIR = DATA_DIR / "00_population"
INPUT_DIR = DATA_DIR / "01_preprocessed"
OUTPUT_DIR = DATA_DIR / "02_features"
OUTPUT_DIR.mkdir(exist_ok=True, parents=True)

population_dir = POPULATION_DIR
poplation_filename = "population.csv"
input_dir = INPUT_DIR
results_filename= "results.csv"
race_info_filename = "race_info.csv"
horse_results_filename = "horse_results.csv"
jockey_leading_filename = "jockey_leading.csv"
trainer_leading_filename = "trainer_leading.csv"
peds_filename = "peds.csv"
sire_leading_filename = "sire_leading.csv"
output_dir = OUTPUT_DIR
output_filename = "features.csv"

population = pd.read_csv(population_dir / poplation_filename, sep="\t")
results = pd.read_csv(input_dir / results_filename, sep="\t")
race_info = pd.read_csv(input_dir / race_info_filename, sep="\t")
horse_results = pd.read_csv(input_dir / horse_results_filename, sep="\t")
jockey_leading = pd.read_csv(input_dir / jockey_leading_filename, sep="\t")
trainer_leading = pd.read_csv(
    input_dir / trainer_leading_filename, sep="\t"
)
peds = pd.read_csv(input_dir / peds_filename, sep="\t")
sire_leading = pd.read_csv(input_dir / sire_leading_filename, sep="\t")
output_dir = output_dir
agg_horse_per_group_cols_dfs = {}

In [ ]:

race_info[["race_id","curve"]]

In [106]:

race_info[race_info["curve"] == 1]


,race_id,date,race_type,around,course_len,weather,ground_state,race_class,place,season,place_season,place_season_type,kaisai_race,kaisai_race_type,day,day_type,kaisai_count,kaisai_count_type,place_season_day_type,day_condition,place_season_condition_type,place_season_condition_type_categori,place_course_category,place_course_tough,goal_range,curve,goal_slope,place_season_type_course_len,lap_type,race_day_count,race_date_day_count,goal_range_100
391,202309010110,2023-02-11,1,0,2000,1,0,4,9,4,94,941,10,101,1,11,1,11,94011,1,9411,3,901,1,358.7,1.0,1.0,9412000,-1,10,73807210,3.587
421,202309010206,2023-02-12,1,0,2000,1,0,1,9,4,94,941,6,61,2,21,1,11,94021,1,9411,3,901,1,358.7,1.0,1.0,9412000,-1,6,73807306,3.587
426,202309010211,2023-02-12,1,0,2200,1,0,7,9,4,94,941,11,111,2,21,1,11,94021,1,9411,3,901,1,358.7,1.0,1.0,9412200,-1,11,73807311,3.587
457,202309010307,2023-02-18,1,0,2000,2,0,2,9,4,94,941,7,71,3,31,1,11,94031,1,9411,3,901,1,358.7,1.0,1.0,9412000,-1,7,73807907,3.587
461,202309010311,2023-02-18,1,0,1400,2,0,6,9,4,94,941,11,111,3,31,1,11,94031,1,9411,3,901,1,358.7,1.0,1.0,9411400,4,11,73807911,3.587
490,202309010406,2023-02-19,1,0,1400,0,2,2,9,4,94,941,6,61,4,41,1,11,94041,2,9421,2,901,1,358.7,1.0,1.0,9411400,4,6,73808006,3.587
523,202309010504,2023-02-25,1,0,2000,1,2,1,9,4,94,941,4,41,5,51,1,11,94051,2,9421,2,901,1,358.7,1.0,1.0,9412000,-1,4,73808604,3.587
528,202309010509,2023-02-25,1,0,2200,1,0,5,9,4,94,941,9,91,5,51,1,11,94051,2,9421,2,901,1,358.7,1.0,1.0,9412200,-1,9,73808609,3.587
531,202309010512,2023-02-25,1,0,1400,1,0,3,9,4,94,941,12,121,5,51,1,11,94051,2,9421,2,901,1,358.7,1.0,1.0,9411400,4,12,73808612,3.587
562,202309010609,2023-02-26,1,0,3000,2,0,4,9,4,94,941,9,91,6,61,1,11,94061,2,9421,2,901,1,358.7,1.0,1.0,9413000,-1,9,73808709,3.587


In [108]:
curve_counts = race_info["curve"].value_counts()
print(curve_counts)

curve
5.0    418
2.0    281
3.0    272
1.0    178
0.0     77
4.0     18
Name: count, dtype: int64


In [128]:
"""
特筆見る

急カーブは小回りよりさらにきつい
小回りで買った差し？　差し不利
小スパイラル　小回りほどではないが、先行有利
スパイラル　小回りほどではないが、先行有利

大回り・複合で勝った先行？　先行不利

"急": 1,
"小回": 2,
"小スパ": 3,
"スパ": 4,
"複合": 5,
None: None 
"""
n_races: list[int] = [1, 3, 5, 10]

baselog = (
    population
    .merge(
        horse_results,
        on="horse_id",
        suffixes=("", "_horse"),
    )
    .merge(
        race_info[["race_id","curve"]], 
        on="race_id",
    )
    .query("date_horse < date")
    .sort_values("date_horse", ascending=False)
)
def calculate_scores(baselog):
    # スコア初期化
    baselog["score_curve_range"] = 0.0

    # # 急カーブで差し・追い込み馬がshowに入った場合
    # condition1 = (baselog["curve"] == 1) & (baselog["race_position"].isin([3, 4])) & (baselog["show"] == 1)
    # baselog.loc[condition1, "score_curve_range"] += (baselog["race_class"] + 1)

    # # 小回りカーブで差し・追い込み馬がshowに入った場合
    # condition2 = (baselog["curve"] == 2) & (baselog["race_position"].isin([3, 4])) & (baselog["show"] == 1)
    # baselog.loc[condition2, "score_curve_range"] += (baselog["race_class"] + 1) / 2
    
    # # 小スパカーブで差し・追い込み馬がshowに入った場合
    # condition3 = (baselog["curve"] == 3) & (baselog["race_position"].isin([3, 4])) & (baselog["show"] == 1)
    # baselog.loc[condition3, "score_curve_range"] += (baselog["race_class"] + 1) / 2.3

    # # スパカーブで差し・追い込み馬がshowに入った場合
    # condition4 = (baselog["curve"] == 4) & (baselog["race_position"].isin([3, 4])) & (baselog["show"] == 1)
    # baselog.loc[condition4, "score_curve_range"] += (baselog["race_class"] + 1) / 2.6

    # # 複合カーブで差し・追い込み馬がshowに入った場合
    # condition5 = (baselog["curve"] == 5) & (baselog["race_position"].isin([3, 4])) & (baselog["show"] == 1)
    # baselog.loc[condition5, "score_curve_range"] += (baselog["race_class"] + 1)/ 5   
    
    # # 複合カーブで先行がshowに入った場合
    # condition6 = (baselog["curve"] == 5) & (baselog["race_position"].isin([1, 2])) & (baselog["show"] == 1)
    # baselog.loc[condition6, "score_curve_range"] += (baselog["race_class"] + 1)   
    
    # # カーブで先行がshowに入った場合
    # condition8 = (baselog["curve"].isin([1, 2,3,4])) & (baselog["race_position"].isin([1, 2])) & (baselog["show"] == 1)
    # baselog.loc[condition8, "score_curve_range"] += (baselog["race_class"] + 1)/  5

    
    # 急カーブで差し・追い込み馬がshowに入った場合
    condition1 = (baselog["curve"] == 1) & (baselog["race_position"].isin([3, 4])) & (baselog["show"] == 1)
    baselog.loc[condition1, "score_curve_range"] += (baselog["race_class"] + 1) / 1.8

    # 小回りカーブで差し・追い込み馬がshowに入った場合
    condition2 = (baselog["curve"] == 2) & (baselog["race_position"].isin([3, 4])) & (baselog["show"] == 1)
    baselog.loc[condition2, "score_curve_range"] += (baselog["race_class"] + 1) / 2.1
    
    # 小スパカーブで差し・追い込み馬がshowに入った場合
    condition3 = (baselog["curve"] == 3) & (baselog["race_position"].isin([3, 4])) & (baselog["show"] == 1)
    baselog.loc[condition3, "score_curve_range"] += (baselog["race_class"] + 1) / 2.4

    # スパカーブで差し・追い込み馬がshowに入った場合
    condition4 = (baselog["curve"] == 4) & (baselog["race_position"].isin([3, 4])) & (baselog["show"] == 1)
    baselog.loc[condition4, "score_curve_range"] += (baselog["race_class"] + 1) / 2.7

    # 複合カーブで差し・追い込み馬がshowに入った場合
    condition5 = (baselog["curve"] == 5) & (baselog["race_position"].isin([3, 4])) & (baselog["show"] == 1)
    baselog.loc[condition5, "score_curve_range"] += (baselog["race_class"] + 1)/ 5   
    
    # 複合カーブで先行がshowに入った場合
    condition6 = (baselog["curve"] == 5) & (baselog["race_position"].isin([1, 2])) & (baselog["show"] == 1)
    baselog.loc[condition6, "score_curve_range"] += (baselog["race_class"] + 1)/ 1.8    
    
    # カーブで先行がshowに入った場合
    condition8 = (baselog["curve"].isin([1, 2,3,4])) & (baselog["race_position"].isin([1, 2])) & (baselog["show"] == 1)
    baselog.loc[condition8, "score_curve_range"] += (baselog["race_class"] + 1)/  5

    
     # showに入りらなかった場合 
    condition7 = (baselog["show"] != 1)
    baselog.loc[condition7, "score_curve_range"] += 0 
    
     # showに入りらなかった場合 
    condition9 = (~baselog["curve"].isin([1, 2, 3, 4])) & (baselog["show"] == 1)
    baselog.loc[condition9, "score_curve_range"] += 0 
    

    return baselog
result_df = calculate_scores(baselog)

grouped_df = result_df.groupby(["race_id", "horse_id"])
merged_df = population.copy()

for n_race in tqdm(n_races, desc=f"agg_curve_per_score"):
    # スコアの集計（標準化しない版）
    raw_df = (
        grouped_df.head(n_race)
        .groupby(["race_id", "horse_id"])[["score_curve_range"]]
        .agg(["mean", "max", "min"])
    )
    raw_df.columns = [
        "_".join(col) + f"_{n_race}races_per_curve_raw" for col in raw_df.columns
    ]
    
    # スコアの集計（標準化する版）
    std_df = raw_df.copy()  # 同じデータを使用して標準化
    tmp_df = raw_df.groupby(["race_id"])
    std_df = ((raw_df - tmp_df.mean()) / tmp_df.std()).add_suffix("_relative")
    
    # merged_df に追加
    merged_df = merged_df.merge(raw_df, on=["race_id", "horse_id"], how="left")
    merged_df = merged_df.merge(std_df, on=["race_id", "horse_id"], how="left")


agg_curve_per_score:   0%|          | 0/4 [00:00<?, ?it/s]

In [113]:
merged_df

,race_id,date,horse_id,score_curve_range_mean_1races_per_curve_raw,score_curve_range_max_1races_per_curve_raw,score_curve_range_min_1races_per_curve_raw,score_curve_range_mean_1races_per_curve_raw_relative,score_curve_range_max_1races_per_curve_raw_relative,score_curve_range_min_1races_per_curve_raw_relative,score_curve_range_mean_3races_per_curve_raw,score_curve_range_max_3races_per_curve_raw,score_curve_range_min_3races_per_curve_raw,score_curve_range_mean_3races_per_curve_raw_relative,score_curve_range_max_3races_per_curve_raw_relative,score_curve_range_min_3races_per_curve_raw_relative,score_curve_range_mean_5races_per_curve_raw,score_curve_range_max_5races_per_curve_raw,score_curve_range_min_5races_per_curve_raw,score_curve_range_mean_5races_per_curve_raw_relative,score_curve_range_max_5races_per_curve_raw_relative,score_curve_range_min_5races_per_curve_raw_relative,score_curve_range_mean_10races_per_curve_raw,score_curve_range_max_10races_per_curve_raw,score_curve_range_min_10races_per_curve_raw,score_curve_range_mean_10races_per_curve_raw_relative,score_curve_range_max_10races_per_curve_raw_relative,score_curve_range_min_10races_per_curve_raw_relative
0,202306010101,2023-01-05,2020103575,0.0,0.0,0.0,NaN,NaN,NaN,0.000000,0.0,0.0,NaN,NaN,NaN,0.0,0.0,0.0,NaN,NaN,NaN,0.000000,0.000000,0.0,NaN,NaN,NaN
1,202306010101,2023-01-05,2020107073,0.0,0.0,0.0,NaN,NaN,NaN,0.000000,0.0,0.0,NaN,NaN,NaN,0.0,0.0,0.0,NaN,NaN,NaN,0.000000,0.000000,0.0,NaN,NaN,NaN
2,202306010101,2023-01-05,2020102562,0.0,0.0,0.0,NaN,NaN,NaN,0.000000,0.0,0.0,NaN,NaN,NaN,0.0,0.0,0.0,NaN,NaN,NaN,0.000000,0.000000,0.0,NaN,NaN,NaN
3,202306010101,2023-01-05,2020106345,0.0,0.0,0.0,NaN,NaN,NaN,0.000000,0.0,0.0,NaN,NaN,NaN,0.0,0.0,0.0,NaN,NaN,NaN,0.000000,0.000000,0.0,NaN,NaN,NaN
4,202306010101,2023-01-05,2020100039,0.0,0.0,0.0,NaN,NaN,NaN,0.000000,0.0,0.0,NaN,NaN,NaN,0.0,0.0,0.0,NaN,NaN,NaN,0.000000,0.000000,0.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46207,202309050912,2023-12-28,2017104873,0.0,0.0,0.0,-0.589392,-0.589392,-0.589392,0.000000,0.0,0.0,-0.814603,-0.791983,-0.344729,0.0,0.0,0.0,-0.931928,-0.890754,-0.25,0.333333,3.333333,0.0,-0.652218,0.367201,NaN
46208,202309050912,2023-12-28,2019100653,0.0,0.0,0.0,-0.589392,-0.589392,-0.589392,0.000000,0.0,0.0,-0.814603,-0.791983,-0.344729,0.0,0.0,0.0,-0.931928,-0.890754,-0.25,0.120000,1.200000,0.0,-1.162217,-1.113167,NaN
46209,202309050912,2023-12-28,2017106240,1.0,1.0,1.0,0.294696,0.294696,0.294696,0.333333,1.0,0.0,-0.310725,-0.141038,-0.344729,0.2,1.0,0.0,-0.486385,-0.283422,-0.25,0.620000,4.000000,0.0,0.033092,0.829815,NaN
46210,202309050912,2023-12-28,2018103205,0.0,0.0,0.0,-0.589392,-0.589392,-0.589392,0.000000,0.0,0.0,-0.814603,-0.791983,-0.344729,0.8,4.0,0.0,0.850245,1.538575,-0.25,0.880000,4.000000,0.0,0.654653,0.829815,NaN


In [130]:

sorted_df = merged_df.sort_values(by="score_curve_range_mean_5races_per_curve_raw", ascending=False)

# 表示 (上位10行のみ)
sorted_df.head(10)


,race_id,date,horse_id,score_curve_range_mean_1races_per_curve_raw,score_curve_range_max_1races_per_curve_raw,score_curve_range_min_1races_per_curve_raw,score_curve_range_mean_1races_per_curve_raw_relative,score_curve_range_max_1races_per_curve_raw_relative,score_curve_range_min_1races_per_curve_raw_relative,score_curve_range_mean_3races_per_curve_raw,score_curve_range_max_3races_per_curve_raw,score_curve_range_min_3races_per_curve_raw,score_curve_range_mean_3races_per_curve_raw_relative,score_curve_range_max_3races_per_curve_raw_relative,score_curve_range_min_3races_per_curve_raw_relative,score_curve_range_mean_5races_per_curve_raw,score_curve_range_max_5races_per_curve_raw,score_curve_range_min_5races_per_curve_raw,score_curve_range_mean_5races_per_curve_raw_relative,score_curve_range_max_5races_per_curve_raw_relative,score_curve_range_min_5races_per_curve_raw_relative,score_curve_range_mean_10races_per_curve_raw,score_curve_range_max_10races_per_curve_raw,score_curve_range_min_10races_per_curve_raw,score_curve_range_mean_10races_per_curve_raw_relative,score_curve_range_max_10races_per_curve_raw_relative,score_curve_range_min_10races_per_curve_raw_relative
45678,202306050811,2023-12-24,2020102899,4.285714,4.285714,4.285714,1.371180,1.371180,1.371180,4.126984,4.285714,3.809524,1.764524,0.755978,2.413839,4.000000,4.285714,3.333333,2.027123,0.720911,3.273404,3.366667,4.285714,0.200000,2.225655,0.593347,0.968246
23526,202309030811,2023-06-25,2019105346,5.000000,5.000000,5.000000,1.371371,1.371371,1.371371,3.148148,5.000000,0.000000,1.952469,0.931599,-0.338915,3.777778,5.000000,0.000000,2.217677,0.857435,-0.242536,2.291111,5.000000,0.000000,1.185640,0.807930,-0.242536
42061,202305050812,2023-11-26,2018103559,4.444444,4.444444,4.444444,1.270185,1.270185,1.270185,2.962963,4.444444,0.000000,1.233300,1.070748,-0.449089,3.777778,5.000000,0.000000,2.089758,1.200703,-0.445639,2.833333,5.000000,0.000000,1.608660,1.126211,-0.388498
16372,202308010411,2023-04-30,2018103559,4.444444,4.444444,4.444444,1.651631,1.651631,1.651631,3.148148,5.000000,0.000000,1.368088,1.229417,-0.346479,3.777778,5.000000,0.000000,1.914286,1.028562,-0.242536,2.888889,5.000000,0.000000,1.934227,0.912361,NaN
34270,202306040711,2023-09-24,2018103559,4.444444,4.444444,4.444444,2.570645,2.570645,2.570645,3.148148,5.000000,0.000000,1.843206,1.505929,NaN,3.777778,5.000000,0.000000,2.657935,1.205952,NaN,2.888889,5.000000,0.000000,2.537437,1.039375,NaN
19122,202305021011,2023-05-21,2020103364,5.000000,5.000000,5.000000,1.865924,1.865924,1.865924,3.333333,5.000000,0.555556,2.420346,1.756891,0.619141,3.333333,5.000000,0.555556,2.546224,1.682874,1.435415,3.333333,5.000000,0.555556,2.721039,1.682874,1.435415
9824,202309011011,2023-03-12,2020100686,3.333333,3.333333,3.333333,2.904981,2.904981,2.904981,3.333333,3.333333,3.333333,3.271321,2.023682,3.753537,3.333333,3.333333,3.333333,3.493911,2.025866,3.946544,3.333333,3.333333,3.333333,3.520678,2.086664,3.946544
31562,202301020711,2023-09-02,2021102388,3.333333,3.333333,3.333333,2.756101,2.756101,2.756101,3.333333,3.333333,3.333333,2.786750,2.756101,2.782824,3.333333,3.333333,3.333333,2.786750,2.756101,2.782824,3.333333,3.333333,3.333333,2.786750,2.756101,2.782824
12914,202309020511,2023-04-08,2018101646,3.888889,3.888889,3.888889,2.139443,2.139443,2.139443,3.703704,3.888889,3.333333,2.549212,1.503190,2.815399,3.333333,3.888889,2.777778,2.614204,1.068356,3.175426,2.615556,3.888889,0.600000,2.512239,0.891270,3.175426
18122,202305020811,2023-05-14,2018105233,5.000000,5.000000,5.000000,1.842843,1.842843,1.842843,2.200000,5.000000,0.000000,1.563192,1.537460,-0.357874,3.320000,5.000000,0.000000,2.239799,1.388932,-0.357874,2.604444,5.000000,0.000000,2.228947,1.261661,-0.250000


In [142]:
"""
特筆見る

坂のあるところ
あるなら差しが有利、ただしペースが早いと先行有利
ないならどっちも
だから、
坂があってスローペースや普通のとき、差しが有利、先行評価
坂があって、ハイペースのとき、先行有利、差し評価
ゆるさかは/1.5
ないなら同評価
"平坦": 1,        
"緩坂": 2,
"急坂": 3
"""
n_races: list[int] = [1, 3, 5, 10]

baselog = (
    population
    .merge(
        horse_results,
        on="horse_id",
        suffixes=("", "_horse"),
    )
    .merge(
        race_info[["race_id","goal_slope"]], 
        on="race_id",
    )
    .query("date_horse < date")
    .sort_values("date_horse", ascending=False)
)
def calculate_scores(baselog):
    # スコア初期化
    baselog["score_goal_slope"] = 0.0


    # 平坦の時、特に評価なし
    condition1 = (baselog["goal_slope"] == 1) & (baselog["race_position"].isin([1,2,3, 4])) & (baselog["show"] == 1)
    baselog.loc[condition1, "score_goal_slope"] += (baselog["race_class"] + 1) /5

    # ゆる坂ペースが早いとき
    condition2 = (baselog["goal_slope"] == 2) & (baselog["race_position"].isin([3, 4])) & baselog["pase_category"].isin([4]) & (baselog["show"] == 1)
    baselog.loc[condition2, "score_goal_slope"] += (baselog["race_class"] + 1) / 2.2
    
    # ゆる坂ペースが遅いとき
    condition3 = (baselog["goal_slope"] == 2) & (baselog["race_position"].isin([1,2])) & baselog["pase_category"].isin([1]) & (baselog["show"] == 1)
    baselog.loc[condition3, "score_goal_slope"] += (baselog["race_class"] + 1) / 2.2

    # ゆる坂ペースが早いとき
    condition4 = (baselog["goal_slope"] == 2) & (baselog["race_position"].isin([3, 4])) & baselog["pase_category"].isin([3]) & (baselog["show"] == 1)
    baselog.loc[condition4, "score_goal_slope"] += (baselog["race_class"] + 1) / 3.0
    
    # ゆる坂ペースが遅いとき
    condition5 = (baselog["goal_slope"] == 2) & (baselog["race_position"].isin([1,2])) & baselog["pase_category"].isin([2]) & (baselog["show"] == 1)
    baselog.loc[condition5, "score_goal_slope"] += (baselog["race_class"] + 1) / 3.0


    # ゆる坂ペースそれ以外
    condition6 = (baselog["goal_slope"] == 2) & (baselog["race_position"].isin([1,2])) & baselog["pase_category"].isin([3,4]) & (baselog["show"] == 1)
    baselog.loc[condition6, "score_goal_slope"] += (baselog["race_class"] + 1) /4.5

    # ゆる坂ペースそれ以外
    condition7 = (baselog["goal_slope"] == 2) & (baselog["race_position"].isin([3, 4])) & baselog["pase_category"].isin([1,2]) & (baselog["show"] == 1)
    baselog.loc[condition7, "score_goal_slope"] += (baselog["race_class"] + 1) /4.5
    
    
    
    # 厳しい坂でペースが早いとき
    condition8 = (baselog["goal_slope"] == 3) & (baselog["race_position"].isin([3, 4])) & baselog["pase_category"].isin([4]) & (baselog["show"] == 1)
    baselog.loc[condition8, "score_goal_slope"] += (baselog["race_class"] + 1) /1.5
    
    # 厳しい坂ペースが遅いとき
    condition9 = (baselog["goal_slope"] == 3) & (baselog["race_position"].isin([1,2])) & baselog["pase_category"].isin([1]) & (baselog["show"] == 1)
    baselog.loc[condition9, "score_goal_slope"] += (baselog["race_class"] + 1) /1.5

    # 厳しい坂でペースが早いとき
    condition10 = (baselog["goal_slope"] == 3) & (baselog["race_position"].isin([3, 4])) & baselog["pase_category"].isin([3]) & (baselog["show"] == 1)
    baselog.loc[condition10, "score_goal_slope"] += (baselog["race_class"] + 1) / 2.0
    
    # 厳しい坂ペースが遅いとき
    condition11 = (baselog["goal_slope"] == 3) & (baselog["race_position"].isin([1,2])) & baselog["pase_category"].isin([2]) & (baselog["show"] == 1)
    baselog.loc[condition11, "score_goal_slope"] += (baselog["race_class"] + 1) / 2.0

    
    # 厳しい坂ペースそれ以外
    condition12 = (baselog["goal_slope"] == 3) & (baselog["race_position"].isin([1,2])) & baselog["pase_category"].isin([3,4]) & (baselog["show"] == 1)
    baselog.loc[condition12, "score_goal_slope"] += (baselog["race_class"] + 1) / 3.0

    # 厳しい坂ペースそれ以外
    condition13 = (baselog["goal_slope"] == 3) & (baselog["race_position"].isin([3, 4])) & baselog["pase_category"].isin([1,2]) & (baselog["show"] == 1)
    baselog.loc[condition13, "score_goal_slope"] += (baselog["race_class"] + 1) / 3.0

    

     # showに入りらなかった場合 
    condition14 = (baselog["show"] != 1)
    baselog.loc[condition14, "score_goal_slope"] += 0 
    
     # showに入りらなかった場合 
    condition15 = (~baselog["goal_slope"].isin([1, 2, 3])) & (baselog["show"] == 1)
    baselog.loc[condition15, "score_goal_slope"] += 0 
    

    return baselog
result_df = calculate_scores(baselog)

grouped_df = result_df.groupby(["race_id", "horse_id"])
merged_df = population.copy()

for n_race in tqdm(n_races, desc=f"agg_goal_slope_per_score"):
    # スコアの集計（標準化しない版）
    raw_df = (
        grouped_df.head(n_race)
        .groupby(["race_id", "horse_id"])[["score_goal_slope"]]
        .agg(["mean", "max", "min"])
    )
    raw_df.columns = [
        "_".join(col) + f"_{n_race}races_per_goal_slope_raw" for col in raw_df.columns
    ]
    
    # スコアの集計（標準化する版）
    std_df = raw_df.copy()  # 同じデータを使用して標準化
    tmp_df = raw_df.groupby(["race_id"])
    std_df = ((raw_df - tmp_df.mean()) / tmp_df.std()).add_suffix("_relative")
    
    # merged_df に追加
    merged_df = merged_df.merge(raw_df, on=["race_id", "horse_id"], how="left")
    merged_df = merged_df.merge(std_df, on=["race_id", "horse_id"], how="left")


agg_goal_slope_per_score:   0%|          | 0/4 [00:00<?, ?it/s]

In [138]:
merged_df

,race_id,date,horse_id,score_goal_slope_mean_1races_per_goal_slope_raw,score_goal_slope_max_1races_per_goal_slope_raw,score_goal_slope_min_1races_per_goal_slope_raw,score_goal_slope_mean_1races_per_goal_slope_raw_relative,score_goal_slope_max_1races_per_goal_slope_raw_relative,score_goal_slope_min_1races_per_goal_slope_raw_relative,score_goal_slope_mean_3races_per_goal_slope_raw,score_goal_slope_max_3races_per_goal_slope_raw,score_goal_slope_min_3races_per_goal_slope_raw,score_goal_slope_mean_3races_per_goal_slope_raw_relative,score_goal_slope_max_3races_per_goal_slope_raw_relative,score_goal_slope_min_3races_per_goal_slope_raw_relative,score_goal_slope_mean_5races_per_goal_slope_raw,score_goal_slope_max_5races_per_goal_slope_raw,score_goal_slope_min_5races_per_goal_slope_raw,score_goal_slope_mean_5races_per_goal_slope_raw_relative,score_goal_slope_max_5races_per_goal_slope_raw_relative,score_goal_slope_min_5races_per_goal_slope_raw_relative,score_goal_slope_mean_10races_per_goal_slope_raw,score_goal_slope_max_10races_per_goal_slope_raw,score_goal_slope_min_10races_per_goal_slope_raw,score_goal_slope_mean_10races_per_goal_slope_raw_relative,score_goal_slope_max_10races_per_goal_slope_raw_relative,score_goal_slope_min_10races_per_goal_slope_raw_relative
0,202306010101,2023-01-05,2020103575,0.0,0.0,0.0,NaN,NaN,NaN,0.000000,0.0,0.0,NaN,NaN,NaN,0.0,0.0,0.0,NaN,NaN,NaN,0.000000,0.000000,0.0,NaN,NaN,NaN
1,202306010101,2023-01-05,2020107073,0.0,0.0,0.0,NaN,NaN,NaN,0.000000,0.0,0.0,NaN,NaN,NaN,0.0,0.0,0.0,NaN,NaN,NaN,0.000000,0.000000,0.0,NaN,NaN,NaN
2,202306010101,2023-01-05,2020102562,0.0,0.0,0.0,NaN,NaN,NaN,0.000000,0.0,0.0,NaN,NaN,NaN,0.0,0.0,0.0,NaN,NaN,NaN,0.000000,0.000000,0.0,NaN,NaN,NaN
3,202306010101,2023-01-05,2020106345,0.0,0.0,0.0,NaN,NaN,NaN,0.000000,0.0,0.0,NaN,NaN,NaN,0.0,0.0,0.0,NaN,NaN,NaN,0.000000,0.000000,0.0,NaN,NaN,NaN
4,202306010101,2023-01-05,2020100039,0.0,0.0,0.0,NaN,NaN,NaN,0.000000,0.0,0.0,NaN,NaN,NaN,0.0,0.0,0.0,NaN,NaN,NaN,0.000000,0.000000,0.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46207,202309050912,2023-12-28,2017104873,0.0,0.0,0.0,-0.721361,-0.721361,-0.721361,0.000000,0.0,0.0,-0.826597,-0.905047,-0.353553,0.0,0.0,0.0,-0.873795,-0.987117,-0.25,0.166667,1.666667,0.0,-1.200201,-0.814847,NaN
46208,202309050912,2023-12-28,2019100653,0.0,0.0,0.0,-0.721361,-0.721361,-0.721361,0.000000,0.0,0.0,-0.826597,-0.905047,-0.353553,0.0,0.0,0.0,-0.873795,-0.987117,-0.25,0.200000,2.000000,0.0,-1.120483,-0.537452,NaN
46209,202309050912,2023-12-28,2017106240,2.5,2.5,2.5,1.751876,1.751876,1.751876,0.833333,2.5,0.0,0.326794,1.144116,-0.353553,0.5,2.5,0.0,-0.024763,0.780854,-0.25,0.850000,4.000000,0.0,0.434021,1.126915,NaN
46210,202309050912,2023-12-28,2018103205,0.0,0.0,0.0,-0.721361,-0.721361,-0.721361,0.000000,0.0,0.0,-0.826597,-0.905047,-0.353553,0.4,2.0,0.0,-0.194570,0.427260,-0.25,0.666667,2.666667,0.0,-0.004429,0.017337,NaN


In [143]:

sorted_df = merged_df.sort_values(by="score_goal_slope_mean_5races_per_goal_slope_raw_relative", ascending=False)

# 表示 (上位10行のみ)
sorted_df.head(10)

,race_id,date,horse_id,score_goal_slope_mean_1races_per_goal_slope_raw,score_goal_slope_max_1races_per_goal_slope_raw,score_goal_slope_min_1races_per_goal_slope_raw,score_goal_slope_mean_1races_per_goal_slope_raw_relative,score_goal_slope_max_1races_per_goal_slope_raw_relative,score_goal_slope_min_1races_per_goal_slope_raw_relative,score_goal_slope_mean_3races_per_goal_slope_raw,score_goal_slope_max_3races_per_goal_slope_raw,score_goal_slope_min_3races_per_goal_slope_raw,score_goal_slope_mean_3races_per_goal_slope_raw_relative,score_goal_slope_max_3races_per_goal_slope_raw_relative,score_goal_slope_min_3races_per_goal_slope_raw_relative,score_goal_slope_mean_5races_per_goal_slope_raw,score_goal_slope_max_5races_per_goal_slope_raw,score_goal_slope_min_5races_per_goal_slope_raw,score_goal_slope_mean_5races_per_goal_slope_raw_relative,score_goal_slope_max_5races_per_goal_slope_raw_relative,score_goal_slope_min_5races_per_goal_slope_raw_relative,score_goal_slope_mean_10races_per_goal_slope_raw,score_goal_slope_max_10races_per_goal_slope_raw,score_goal_slope_min_10races_per_goal_slope_raw,score_goal_slope_mean_10races_per_goal_slope_raw_relative,score_goal_slope_max_10races_per_goal_slope_raw_relative,score_goal_slope_min_10races_per_goal_slope_raw_relative
24782,202307030307,2023-07-08,2020103587,0.000000,0.000000,0.000000,-0.235702,-0.235702,-0.235702,0.444444,1.333333,0.000000,3.563359,3.563359,NaN,0.600000,1.333333,0.000000,3.890415,3.482600,NaN,0.600000,1.333333,0.000000,3.620375,2.539334,NaN
1448,202306010506,2023-01-14,2020105835,0.666667,0.666667,0.666667,3.880570,3.880570,3.880570,0.666667,1.333333,0.000000,3.674287,3.449772,NaN,0.533333,1.333333,0.000000,3.761613,3.449772,NaN,0.533333,1.333333,0.000000,3.843180,3.449772,NaN
5415,202309010105,2023-02-11,2020104572,0.666667,0.666667,0.666667,3.750000,3.750000,3.750000,0.888889,1.333333,0.666667,3.684462,3.634872,3.750000,0.750000,1.333333,0.333333,3.658488,3.634872,3.750000,0.750000,1.333333,0.333333,3.658488,3.634872,3.750000
25447,202302020511,2023-07-15,2021102392,0.400000,0.400000,0.400000,1.932184,1.932184,1.932184,0.300000,0.400000,0.200000,3.614784,1.932184,0.378932,0.300000,0.400000,0.200000,3.614784,1.932184,0.378932,0.300000,0.400000,0.200000,3.614784,1.932184,0.378932
20834,202305030304,2023-06-10,2020102542,0.444444,0.444444,0.444444,2.656845,2.656845,2.656845,0.451178,0.909091,0.000000,3.177770,3.391192,NaN,0.492929,0.909091,0.000000,3.599502,3.327431,NaN,0.447330,0.909091,0.000000,3.724837,3.327431,NaN
25531,202303020507,2023-07-15,2020102190,0.400000,0.400000,0.400000,3.750000,3.750000,3.750000,0.400000,0.400000,0.400000,3.550235,2.561738,3.750000,0.320000,0.400000,0.000000,3.537638,2.015564,NaN,0.266667,0.400000,0.000000,3.612451,1.919672,NaN
26429,202303020703,2023-07-22,2020101586,0.400000,0.400000,0.400000,3.614784,3.614784,3.614784,0.400000,0.400000,0.400000,3.421762,2.463060,3.614784,0.400000,0.400000,0.400000,3.481788,1.932184,3.614784,0.400000,0.400000,0.400000,3.481788,1.932184,3.614784
30697,202304030507,2023-08-26,2020102819,0.400000,0.400000,0.400000,2.173067,2.173067,2.173067,0.133333,0.400000,0.000000,2.173067,2.173067,NaN,0.200000,0.400000,0.000000,3.475478,2.173067,NaN,0.200000,0.400000,0.000000,3.183518,1.818119,NaN
38888,202305050103,2023-11-04,2021103659,0.222222,0.222222,0.222222,3.474396,3.474396,3.474396,0.222222,0.222222,0.222222,3.474396,3.474396,3.474396,0.222222,0.222222,0.222222,3.474396,3.474396,3.474396,0.222222,0.222222,0.222222,3.474396,3.474396,3.474396
40113,202303030402,2023-11-12,2021104924,0.400000,0.400000,0.400000,3.750000,3.750000,3.750000,0.266667,0.400000,0.000000,3.501049,3.332380,NaN,0.250000,0.400000,0.000000,3.469640,3.332380,NaN,0.250000,0.400000,0.000000,3.469640,3.332380,NaN


In [174]:
"""
特筆見る

スタミナ指標
かっているとして
長ければ+
小回りで-
大回りでさらに+
⑤大回りのコースは、カーブで息が入りにくいので、距離に不安のある馬には厳しい


race_class*コースの長さ/2000

こいつは相対値以外も出していい

"""
n_races: list[int] = [1, 3, 5, 10]

baselog = (
    population
    .merge(
        horse_results,
        on="horse_id",
        suffixes=("", "_horse"),
    )
    .merge(
        race_info[["race_id","curve",'place_season_condition_type_categori']], 
        on="race_id",
    )
    .query("date_horse < date")
    .sort_values("date_horse", ascending=False)
)
def calculate_scores(baselog):
    # スコア初期化
    baselog["score_stamina"] = 0.0



    # 急カーブ場合
    condition1 = (baselog["curve"] == 1) & (baselog["show"] == 1)
    baselog.loc[condition1, "score_stamina"] += (baselog["race_class"] + 1+(2/baselog['place_season_condition_type_categori'])) *baselog["course_len"]/2000/ 2.5 

    # 小回りカーブで場合
    condition2 = (baselog["curve"] == 2)  & (baselog["show"] == 1)
    baselog.loc[condition2, "score_stamina"] += (baselog["race_class"] + 1+(2/baselog['place_season_condition_type_categori'])) *baselog["course_len"]/2000/ 2.1
    
    # 小スパカーブ場合
    condition3 = (baselog["curve"] == 3) & (baselog["show"] == 1)
    baselog.loc[condition3, "score_stamina"] += (baselog["race_class"] + 1+(2/baselog['place_season_condition_type_categori'])) *baselog["course_len"]/2000/ 1.8

    # スパカーブ場合
    condition4 = (baselog["curve"] == 4)  & (baselog["show"] == 1)
    baselog.loc[condition4, "score_stamina"] += (baselog["race_class"] + 1+(2/baselog['place_season_condition_type_categori'])) *baselog["course_len"]/2000/ 1.5

    # 複合カーブで場合
    condition5 = (baselog["curve"] == 5)  & (baselog["show"] == 1)
    baselog.loc[condition5, "score_stamina"] += (baselog["race_class"] + 1+(2/baselog['place_season_condition_type_categori']))*baselog["course_len"]/2000
    
    
     # showに入りらなかった場合 
    condition7 = (baselog["show"] != 1)
    baselog.loc[condition7, "score_stamina"] += 0 
    
     # showに入りらなかった場合 
    condition8 = (~baselog["curve"].isin([1, 2, 3, 4])) & (baselog["show"] == 1)
    baselog.loc[condition8, "score_stamina"] += 0 
    

    return baselog
result_df = calculate_scores(baselog)

grouped_df = result_df.groupby(["race_id", "horse_id"])
merged_df = population.copy()

for n_race in tqdm(n_races, desc=f"agg_stamina_per_score"):
    # スコアの集計（標準化しない版）
    raw_df = (
        grouped_df.head(n_race)
        .groupby(["race_id", "horse_id"])[["score_stamina"]]
        .agg(["mean", "max", "min"])
    )
    raw_df.columns = [
        "_".join(col) + f"_{n_race}races_per_stamina_raw" for col in raw_df.columns
    ]
    
    # スコアの集計（標準化する版）
    std_df = raw_df.copy()  # 同じデータを使用して標準化
    tmp_df = raw_df.groupby(["race_id"])
    std_df = ((raw_df - tmp_df.mean()) / tmp_df.std()).add_suffix("_relative")
    
    # merged_df に追加
    merged_df = merged_df.merge(raw_df, on=["race_id", "horse_id"], how="left")
    merged_df = merged_df.merge(std_df, on=["race_id", "horse_id"], how="left")

merged_df2 = merged_df.merge(race_info[["race_id","course_len"]], on="race_id")
merged_df2["course_len_relative"] = (merged_df2["course_len"] / 1800) -1
merged_df2.loc[merged_df2["course_len_relative"] < 0, "course_len_relative"] = 0
merged_df2["course_len_relative"] = merged_df2["course_len_relative"] * 5
# 対象の列名リスト
columns_to_multiply = [
    'score_stamina_mean_1races_per_stamina_raw',
    'score_stamina_max_1races_per_stamina_raw',
    'score_stamina_min_1races_per_stamina_raw',
    'score_stamina_mean_1races_per_stamina_raw_relative',
    'score_stamina_max_1races_per_stamina_raw_relative',
    'score_stamina_min_1races_per_stamina_raw_relative',
    'score_stamina_mean_3races_per_stamina_raw',
    'score_stamina_max_3races_per_stamina_raw',
    'score_stamina_min_3races_per_stamina_raw',
    'score_stamina_mean_3races_per_stamina_raw_relative',
    'score_stamina_max_3races_per_stamina_raw_relative',
    'score_stamina_min_3races_per_stamina_raw_relative',
    'score_stamina_mean_5races_per_stamina_raw',
    'score_stamina_max_5races_per_stamina_raw',
    'score_stamina_min_5races_per_stamina_raw',
    'score_stamina_mean_5races_per_stamina_raw_relative',
    'score_stamina_max_5races_per_stamina_raw_relative',
    'score_stamina_min_5races_per_stamina_raw_relative',
    'score_stamina_mean_10races_per_stamina_raw',
    'score_stamina_max_10races_per_stamina_raw',
    'score_stamina_min_10races_per_stamina_raw',
    'score_stamina_mean_10races_per_stamina_raw_relative',
    'score_stamina_max_10races_per_stamina_raw_relative',
    'score_stamina_min_10races_per_stamina_raw_relative'
]

# 'course_len_relative'を各列に掛け算
for col in columns_to_multiply:
    merged_df2[col] = merged_df2[col] * merged_df2['course_len_relative']
# 'course_len'と'course_len_relative'の列を削除
merged_df2 = merged_df2.drop(columns=['course_len', 'course_len_relative'])


agg_stamina_per_score:   0%|          | 0/4 [00:00<?, ?it/s]

In [176]:
sorted_df = merged_df2.sort_values(by="score_stamina_mean_5races_per_stamina_raw_relative", ascending=False)

# 表示 (上位10行のみ)
sorted_df.head(20)

,race_id,date,horse_id,score_stamina_mean_1races_per_stamina_raw,score_stamina_max_1races_per_stamina_raw,score_stamina_min_1races_per_stamina_raw,score_stamina_mean_1races_per_stamina_raw_relative,score_stamina_max_1races_per_stamina_raw_relative,score_stamina_min_1races_per_stamina_raw_relative,score_stamina_mean_3races_per_stamina_raw,score_stamina_max_3races_per_stamina_raw,score_stamina_min_3races_per_stamina_raw,score_stamina_mean_3races_per_stamina_raw_relative,score_stamina_max_3races_per_stamina_raw_relative,score_stamina_min_3races_per_stamina_raw_relative,score_stamina_mean_5races_per_stamina_raw,score_stamina_max_5races_per_stamina_raw,score_stamina_min_5races_per_stamina_raw,score_stamina_mean_5races_per_stamina_raw_relative,score_stamina_max_5races_per_stamina_raw_relative,score_stamina_min_5races_per_stamina_raw_relative,score_stamina_mean_10races_per_stamina_raw,score_stamina_max_10races_per_stamina_raw,score_stamina_min_10races_per_stamina_raw,score_stamina_mean_10races_per_stamina_raw_relative,score_stamina_max_10races_per_stamina_raw_relative,score_stamina_min_10races_per_stamina_raw_relative
6316,202305010711,2023-02-18,2018100255,0.000000,0.000000,0.000000,-4.070565,-4.070565,-4.070565,40.000000,66.666667,0.000000,8.929541,6.420956,-3.198320,51.733333,69.333333,0.000000,18.314492,2.878272,NaN,39.333333,69.333333,0.0,17.383669,-2.081753,NaN
42399,202306050111,2023-12-02,2018101348,0.000000,0.000000,0.000000,-4.579106,-4.579106,-4.579106,18.888889,56.666667,0.000000,8.754774,16.199012,-2.500000,24.142857,56.666667,0.000000,16.217564,11.254953,NaN,16.928571,56.666667,0.0,10.858736,8.824426,NaN
38663,202308020910,2023-10-29,2018104638,34.666667,34.666667,34.666667,12.826820,12.826820,12.826820,28.888889,52.000000,0.000000,12.355817,7.635870,NaN,37.333333,52.000000,0.000000,16.195687,5.861221,NaN,29.700000,52.000000,0.0,11.710464,2.744138,NaN
10812,202309011211,2023-03-19,2019104909,75.000000,75.000000,75.000000,18.970042,18.970042,18.970042,74.555556,90.000000,58.666667,16.543815,9.063744,23.162641,65.266667,90.000000,44.000000,16.059992,8.845910,23.162641,38.366667,90.000000,0.0,10.812711,7.360953,NaN
25531,202303020507,2023-07-15,2020102190,4.938272,4.938272,4.938272,16.666667,16.666667,16.666667,5.596708,5.925926,4.938272,15.754075,11.659336,16.666667,4.543210,5.925926,0.000000,15.760949,9.430115,NaN,3.347051,5.925926,0.0,15.984690,9.272533,NaN
37770,202308020711,2023-10-22,2020102899,58.666667,58.666667,58.666667,6.476924,6.476924,6.476924,63.555556,72.000000,58.666667,12.168376,6.914769,17.407521,48.666667,72.000000,6.000000,14.719919,9.322069,16.629691,48.666667,72.000000,6.0,16.579177,9.322069,16.629691
26961,202303020808,2023-07-23,2019100221,19.259259,19.259259,19.259259,14.361123,14.361123,14.361123,18.271605,19.259259,16.296296,14.532563,12.256669,15.441761,10.962963,19.259259,0.000000,13.853757,11.908502,NaN,7.259259,19.259259,0.0,13.455895,12.668424,NaN
16362,202308010411,2023-04-30,2019104909,93.333333,93.333333,93.333333,8.809083,8.809083,8.809083,95.277778,105.000000,87.500000,14.878840,8.484012,23.105443,81.122222,105.000000,51.333333,13.545117,6.194818,30.182211,54.094444,105.000000,0.0,11.373643,6.193507,NaN
30801,202310030502,2023-08-26,2020105550,4.938272,4.938272,4.938272,16.666667,16.666667,16.666667,3.456790,5.432099,0.000000,11.306339,6.352376,NaN,3.851852,5.432099,0.000000,13.439943,5.723607,NaN,3.259259,5.432099,0.0,12.619770,5.723607,NaN
302,202307010110,2023-01-05,2017100084,0.000000,0.000000,0.000000,-1.666667,-1.666667,-1.666667,11.111111,33.333333,0.000000,7.354426,10.418319,-1.666667,16.800000,33.333333,0.000000,13.377079,8.073567,NaN,14.000000,33.333333,0.0,9.987976,3.033819,NaN


In [ ]:
%autoreload

In [ ]:
#まとめて実行
# レース結果テーブルの前処理
results_preprocessed = preprocessing.process_results()
# 馬の過去成績テーブルの加工
horse_results_preprocessed = preprocessing.process_horse_results()
# レース情報テーブルの前処理
race_info_preprocessed = preprocessing.process_race_info()

In [7]:
import re
from pathlib import Path

import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from tqdm.notebook import tqdm
from webdriver_manager.chrome import ChromeDriverManager
from io import StringIO 

DATA_DIR = Path("..", "data")
POPULATION_DIR = DATA_DIR / "00_population"
INPUT_DIR = DATA_DIR / "01_preprocessed"
OUTPUT_DIR = DATA_DIR / "02_features"
OUTPUT_DIR.mkdir(exist_ok=True, parents=True)

population_dir = POPULATION_DIR
poplation_filename = "population.csv"
input_dir = INPUT_DIR
results_filename= "results.csv"
race_info_filename = "race_info.csv"
horse_results_filename = "horse_results.csv"
jockey_leading_filename = "jockey_leading.csv"
trainer_leading_filename = "trainer_leading.csv"
peds_filename = "peds.csv"
sire_leading_filename = "sire_leading.csv"
output_dir = OUTPUT_DIR
output_filename = "features.csv"

population = pd.read_csv(population_dir / poplation_filename, sep="\t")
results = pd.read_csv(input_dir / results_filename, sep="\t")
race_info = pd.read_csv(input_dir / race_info_filename, sep="\t")
horse_results = pd.read_csv(input_dir / horse_results_filename, sep="\t")
jockey_leading = pd.read_csv(input_dir / jockey_leading_filename, sep="\t")
trainer_leading = pd.read_csv(
    input_dir / trainer_leading_filename, sep="\t"
)
peds = pd.read_csv(input_dir / peds_filename, sep="\t")
sire_leading = pd.read_csv(input_dir / sire_leading_filename, sep="\t")
output_dir = output_dir
agg_horse_per_group_cols_dfs = {}

In [47]:
"""
特筆見る
オールスコア
コースの長さが長い時だけ、スタミナ指標を追加したやつも入れる
"""
n_races: list[int] = [1, 3, 5, 10]

baselog = (
    population
    .merge(
        horse_results,
        on="horse_id",
        suffixes=("", "_horse"),
    )
    .query("date_horse < date")
    .sort_values("date_horse", ascending=False)
)
def calculate_scores(baselog):
    # スコア初期化
    baselog["score_pase_potision"] = 0.0

    # スローペースで差し・追い込み馬がshowに入った場合
    condition1 = (baselog["pase_category"] == 1) & (baselog["race_position"].isin([3, 4])) & (baselog["show"] == 1)
    baselog.loc[condition1, "score_pase_potision"] += baselog["race_class"] + 1

    # ハイペースで逃げ・先行馬がshowに入った場合
    condition2 = (baselog["pase_category"] == 4) & (baselog["race_position"].isin([1, 2])) & (baselog["show"] == 1)
    baselog.loc[condition2, "score_pase_potision"] += baselog["race_class"] + 1

    # ミドルスローペースで差し・追い込み馬がshowに入った場合
    condition3 = (baselog["pase_category"] == 2) & (baselog["race_position"].isin([3, 4])) & (baselog["show"] == 1)
    baselog.loc[condition3, "score_pase_potision"] += (baselog["race_class"] + 1) / 1.8

    # ミドルハイペースで逃げ・先行馬がshowに入った場合
    condition4 = (baselog["pase_category"] == 3) & (baselog["race_position"].isin([1, 2])) & (baselog["show"] == 1)
    baselog.loc[condition4, "score_pase_potision"] += (baselog["race_class"] + 1) / 1.8

    # 逃げ・先行馬がshowに入った場合
    condition5 =baselog["pase_category"].isin([1, 2]) & (baselog["race_position"].isin([1, 2])) & (baselog["show"] == 1)
    baselog.loc[condition5, "score_pase_potision"] += (baselog["race_class"] + 1) / 3.6
    # 差し・追い込み馬showに入った場合
    condition6 =baselog["pase_category"].isin([3, 4]) & (baselog["race_position"].isin([3, 4])) & (baselog["show"] == 1)
    baselog.loc[condition6, "score_pase_potision"] += (baselog["race_class"] + 1) / 3.6    

     # showに入りらなかった場合 
    condition7 = (baselog["show"] != 1)
    baselog.loc[condition7, "score_pase_potision"] += 0 
   
    # # その他のペースでshowに入った場合
    # condition5 = ~(
    #     (baselog["pase_category"].isin([1, 2, 3, 4])) & 
    #     (baselog["race_position"].isin([1, 2, 3, 4])) & 
    #     (baselog["show"] == 1)
    # )
    # baselog.loc[condition5 & (baselog["show"] == 1), "score_pase_potision"] += (baselog["race_class"] + 1) / 3.6

    return baselog
result_df1 = calculate_scores(baselog)


grouped_df = result_df1.groupby(["race_id", "horse_id"])
merged_df1 = population.copy()

for n_race in tqdm(n_races, desc=f"agg_pase_per_score"):
    # スコアの集計（標準化しない版）
    raw_df = (
        grouped_df.head(n_race)
        .groupby(["race_id", "horse_id"])[["score_pase_potision"]]
        .agg(["mean", "max", "min"])
    )
    raw_df.columns = [
        "_".join(col) + f"_{n_race}races_per_score_raw" for col in raw_df.columns
    ]
    
    # merged_df に追加
    merged_df1 = merged_df1.merge(raw_df, on=["race_id", "horse_id"], how="left")


"""
特筆見る

直線が短い差しは不利
長いと先行不利、というわけではない


"""


baselog = (
    population
    .merge(
        horse_results,
        on="horse_id",
        suffixes=("", "_horse"),
    )
    .merge(
        race_info[["race_id", "goal_range_100"]], 
        on="race_id",
    )
    .query("date_horse < date")
    .sort_values("date_horse", ascending=False)
)
def calculate_scores(baselog):
    # スコア初期化
    baselog["score_goal_range"] = 0.0

    # 直線が短いで差し・追い込み馬がshowに入った場合
    condition1 = (baselog["goal_range_100"] < 3) & (baselog["race_position"].isin([3, 4])) & (baselog["show"] == 1)
    baselog.loc[condition1, "score_goal_range"] += (baselog["race_class"] + 1) / 2

    # 直線が短いで差し・追い込み馬がshowに入った場合
    condition2 = (baselog["goal_range_100"] < 4) & (baselog["goal_range_100"] >= 3) & (baselog["race_position"].isin([3, 4])) & (baselog["show"] == 1)
    baselog.loc[condition2, "score_goal_range"] +=(baselog["race_class"] + 1) / 2.7

    # 直線が短いで差し・追い込み馬がshowに入った場合
    condition3 = (baselog["goal_range_100"] < 4.2) & (baselog["goal_range_100"] >= 4) & (baselog["race_position"].isin([3, 4])) & (baselog["show"] == 1)
    baselog.loc[condition3, "score_goal_range"] +=(baselog["race_class"] + 1) / 3.5

    # 先行馬がshowに入った場合
    condition4 = (baselog["goal_range_100"] >= 0) & (baselog["race_position"].isin([1, 2])) & (baselog["show"] == 1)
    baselog.loc[condition4, "score_goal_range"] += (baselog["race_class"] + 1) / 4.5

    # 直線が短いで差し・追い込み馬がshowに入った場合
    condition5 = (baselog["goal_range_100"] >= 4.2) & (baselog["race_position"].isin([3, 4])) & (baselog["show"] == 1)
    baselog.loc[condition5, "score_goal_range"] +=(baselog["race_class"] + 1) / 4.5

     # showに入りらなかった場合 
    condition7 = (baselog["show"] != 1)
    baselog.loc[condition7, "score_goal_range"] += 0 

    
    return baselog
result_df2 = calculate_scores(baselog)

grouped_df = result_df2.groupby(["race_id", "horse_id"])
merged_df2 = population.copy()

for n_race in tqdm(n_races, desc=f"agg_goal_per_score"):
    # スコアの集計（標準化しない版）
    raw_df = (
        grouped_df.head(n_race)
        .groupby(["race_id", "horse_id"])[["score_goal_range"]]
        .agg(["mean", "max", "min"])
    )
    raw_df.columns = [
        "_".join(col) + f"_{n_race}races_per_goal_raw" for col in raw_df.columns
    ]

    # merged_df に追加
    merged_df2 = merged_df2.merge(raw_df, on=["race_id", "horse_id"], how="left")




"""
特筆見る

急カーブは小回りよりさらにきつい
小回りで買った差し？　差し不利
小スパイラル　小回りほどではないが、先行有利
スパイラル　小回りほどではないが、先行有利

大回り・複合で勝った先行？　先行不利

"急": 1,
"小回": 2,
"小スパ": 3,
"スパ": 4,
"複合": 5,
None: None 
"""


baselog = (
    population
    .merge(
        horse_results,
        on="horse_id",
        suffixes=("", "_horse"),
    )
    .merge(
        race_info[["race_id","curve"]], 
        on="race_id",
    )
    .query("date_horse < date")
    .sort_values("date_horse", ascending=False)
)
def calculate_scores(baselog):
    # スコア初期化
    baselog["score_curve_range"] = 0.0

    # # 急カーブで差し・追い込み馬がshowに入った場合
    # condition1 = (baselog["curve"] == 1) & (baselog["race_position"].isin([3, 4])) & (baselog["show"] == 1)
    # baselog.loc[condition1, "score_curve_range"] += (baselog["race_class"] + 1)

    # # 小回りカーブで差し・追い込み馬がshowに入った場合
    # condition2 = (baselog["curve"] == 2) & (baselog["race_position"].isin([3, 4])) & (baselog["show"] == 1)
    # baselog.loc[condition2, "score_curve_range"] += (baselog["race_class"] + 1) / 2
    
    # # 小スパカーブで差し・追い込み馬がshowに入った場合
    # condition3 = (baselog["curve"] == 3) & (baselog["race_position"].isin([3, 4])) & (baselog["show"] == 1)
    # baselog.loc[condition3, "score_curve_range"] += (baselog["race_class"] + 1) / 2.3

    # # スパカーブで差し・追い込み馬がshowに入った場合
    # condition4 = (baselog["curve"] == 4) & (baselog["race_position"].isin([3, 4])) & (baselog["show"] == 1)
    # baselog.loc[condition4, "score_curve_range"] += (baselog["race_class"] + 1) / 2.6

    # # 複合カーブで差し・追い込み馬がshowに入った場合
    # condition5 = (baselog["curve"] == 5) & (baselog["race_position"].isin([3, 4])) & (baselog["show"] == 1)
    # baselog.loc[condition5, "score_curve_range"] += (baselog["race_class"] + 1)/ 5   
    
    # # 複合カーブで先行がshowに入った場合
    # condition6 = (baselog["curve"] == 5) & (baselog["race_position"].isin([1, 2])) & (baselog["show"] == 1)
    # baselog.loc[condition6, "score_curve_range"] += (baselog["race_class"] + 1)   
    
    # # カーブで先行がshowに入った場合
    # condition8 = (baselog["curve"].isin([1, 2,3,4])) & (baselog["race_position"].isin([1, 2])) & (baselog["show"] == 1)
    # baselog.loc[condition8, "score_curve_range"] += (baselog["race_class"] + 1)/  5

    
    # 急カーブで差し・追い込み馬がshowに入った場合
    condition1 = (baselog["curve"] == 1) & (baselog["race_position"].isin([3, 4])) & (baselog["show"] == 1)
    baselog.loc[condition1, "score_curve_range"] += (baselog["race_class"] + 1) / 1.8

    # 小回りカーブで差し・追い込み馬がshowに入った場合
    condition2 = (baselog["curve"] == 2) & (baselog["race_position"].isin([3, 4])) & (baselog["show"] == 1)
    baselog.loc[condition2, "score_curve_range"] += (baselog["race_class"] + 1) / 2.1
    
    # 小スパカーブで差し・追い込み馬がshowに入った場合
    condition3 = (baselog["curve"] == 3) & (baselog["race_position"].isin([3, 4])) & (baselog["show"] == 1)
    baselog.loc[condition3, "score_curve_range"] += (baselog["race_class"] + 1) / 2.4

    # スパカーブで差し・追い込み馬がshowに入った場合
    condition4 = (baselog["curve"] == 4) & (baselog["race_position"].isin([3, 4])) & (baselog["show"] == 1)
    baselog.loc[condition4, "score_curve_range"] += (baselog["race_class"] + 1) / 2.7

    # 複合カーブで差し・追い込み馬がshowに入った場合
    condition5 = (baselog["curve"] == 5) & (baselog["race_position"].isin([3, 4])) & (baselog["show"] == 1)
    baselog.loc[condition5, "score_curve_range"] += (baselog["race_class"] + 1)/ 5   
    
    # 複合カーブで先行がshowに入った場合
    condition6 = (baselog["curve"] == 5) & (baselog["race_position"].isin([1, 2])) & (baselog["show"] == 1)
    baselog.loc[condition6, "score_curve_range"] += (baselog["race_class"] + 1)/ 1.8    
    
    # カーブで先行がshowに入った場合
    condition8 = (baselog["curve"].isin([1, 2,3,4])) & (baselog["race_position"].isin([1, 2])) & (baselog["show"] == 1)
    baselog.loc[condition8, "score_curve_range"] += (baselog["race_class"] + 1)/  5

    
     # showに入りらなかった場合 
    condition7 = (baselog["show"] != 1)
    baselog.loc[condition7, "score_curve_range"] += 0 
    
     # showに入りらなかった場合 
    condition9 = (~baselog["curve"].isin([1, 2, 3, 4])) & (baselog["show"] == 1)
    baselog.loc[condition9, "score_curve_range"] += 0 
    

    return baselog
result_df3 = calculate_scores(baselog)

grouped_df = result_df3.groupby(["race_id", "horse_id"])
merged_df3 = population.copy()

for n_race in tqdm(n_races, desc=f"agg_curve_per_score"):
    # スコアの集計（標準化しない版）
    raw_df = (
        grouped_df.head(n_race)
        .groupby(["race_id", "horse_id"])[["score_curve_range"]]
        .agg(["mean", "max", "min"])
    )
    raw_df.columns = [
        "_".join(col) + f"_{n_race}races_per_curve_raw" for col in raw_df.columns
    ]
    
    # merged_df に追加
    merged_df3 = merged_df3.merge(raw_df, on=["race_id", "horse_id"], how="left")



"""
特筆見る

坂のあるところ
あるなら差しが有利、ただしペースが早いと先行有利
ないならどっちも
だから、
坂があってスローペースや普通のとき、差しが有利、先行評価
坂があって、ハイペースのとき、先行有利、差し評価
ゆるさかは/1.5
ないなら同評価
"平坦": 1,        
"緩坂": 2,
"急坂": 3
"""


baselog = (
    population
    .merge(
        horse_results,
        on="horse_id",
        suffixes=("", "_horse"),
    )
    .merge(
        race_info[["race_id","goal_slope"]], 
        on="race_id",
    )
    .query("date_horse < date")
    .sort_values("date_horse", ascending=False)
)
def calculate_scores(baselog):
    # スコア初期化
    baselog["score_goal_slope"] = 0.0


    # 平坦の時、特に評価なし
    condition1 = (baselog["goal_slope"] == 1) & (baselog["race_position"].isin([1,2,3, 4])) & (baselog["show"] == 1)
    baselog.loc[condition1, "score_goal_slope"] += (baselog["race_class"] + 1) /5

    # ゆる坂ペースが早いとき
    condition2 = (baselog["goal_slope"] == 2) & (baselog["race_position"].isin([3, 4])) & baselog["pase_category"].isin([4]) & (baselog["show"] == 1)
    baselog.loc[condition2, "score_goal_slope"] += (baselog["race_class"] + 1) / 2.2
    
    # ゆる坂ペースが遅いとき
    condition3 = (baselog["goal_slope"] == 2) & (baselog["race_position"].isin([1,2])) & baselog["pase_category"].isin([1]) & (baselog["show"] == 1)
    baselog.loc[condition3, "score_goal_slope"] += (baselog["race_class"] + 1) / 2.2

    # ゆる坂ペースが早いとき
    condition4 = (baselog["goal_slope"] == 2) & (baselog["race_position"].isin([3, 4])) & baselog["pase_category"].isin([3]) & (baselog["show"] == 1)
    baselog.loc[condition4, "score_goal_slope"] += (baselog["race_class"] + 1) / 3.0
    
    # ゆる坂ペースが遅いとき
    condition5 = (baselog["goal_slope"] == 2) & (baselog["race_position"].isin([1,2])) & baselog["pase_category"].isin([2]) & (baselog["show"] == 1)
    baselog.loc[condition5, "score_goal_slope"] += (baselog["race_class"] + 1) / 3.0


    # ゆる坂ペースそれ以外
    condition6 = (baselog["goal_slope"] == 2) & (baselog["race_position"].isin([1,2])) & baselog["pase_category"].isin([3,4]) & (baselog["show"] == 1)
    baselog.loc[condition6, "score_goal_slope"] += (baselog["race_class"] + 1) /4.5

    # ゆる坂ペースそれ以外
    condition7 = (baselog["goal_slope"] == 2) & (baselog["race_position"].isin([3, 4])) & baselog["pase_category"].isin([1,2]) & (baselog["show"] == 1)
    baselog.loc[condition7, "score_goal_slope"] += (baselog["race_class"] + 1) /4.5
    
    
    
    # 厳しい坂でペースが早いとき
    condition8 = (baselog["goal_slope"] == 3) & (baselog["race_position"].isin([3, 4])) & baselog["pase_category"].isin([4]) & (baselog["show"] == 1)
    baselog.loc[condition8, "score_goal_slope"] += (baselog["race_class"] + 1) /1.5
    
    # 厳しい坂ペースが遅いとき
    condition9 = (baselog["goal_slope"] == 3) & (baselog["race_position"].isin([1,2])) & baselog["pase_category"].isin([1]) & (baselog["show"] == 1)
    baselog.loc[condition9, "score_goal_slope"] += (baselog["race_class"] + 1) /1.5

    # 厳しい坂でペースが早いとき
    condition10 = (baselog["goal_slope"] == 3) & (baselog["race_position"].isin([3, 4])) & baselog["pase_category"].isin([3]) & (baselog["show"] == 1)
    baselog.loc[condition10, "score_goal_slope"] += (baselog["race_class"] + 1) / 2.0
    
    # 厳しい坂ペースが遅いとき
    condition11 = (baselog["goal_slope"] == 3) & (baselog["race_position"].isin([1,2])) & baselog["pase_category"].isin([2]) & (baselog["show"] == 1)
    baselog.loc[condition11, "score_goal_slope"] += (baselog["race_class"] + 1) / 2.0

    
    # 厳しい坂ペースそれ以外
    condition12 = (baselog["goal_slope"] == 3) & (baselog["race_position"].isin([1,2])) & baselog["pase_category"].isin([3,4]) & (baselog["show"] == 1)
    baselog.loc[condition12, "score_goal_slope"] += (baselog["race_class"] + 1) / 3.0

    # 厳しい坂ペースそれ以外
    condition13 = (baselog["goal_slope"] == 3) & (baselog["race_position"].isin([3, 4])) & baselog["pase_category"].isin([1,2]) & (baselog["show"] == 1)
    baselog.loc[condition13, "score_goal_slope"] += (baselog["race_class"] + 1) / 3.0

    

     # showに入りらなかった場合 
    condition14 = (baselog["show"] != 1)
    baselog.loc[condition14, "score_goal_slope"] += 0 
    
     # showに入りらなかった場合 
    condition15 = (~baselog["goal_slope"].isin([1, 2, 3])) & (baselog["show"] == 1)
    baselog.loc[condition15, "score_goal_slope"] += 0 
    

    return baselog
result_df4 = calculate_scores(baselog)



    
grouped_df = result_df4.groupby(["race_id", "horse_id"])
merged_df4 = population.copy()

for n_race in tqdm(n_races, desc=f"agg_goal_slope_per_score"):
    # スコアの集計（標準化しない版）
    raw_df = (
        grouped_df.head(n_race)
        .groupby(["race_id", "horse_id"])[["score_goal_slope"]]
        .agg(["mean", "max", "min"])
    )
    raw_df.columns = [
        "_".join(col) + f"_{n_race}races_per_goal_slope_raw" for col in raw_df.columns
    ]
    

    # merged_df に追加
    merged_df4 = merged_df4.merge(raw_df, on=["race_id", "horse_id"], how="left")




"""
特筆見る

スタミナ指標
かっているとして
長ければ+
小回りで-
大回りでさらに+
⑤大回りのコースは、カーブで息が入りにくいので、距離に不安のある馬には厳しい


race_class*コースの長さ/2000

こいつは相対値以外も出していい

"""


baselog = (
    population
    .merge(
        horse_results,
        on="horse_id",
        suffixes=("", "_horse"),
    )
    .merge(
        race_info[["race_id","curve",'place_season_condition_type_categori']], 
        on="race_id",
    )
    .query("date_horse < date")
    .sort_values("date_horse", ascending=False)
)
def calculate_scores(baselog):
    # スコア初期化
    baselog["score_stamina"] = 0.0


    # 急カーブ場合
    condition1 = (baselog["curve"] == 1) & (baselog["show"] == 1)
    baselog.loc[condition1, "score_stamina"] += (baselog["race_class"] + 1+(2/baselog['place_season_condition_type_categori'])) *baselog["course_len"]/2000/ 2.5

    # 小回りカーブで場合
    condition2 = (baselog["curve"] == 2)  & (baselog["show"] == 1)
    baselog.loc[condition2, "score_stamina"] += (baselog["race_class"] + 1+(2/baselog['place_season_condition_type_categori'])) *baselog["course_len"]/2000/ 2.1
    
    # 小スパカーブ場合
    condition3 = (baselog["curve"] == 3) & (baselog["show"] == 1)
    baselog.loc[condition3, "score_stamina"] += (baselog["race_class"] + 1+(2/baselog['place_season_condition_type_categori'])) *baselog["course_len"]/2000/ 1.8

    # スパカーブ場合
    condition4 = (baselog["curve"] == 4)  & (baselog["show"] == 1)
    baselog.loc[condition4, "score_stamina"] += (baselog["race_class"] + 1+(2/baselog['place_season_condition_type_categori'])) *baselog["course_len"]/2000/ 1.5

    # 複合カーブで場合
    condition5 = (baselog["curve"] == 5)  & (baselog["show"] == 1)
    baselog.loc[condition5, "score_stamina"] += (baselog["race_class"] + 1+(2/baselog['place_season_condition_type_categori']))*baselog["course_len"]/2000
    
    
     # showに入りらなかった場合 
    condition7 = (baselog["show"] != 1)
    baselog.loc[condition7, "score_stamina"] += 0 
    
     # showに入りらなかった場合 
    condition8 = (~baselog["curve"].isin([1, 2, 3, 4])) & (baselog["show"] == 1)
    baselog.loc[condition8, "score_stamina"] += 0 
    

    return baselog
result_df5 = calculate_scores(baselog)

grouped_df = result_df5.groupby(["race_id", "horse_id"])
merged_df5 = population.copy()

for n_race in tqdm(n_races, desc=f"agg_stamina_per_score"):
    # スコアの集計（標準化しない版）
    raw_df = (
        grouped_df.head(n_race)
        .groupby(["race_id", "horse_id"])[["score_stamina"]]
        .agg(["mean", "max", "min"])
    )
    raw_df.columns = [
        "_".join(col) + f"_{n_race}races_per_stamina_raw" for col in raw_df.columns
    ]
    
  
    # merged_df に追加
    merged_df5 = merged_df5.merge(raw_df, on=["race_id", "horse_id"], how="left")
    

merged_df5 = merged_df5.merge(race_info[["race_id","course_len"]], on="race_id")
merged_df5["course_len_relative"] = (merged_df5["course_len"] / 2000) -1
merged_df5.loc[merged_df5["course_len_relative"] < 0, "course_len_relative"] = 0
merged_df5["course_len_relative"] = merged_df5["course_len_relative"] * 2.2
# 対象の列名リスト
columns_to_multiply = [
    'score_stamina_mean_1races_per_stamina_raw',
    'score_stamina_max_1races_per_stamina_raw',
    'score_stamina_min_1races_per_stamina_raw',

    'score_stamina_mean_3races_per_stamina_raw',
    'score_stamina_max_3races_per_stamina_raw',
    'score_stamina_min_3races_per_stamina_raw',

    'score_stamina_mean_5races_per_stamina_raw',
    'score_stamina_max_5races_per_stamina_raw',
    'score_stamina_min_5races_per_stamina_raw',

    'score_stamina_mean_10races_per_stamina_raw',
    'score_stamina_max_10races_per_stamina_raw',
    'score_stamina_min_10races_per_stamina_raw',

]

# 'course_len_relative'を各列に掛け算
for col in columns_to_multiply:
    merged_df5[col] = merged_df5[col] * merged_df5['course_len_relative']
# 'course_len'と'course_len_relative'の列を削除
merged_df5 = merged_df5.drop(columns=['course_len', 'course_len_relative'])


results_df = (
    merged_df1
    .merge(
        merged_df2,
        on=("race_id","date","horse_id"),
    )
    .merge(
        merged_df3,
        on=("race_id","date","horse_id"),
    )
    .merge(
        merged_df4,
        on=("race_id","date","horse_id"),
    )
    .merge(
        merged_df5,
        on=("race_id","date","horse_id"),
    )
)




# 合計列を作成するレース数のリスト
n_races = [1, 3, 5, 10]

# "mean", "min", "max" のそれぞれに対して処理
for agg_type in ["mean", "min", "max"]:
    for n_race in n_races:
        # 対象となる列をフィルタリング
        target_columns = [
            col for col in results_df.columns if f"{agg_type}_{n_race}races" in col
        ]
        
        # 合計列を作成
        results_df[f"{agg_type}_{n_race}races_sum"] = results_df[target_columns].sum(axis=1)
# 'column1' と 'column2' を削除
results_df = results_df.drop(columns=["date",'score_pase_potision_mean_1races_per_score_raw', 'score_pase_potision_max_1races_per_score_raw', 'score_pase_potision_min_1races_per_score_raw', 'score_pase_potision_mean_3races_per_score_raw', 'score_pase_potision_max_3races_per_score_raw', 'score_pase_potision_min_3races_per_score_raw', 'score_pase_potision_mean_5races_per_score_raw', 'score_pase_potision_max_5races_per_score_raw', 'score_pase_potision_min_5races_per_score_raw', 'score_pase_potision_mean_10races_per_score_raw', 'score_pase_potision_max_10races_per_score_raw', 'score_pase_potision_min_10races_per_score_raw', 'score_goal_range_mean_1races_per_goal_raw', 'score_goal_range_max_1races_per_goal_raw', 'score_goal_range_min_1races_per_goal_raw', 'score_goal_range_mean_3races_per_goal_raw', 'score_goal_range_max_3races_per_goal_raw', 'score_goal_range_min_3races_per_goal_raw', 'score_goal_range_mean_5races_per_goal_raw', 'score_goal_range_max_5races_per_goal_raw', 'score_goal_range_min_5races_per_goal_raw', 'score_goal_range_mean_10races_per_goal_raw', 'score_goal_range_max_10races_per_goal_raw', 'score_goal_range_min_10races_per_goal_raw', 'score_curve_range_mean_1races_per_curve_raw', 'score_curve_range_max_1races_per_curve_raw', 'score_curve_range_min_1races_per_curve_raw', 'score_curve_range_mean_3races_per_curve_raw', 'score_curve_range_max_3races_per_curve_raw', 'score_curve_range_min_3races_per_curve_raw', 'score_curve_range_mean_5races_per_curve_raw', 'score_curve_range_max_5races_per_curve_raw', 'score_curve_range_min_5races_per_curve_raw', 'score_curve_range_mean_10races_per_curve_raw', 'score_curve_range_max_10races_per_curve_raw', 'score_curve_range_min_10races_per_curve_raw', 'score_goal_slope_mean_1races_per_goal_slope_raw', 'score_goal_slope_max_1races_per_goal_slope_raw', 'score_goal_slope_min_1races_per_goal_slope_raw', 'score_goal_slope_mean_3races_per_goal_slope_raw', 'score_goal_slope_max_3races_per_goal_slope_raw', 'score_goal_slope_min_3races_per_goal_slope_raw', 'score_goal_slope_mean_5races_per_goal_slope_raw', 'score_goal_slope_max_5races_per_goal_slope_raw', 'score_goal_slope_min_5races_per_goal_slope_raw', 'score_goal_slope_mean_10races_per_goal_slope_raw', 'score_goal_slope_max_10races_per_goal_slope_raw', 'score_goal_slope_min_10races_per_goal_slope_raw', 'score_stamina_mean_1races_per_stamina_raw', 'score_stamina_max_1races_per_stamina_raw', 'score_stamina_min_1races_per_stamina_raw', 'score_stamina_mean_3races_per_stamina_raw', 'score_stamina_max_3races_per_stamina_raw', 'score_stamina_min_3races_per_stamina_raw', 'score_stamina_mean_5races_per_stamina_raw', 'score_stamina_max_5races_per_stamina_raw', 'score_stamina_min_5races_per_stamina_raw', 'score_stamina_mean_10races_per_stamina_raw', 'score_stamina_max_10races_per_stamina_raw', 'score_stamina_min_10races_per_stamina_raw'])



# population をコピーして merged_df を作成
merged_df = population.copy()


# スコアの集計（標準化）
tmp_df = results_df.groupby(["race_id"])

# 標準化されたデータを計算
std_df = (
    (results_df.set_index(["race_id", "horse_id"]) - tmp_df.mean()) / tmp_df.std()
).add_suffix("_relative").reset_index()

# merged_df に追加
merged_df = merged_df.merge(results_df, on=["race_id", "horse_id"], how="left")
merged_df = merged_df.merge(std_df, on=["race_id", "horse_id"], how="left")

# 結果を確認
merged_df = merged_df.drop(columns="horse_id_relative")

agg_pase_per_score:   0%|          | 0/4 [00:00<?, ?it/s]

agg_goal_per_score:   0%|          | 0/4 [00:00<?, ?it/s]

agg_curve_per_score:   0%|          | 0/4 [00:00<?, ?it/s]

agg_goal_slope_per_score:   0%|          | 0/4 [00:00<?, ?it/s]

agg_stamina_per_score:   0%|          | 0/4 [00:00<?, ?it/s]

In [48]:
merged_df

,race_id,date,horse_id,mean_1races_sum,mean_3races_sum,mean_5races_sum,mean_10races_sum,min_1races_sum,min_3races_sum,min_5races_sum,min_10races_sum,max_1races_sum,max_3races_sum,max_5races_sum,max_10races_sum,max_10races_sum_relative,max_1races_sum_relative,max_3races_sum_relative,max_5races_sum_relative,mean_10races_sum_relative,mean_1races_sum_relative,mean_3races_sum_relative,mean_5races_sum_relative,min_10races_sum_relative,min_1races_sum_relative,min_3races_sum_relative,min_5races_sum_relative
0,202306010101,2023-01-05,2020103575,0.555556,0.851852,0.638889,0.638889,0.555556,0.0,0.0,0.0,0.555556,2.000000,2.000000,2.000000,2.508836,0.698212,2.508836,2.508836,3.132054,0.698212,2.529227,2.795223,NaN,0.698212,NaN,NaN
1,202306010101,2023-01-05,2020107073,0.555556,0.185185,0.185185,0.185185,0.555556,0.0,0.0,0.0,0.555556,0.555556,0.555556,0.555556,0.396132,0.698212,0.396132,0.396132,0.615037,0.698212,0.231380,0.511982,NaN,0.698212,NaN,NaN
2,202306010101,2023-01-05,2020102562,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,-0.416447,-0.375960,-0.416447,-0.416447,-0.412318,-0.375960,-0.406910,-0.419953,NaN,-0.375960,NaN,NaN
3,202306010101,2023-01-05,2020106345,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,-0.416447,-0.375960,-0.416447,-0.416447,-0.412318,-0.375960,-0.406910,-0.419953,NaN,-0.375960,NaN,NaN
4,202306010101,2023-01-05,2020100039,2.000000,0.851852,0.511111,0.365079,2.000000,0.0,0.0,0.0,2.000000,2.000000,2.000000,2.000000,2.508836,3.491060,2.508836,2.508836,1.613038,3.491060,2.529227,2.152188,NaN,3.491060,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46207,202309050912,2023-12-28,2017104873,0.000000,0.000000,0.000000,0.951852,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,9.518519,-0.299303,-0.738333,-0.933850,-1.052632,-1.170109,-0.738333,-0.859591,-0.948433,NaN,-0.738333,-0.357874,-0.25
46208,202309050912,2023-12-28,2019100653,0.000000,0.000000,0.000000,1.266667,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,12.666667,0.581380,-0.738333,-0.933850,-1.052632,-0.917826,-0.738333,-0.859591,-0.948433,NaN,-0.738333,-0.357874,-0.25
46209,202309050912,2023-12-28,2017106240,7.777778,2.592593,1.555556,2.686667,7.777778,0.0,0.0,0.0,7.777778,7.777778,7.777778,12.888889,0.643546,1.054152,0.537538,0.328462,0.220122,1.054152,0.037374,-0.214145,NaN,1.054152,-0.357874,-0.25
46210,202309050912,2023-12-28,2018103205,0.000000,0.000000,2.284444,2.973333,0.000000,0.0,0.0,0.0,0.000000,0.000000,11.422222,13.111111,0.705712,-0.738333,-0.933850,0.975603,0.449848,-0.738333,-0.859591,0.129921,NaN,-0.738333,-0.357874,-0.25


In [ ]:
#まとめて実行
# レース結果テーブルの前処理
results_preprocessed = preprocessing.process_results()
# 馬の過去成績テーブルの加工
horse_results_preprocessed = preprocessing.process_horse_results()
# レース情報テーブルの前処理
race_info_preprocessed = preprocessing.process_race_info()
# 騎手リーディングテーブルの前処理
jockey_leading_preprocessed = preprocessing.process_jockey_leading()
# 調教師リーディングテーブルの前処理
trainer_leading_preprocessed = preprocessing.process_trainer_leading()
# 血統テーブルの前処理
peds_preprocessed = preprocessing.process_peds()
# 種牡馬リーディングテーブルの前処理
sire_leading_preprocessed = preprocessing.process_sire_leading()

In [124]:
race_info
race_info2 = race_info[race_info["course_len"] == 2]
race_info2

,race_id,date,race_type,around,course_len,weather,ground_state,race_class,place,season,place_season,place_season_type,kaisai_race,kaisai_race_type,day,day_type,kaisai_count,kaisai_count_type,place_season_day_type,day_condition,place_season_condition_type,place_season_condition_type_categori,place_course_category,place_course_tough,goal_range,curve,goal_slope,place_season_type_course_len,lap_type,race_day_count,race_date_day_count
2793,202306050111,2023-12-02,1,0,2,1,0,7,6,4,64,641,11,111,1,11,5,51,64011,1,6411,3,6,1,-10000,-10000,-10000,6412,-1,11,73836611


In [126]:
horse_results
horse_results2= horse_results[horse_results["course_len"] == 2]
horse_results2

,horse_id,rank_per_horse,date,rank,prize,rank_diff,umaban,wakuban,weather,race_type,course_len,ground_state,race_class,n_horses,time,time_courselen,nobori,win,rentai,show,place,corner_1_per_horse,corner_2_per_horse,corner_3_per_horse,corner_4_per_horse,corner_1,corner_2,corner_3,corner_4,pase_1,pase_2
